<a href="https://colab.research.google.com/github/CiciDu/testrepo/blob/main/multifirefly_stage8_forPoster%26PPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this stage, I decide to speed up some of the codes and improve the codes' readability by eliminating unnecessary cells and adding comments

## General setups

note: on_off_lines have the maximum distance of 400, while connect_ff_path has the maximum distance of 250

In [ ]:
# for monkey
NEW_DATASET = False
MONKEY_DATA = True
NO_PLOT_NEEDED = True
data_folder_name = "0219"
data_num = 19

In [ ]:
# for agent
NEW_DATASET = True
MONKEY_DATA = False
NO_PLOT_NEEDED = True
data_folder_name = "LSTM_July_29"
data_num = 721
trial_total_num = 30

In [ ]:
import numpy as np
np.random.seed(7777)
#rng = np.random.default_rng(2021) 
NEW_DATASET = True
MONKEY_DATA = True
SHOW_INTERESTING = False
trial_total_num = 20
list_of_colors = ["navy", "magenta", "white", "gray", "brown", "black"] # For plotting ff clusters
point_index_array = np.arange(1,100,10)

### Connect to Google Drive

In [ ]:
! pip install google.colab
from google.colab import drive
drive.mount('/content/gdrive')

### unzip data

In [ ]:
!pip install neo
import neo
import os.path
from os import path

#!unzip gdrive/MyDrive/behavioural_data.zip
!unzip gdrive/MyDrive/fireflies_data/0219.zip
#!unzip gdrive/MyDrive/fireflies_data/0220.zip
#!unzip gdrive/MyDrive/fireflies_data/0221.zip
data_folder_name = "0219"
data_num = 19
#seg_reader = neo.io.Spike2IO(filename="/content/behavioural_data/m51s26.smr").read_segment()
seg_reader = neo.io.Spike2IO(filename="/content/0219/m51c0936.smr").read_segment()

     |████████████████████████████████| 639 kB 4.8 MB/s 
     |████████████████████████████████| 85 kB 20.5 MB/s 
  Created wheel for quantities: filename=quantities-0.13.0-py3-none-any.whl size=77858 sha256=518040d62252c17712a36d5cfd9f4d4bbfc86a46c2a7f8db97c0a9979fb19a10
  Stored in directory: /Users/dusiyi/Library/Caches/pip/wheels/a2/54/36/12af6e58a292a912f4bcd62d146d593fadcd414a258a970cf2
Successfully built quantities
unzip:  cannot find or open gdrive/MyDrive/fireflies_data/0219.zip, gdrive/MyDrive/fireflies_data/0219.zip.zip or gdrive/MyDrive/fireflies_data/0219.zip.ZIP.


FileNotFoundError: ignored

### Import packages

In [ ]:
import os.path
from os import path

import numpy as np
import matplotlib


matplotlib.rcParams.update(matplotlib.rcParamsDefault)

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import ListedColormap
from IPython.display import HTML
import pandas as pd
import math
import collections
import re
import os, sys
from contextlib import contextmanager
from scipy.signal import decimate
from matplotlib import rc, cm
from sklearn.linear_model import LinearRegression
import torch
from math import pi
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import seaborn as sns

!pip install matplotlib-scalebar
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.ticker as ticker

from numpy import linalg as LA
import csv 
from scipy.cluster.hierarchy import linkage
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
from random import randint


class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as mtick

### Plots

In [ ]:
def plt_config(title=None, xlim=None, ylim=None, xlabel=None, ylabel=None, colorbar=False, sci=False):
    for field in ['title', 'xlim', 'ylim', 'xlabel', 'ylabel']:
        if eval(field) != None: getattr(plt, field)(eval(field))
    if isinstance(sci, str): plt.ticklabel_format(style='sci', axis=sci, scilimits=(0,0))
    if isinstance(colorbar,str): plt.colorbar(label=colorbar)
    elif colorbar: plt.colorbar(label = '$Number\ of\ Entries$')

@contextmanager
def initiate_plot(dimx=24, dimy=9, dpi=100, fontweight='normal'):
    plt.rcParams['figure.figsize'] = (dimx, dimy)
    plt.rcParams['font.weight'] = fontweight
    plt.rcParams['mathtext.default'] = 'regular'
    plt.rcParams["font.family"] = 'Arial'
    global fig; fig = plt.figure(dpi=dpi)
    yield
    plt.show() 

### Animation

In [ ]:
rc('animation', html='jshtml')
matplotlib.rcParams['animation.embed_limit'] = 2**128

### smr_extractor

In [ ]:
class smr_extractor(object):   
    def __init__(self):
        self.folder_path = data_folder_name
        self.files_names = [file for file in os.listdir(self.folder_path) if 'smr' in file]
        self.full_path_file_names = [os.path.join(self.folder_path,self.files_names[i]) 
                                     for i,value in enumerate(self.files_names)] # a list contains 2 file path in total
    def extract_data(self):
        Channel_signal_output = []
        marker_list = []
        
        for index, file_name in enumerate(self.full_path_file_names):  # loop 2 files one by one
            seg_reader = neo.io.Spike2IO(filename=file_name).read_segment() # read file
            
            if index == 0: # get sampling rate, only need to get it once
                smr_sampling_rate = seg_reader.analogsignals[0].sampling_rate 
                
            analog_length = min([i.size for i in seg_reader.analogsignals]) # in case analog channels have different shape
            
            Channel_index = [] # create an empty list to store channel names

            for C_index, C_data in enumerate(seg_reader.analogsignals[:-1]): # -1 indicates we disgard 'Raw' channel   
                shape = seg_reader.analogsignals[C_index].shape[1] # See how many channels are contained in each element of the list
                if C_index==0:
                  Channel_signal = C_data.as_array()[:analog_length,];
                else:
                  Channel_signal = np.append(Channel_signal, C_data.as_array()[:analog_length,], axis=1)
                for i in range(shape):
                  Channel_index.append(seg_reader.analogsignals[C_index].name) # get channel name one by one and put in Channel_index

            Channel_signal = np.append(Channel_signal, np.asarray(seg_reader.analogsignals[0].times[:analog_length,]).reshape(analog_length,1), axis=1)# get time stamps and put in Channel_signal
            Channel_index.append('Time') 

            Channel_signal_output.append(pd.DataFrame(Channel_signal,columns=Channel_index))

            marker_channel_index = [index for index,value in enumerate(seg_reader.events) if value.name == 'marker'][0] #find 'marker' channel
            marker_labels = seg_reader.events[marker_channel_index].get_labels().astype('int') # get 'marker' labels
            marker_values = seg_reader.events[marker_channel_index].as_array() # get 'marker' values
            marker = {'labels': marker_labels, 'values': marker_values} # arrange labels and values in a dict
            marker_list.append(marker)
            
        return Channel_signal_output, marker_list, smr_sampling_rate

### log_extractor

In [ ]:
# Read log file here
class log_extractor(object):
    def __init__(self):
        self.folder_path = data_folder_name
        #self.files_names = [file for file in os.listdir(self.folder_path) if 'txt' and '0' in file]
        self.files_names = ("m51s936.txt",)
        self.full_path_file_names = os.path.join(self.folder_path,self.files_names[0])
        
    def extract_data(self):
        ffLinenumberList = []
        ff_information = []
        ffname_index = 0
        
        with open(self.full_path_file_names,'r',encoding='UTF-8') as content:
            log_content = content.readlines()
         
        for LineNumber, line in enumerate(log_content):
            key_ff = re.search('Firefly', line)
            key_monkey = re.search('Monkey', line)
            if key_ff is not None:
                ffLinenumberList.append(LineNumber)
            if key_monkey is not None:
                monkeyLineNum = LineNumber

        # get ff data
        for index, LineNumber in enumerate(ffLinenumberList):
            FF_Catched_T = []
            FF_Position = []
            FF_believed_position = []
            FF_flash_T = []
            
            if index == len(ffLinenumberList)-1:
                log_content_block = log_content[ffLinenumberList[index]+1:monkeyLineNum]
            else:
                log_content_block = log_content[ffLinenumberList[index]+1:ffLinenumberList[index+1]]
            
            for line in log_content_block:
                if len(line.split(' ')) == 5:
                    if 'inf' not in line:
                        FF_Catched_T.append(float(line.split(' ')[0]))
                        FF_Position.append([float(line.split(' ')[1]),float(line.split(' ')[2])])
                        FF_believed_position.append([float(line.split(' ')[3]),float(line.split(' ')[4])])
                else:
                    try:
                        FF_flash_T.append([float(line.split(' ')[0]),float(line.split(' ')[1])])
                    except:
                        1
                        
            FF_flash_T = np.array(FF_flash_T)
            seperated_ff = np.digitize(FF_flash_T.T[0],FF_Catched_T)
            for j in np.unique(seperated_ff)[:-1]:
                
                ff_information.append({'ff_index':ffname_index,'ff_catched_T':FF_Catched_T[j],'ff_real_position': np.array(FF_Position[j]),
                                  'ff_believed_position': np.array(FF_believed_position[j]),
                                      'ff_flash_T': FF_flash_T[seperated_ff==j]})  
                
                ffname_index = ffname_index+1    
                
        #get monkey data
        Monkey_X = []
        Monkey_Y = []
        Monkey_Position_T = []
        
        for line in log_content[monkeyLineNum+1:]:
            Monkey_X.append(float(line.split(' ')[0]))
            Monkey_Y.append(float(line.split(' ')[1]))
            Monkey_Position_T.append(float(line.split(' ')[2]))
            
        monkey_information = {'monkey_x': np.array(Monkey_X), 'monkey_y': np.array(Monkey_Y), 'monkey_t': np.array(Monkey_Position_T)}
        
        return ff_information, monkey_information
        #self.ff_information = ff_information
        #self.monkey_information = monkey_information

### monkey_smr


In [ ]:
Channel_signal_output,marker_list,smr_sampling_rate = smr_extractor().extract_data()
#Considering the first smr file, use marker_list[0], Channel_signal_output[0]
juice_timestamp = marker_list[0]['values'][marker_list[0]['labels']==4]
Channel_signal_smr1 = Channel_signal_output[0]
Channel_signal_smr1['section'] = np.digitize(Channel_signal_smr1.Time,juice_timestamp) # seperate analog signal by juice timestamps
# Remove tail of analog data
Channel_signal_smr1 = Channel_signal_smr1[Channel_signal_smr1['section']<Channel_signal_smr1['section'].unique()[-1]]
Channel_signal_smr1['Time'].iloc[-1] = juice_timestamp[-1]
# Remove head of analog data
Channel_signal_smr1 = Channel_signal_smr1[Channel_signal_smr1['Time']>marker_list[0]['values'][marker_list[0]['labels']==1][0]]


monkey_smr_dataframe = Channel_signal_smr1[["Time", "Signal stream 1", "Signal stream 2", "Signal stream 3", "Signal stream 10"]].reset_index(drop=True)
monkey_smr_dataframe.columns = ['monkey_t', 'monkey_x', 'monkey_y', 'monkey_speed', 'AngularV']
monkey_smr = dict(zip(monkey_smr_dataframe.columns.tolist(), np.array(monkey_smr_dataframe.values.T.tolist())))

### Data from monkey

ff & monkey information


speed of monkey = delta_position/t
delta_position = sqrt(delta_x^2 + delta_y^2)

Here we can only assume that the trajectory between two time points 
is straight.

To calculate speed_i, we use the information at t_i and t_i+1

In [ ]:
# for monkey
NEW_DATASET = False
MONKEY_DATA = True
NO_PLOT_NEEDED = True
data_folder_name = "0219"
data_num = 19

ff_information,monkey_information = log_extractor().extract_data()
#sort ff by catched time
ff_index = []
ff_catched_T = []
ff_real_position = []
ff_believed_position = []
ff_life = []
ff_flash = []
ff_flash_end = []  # This is the time that the firefly last stops flash
for item in ff_information:
    item['Life'] = np.array([item['ff_flash_T'][0][0],item['ff_catched_T']])
    ff_index = np.hstack((ff_index,item['ff_index']))
    ff_catched_T = np.hstack((ff_catched_T,item['ff_catched_T']))
    ff_real_position.append(item['ff_real_position'])
    ff_believed_position.append(item['ff_believed_position'])
    ff_life.append(item['Life'])
    ff_flash.append(item['ff_flash_T'])
    ff_flash_end.append(item['ff_flash_T'][-1][-1])
sort_index = np.argsort(ff_catched_T)
ff_index_sorted = ff_index[sort_index]
ff_catched_T_sorted = ff_catched_T[sort_index]
ff_real_position_sorted = np.array(ff_real_position)[sort_index]
ff_believed_position_sorted = np.array(ff_believed_position)[sort_index]
ff_life_sorted = np.array(ff_life)[sort_index]
ff_flash_sorted  = np.array(ff_flash, dtype=object)[sort_index].tolist()
ff_flash_end_sorted = np.array(ff_flash_end)[sort_index]

# Use accurate juice timestamps, ff_catched_T_sorted for smr1 (so that the time frame is correct)
ff_catched_T_sorted = ff_catched_T_sorted[:np.where(ff_catched_T_sorted<=Channel_signal_smr1.Time.values[-1])[0][-1]+1]

catched_ff_num = len(ff_catched_T_sorted) - 200  
total_ff_num = len(ff_life_sorted)
M_catched_ff_num = len(ff_catched_T_sorted) - 200
M_total_ff_num = len(ff_life_sorted)


M_ff_catched_T_sorted = ff_catched_T_sorted.copy()
M_ff_real_position_sorted = ff_real_position_sorted.copy()
M_ff_life_sorted = ff_life_sorted.copy()
M_ff_flash_sorted = ff_flash_sorted.copy()
M_ff_believed_position_sorted = ff_believed_position_sorted.copy()

#################################################################################################################################



delta_time = np.diff(monkey_information['monkey_t'])
delta_x = np.diff(monkey_information['monkey_x'])
delta_y = np.diff(monkey_information['monkey_y'])
delta_position = np.sqrt(np.square(delta_x)+np.square(delta_y))
monkey_speed = np.divide(delta_position, delta_time)
monkey_speed = np.append(np.array([0]), monkey_speed)

# If the monkey's speed at one point exceeds 200 
#(this can happen when the monkey reaches the boundary and comes out at another place)
# we replace it with the previous speed.
while np.where(monkey_speed>=200)[0].size > 0:
  index = np.where(monkey_speed>=200)[0]
  monkey_speed1 = np.append(np.array([0]), monkey_speed)
  monkey_speed[index] = monkey_speed1[index]
monkey_information['monkey_speed'] = monkey_speed
monkey_information['monkey_speeddummy'] = (monkey_information['monkey_speed']> 0.01).astype(int) 
monkey_speeddummy = (monkey_speed>0.1).astype(int) 


# Add angle of the monkey

angle = [90]  # The monkey is at 90 degree angle at the beginning
current_angle = 90 # This keeps track of the current angle during the iterations
# Find the time in the data that is closest (right before) the time where we wan to know the monkey's angular position.
for i in range(1, len(monkey_information['monkey_t'])):
  # If the monkey basically stopped at this moment, we keep the previous angle
  if monkey_information['monkey_speed'][i-1] < 1:  # use i-1 because 'monkey_speed' has 1 less element
    angle.append(current_angle)   
  else:
    myradians = math.atan2(monkey_information['monkey_y'][i]-monkey_information['monkey_y'][i-1], monkey_information['monkey_x'][i]-monkey_information['monkey_x'][i-1])
    mydegrees = math.degrees(myradians)
    # Compare the new angle with the previous angle
    # If the different is too large, then the monkey might be going backward, and we will just subtract 180 from the angle
    if 170 < abs(mydegrees-current_angle)%360 < 190 :
      current_angle = current_angle - 180
      if current_angle < -180:
        current_angle += 360
      angle.append(current_angle)
    else: #else, we keep the new angle
      current_angle = mydegrees
      angle.append(current_angle)
angle = np.array(angle)
monkey_information['monkey_angle'] = angle*pi/180

delta_time = np.delete(monkey_information['monkey_t'], 0) - np.delete(monkey_information['monkey_t'], -1)
delta_angle = np.delete(monkey_information['monkey_angle'], 0) - np.delete(monkey_information['monkey_angle'], -1)
monkey_dw = np.append(np.array([0]), np.divide(delta_angle, delta_time))
monkey_information['monkey_dw'] = monkey_dw

monkey_dw = np.divide(np.diff(np.array(monkey_information['monkey_angle']), prepend=monkey_information['monkey_angle'][0]), np.append(delta_time[0], delta_time))
monkey_information['monkey_dw'] = monkey_dw

M_monkey_information = monkey_information.copy()


## Functions

### find_intersection
source: https://codereview.stackexchange.com/questions/203468/find-the-intervals-which-have-a-non-empty-intersection-with-a-given-interval

In [ ]:
def find_intersection(intervals, query):
    """Find intersections between intervals.
    Intervals are open and are represented as pairs (lower bound,
    upper bound).

    Arguments:
    intervals: array_like, shape=(N, 2) -- Array of intervals.
    query: array_like, shape=(2,) -- Interval to query.

    Returns:
    Array of indexes of intervals that overlap with query.

    """
    intervals = np.asarray(intervals)
    lower, upper = query
    return np.where((lower < intervals[:, 1]) & (intervals[:, 0] < upper))[0]

### flashing_ff (based on trial)
Find the indices of the fireflies that have flashed during the trial

In [ ]:
def flashing_ff(ff_flash_sorted, duration):
  ## Find the index of the fireflies that have flashed during the trial
  ## Input: ff_flash_sorted contains the time that each firefly flashes on and off
  ##        duration is the duration of the trial
  ## Output: flash_index contains the indices of the fireflies that have flashed during the trial (among all fireflies)
  flash_index = []
  for index in range(total_ff_num):
    ff = ff_flash_sorted[index]
    if len(find_intersection(ff, duration)) > 0:
      flash_index.append(index)
  return flash_index

###flash_on_ff (dict, based on points)
Find the fireflies that are visible at each time point (for animation)

In [ ]:
# Create a dictionary of {time: [indices of fireflies that are visible], ...}
def flash_on_ff(anim_t, currentTrial, num_trials, ff_flash_sorted):
  alive_ff_during_this_trial = np.where((ff_life_sorted[:,1] > ff_catched_T_sorted[currentTrial-num_trials])\
                                        & (ff_life_sorted[:,0] < ff_catched_T_sorted[currentTrial]))[0]
  flash_on_ff_dict={}
  for time in anim_t:
    visible_ff_indices = [index for index in alive_ff_during_this_trial \
     if len(np.where(np.logical_and(ff_flash_sorted[index][:,0] <= time, \
                                       ff_flash_sorted[index][:,1] >= time))[0]) > 0]
    flash_on_ff_dict[time] = visible_ff_indices
  return flash_on_ff_dict

# To call:
# flash_on_ff_dict = flash_on_ff(anim_t, currentTrial, num_trials, ff_flash_sorted)

### flash_starttime
Find the earliest time that each firefly begins to flash during that trial

In [ ]:
def flash_starttime(flash_index, duration):
  # Find the earliest time that each firefly begins to flash during that trial
  firefly_on = []
  for index in flash_index:
    ff = ff_flash_sorted[index]
    overlapped_intervals = find_intersection(ff, duration)
    first_start_time = ff[overlapped_intervals].flatten()[0]
    if first_start_time >= duration[0]:
      firefly_on.append(first_start_time)
    else:
      firefly_on.append(duration[0])
  return firefly_on

In [ ]:
# Another method
'''
def flash_starttime(flash_index, duration):
  # Find the earliest time that each firefly begins to flash during that trial
  firefly_on = []
  for index in flash_index:
    ff = ff_flash_sorted[index]
    ff_all_starttime = ff[:,0]
    start_time_index = np.argmax(ff_all_starttime >= duration[0])
    start_time = ff_all_starttime[start_time_index]
    # Consider extreme cases
    if start_time >= duration[1]:
      start_time = max(ff_all_starttime[start_time_index-1], duration[0])
    elif ff[start_time_index-1][1] >= duration[0]:
      start_time = duration[0]
    firefly_on.append(start_time)
  firefly_on = np.array(firefly_on)
  return firefly_on
  '''

###believed_ff (updated version)
Match the believed positions of the fireflies to the time when they are captured (for animation)

In [ ]:
# Create a dictionary of {time: [[believed_ff_position], [believed_ff_position2], ...], ...}
def believed_ff(anim_t, currentTrial, num_trials, ff_believed_position_sorted, ff_catched_T_sorted):
  believed_ff_dict={}
  # For each time point:
  for index in range(len(anim_t)):
    time = anim_t[index]
    believed_ff_indices = [ff_believed_position_sorted[ff] for ff in range(currentTrial, currentTrial+num_trials) if time > ff_catched_T_sorted[ff]]
    believed_ff_dict[index] = believed_ff_indices

  # # The last point
  # believed_ff_indices = [(ff_believed_position_sorted[ff]) for ff in range(currentTrial-num_trials+1, currentTrial+1)]
  # believed_ff_dict[len(anim_t)-1] = believed_ff_indices

  return believed_ff_dict

# To call:
# believed_ff_dict = believed_ff(anim_t, currentTrial, num_trials, ff_believed_position_sorted, ff_catched_T_sorted)

### distance_traveled
Find the length of the trajectory run by the monkey in this duration

In [ ]:
def distance_traveled(currentTrial):
  duration = [ff_catched_T_sorted[currentTrial-1], ff_catched_T_sorted[currentTrial]]
  cum_indices = np.where((monkey_information['monkey_t'] >= duration[0]) & (monkey_information['monkey_t'] <= duration[1]))[0]
  if len(cum_indices) > 5:
    cum_t, cum_angle = monkey_information['monkey_t'][cum_indices],  monkey_information['monkey_angle'][cum_indices]
    cum_mx, cum_my = monkey_information['monkey_x'][cum_indices], monkey_information['monkey_y'][cum_indices] 
    cum_speed = monkey_information['monkey_speed'][cum_indices]
    distance = np.sum((cum_t[1:] - cum_t[:-1])*cum_speed)
  return distance

# To Run:
# distance = distance_traveled(currentTrial)

###abs_displacement
Find the absolute displacement between the target for the currentTrial and the target for currentTrial.
Return 9999 if the monkey has hit the border at one point.

In [ ]:
def abs_displacement(currentTrial):
  duration = [ff_catched_T_sorted[currentTrial-1], ff_catched_T_sorted[currentTrial]]
  displacement = 0
  cum_indices = np.where((monkey_information['monkey_t'] >= duration[0]) & (monkey_information['monkey_t'] <= duration[1]))[0]
  if len(cum_indices) > 5:
    cum_t, cum_angle = monkey_information['monkey_t'][cum_indices],  monkey_information['monkey_angle'][cum_indices]
    cum_mx, cum_my = monkey_information['monkey_x'][cum_indices], monkey_information['monkey_y'][cum_indices] 
    flag = True
    # If the monkey has hit the boundary
    if np.any(cum_mx[1:]-cum_mx[:-1] > 10) or np.any(cum_my[1:]-cum_my[:-1] > 10):
      displacement = 9999
    else:
      displacement = LA.norm(ff_believed_position_sorted[currentTrial]-ff_believed_position_sorted[currentTrial-1])
  return displacement

# To call:
# displacement = abs_displacement(currentTrial):

### num_of_stops
Find the stops the monkey made between currentTrial and currentTrial + 1


In [ ]:
if MONKEY_DATA == True:
  def num_of_stops(currentTrial):
    duration = [ff_catched_T_sorted[currentTrial-1], ff_catched_T_sorted[currentTrial]]
    cum_indices = np.where((monkey_information['monkey_t'] > duration[0]) & (monkey_information['monkey_t'] <= duration[1]))[0]
    if len(cum_indices) > 5:
      cum_t, cum_angle = monkey_information['monkey_t'][cum_indices],  monkey_information['monkey_angle'][cum_indices]
      cum_mx, cum_my = monkey_information['monkey_x'][cum_indices], monkey_information['monkey_y'][cum_indices] 
      cum_speeddummy = monkey_information['monkey_speeddummy'][cum_indices]
      zerospeed_index = np.where(cum_speeddummy==0)[0]
      if len(zerospeed_index) > 0 :
        zerospeedx, zerospeedy = cum_mx[zerospeed_index], cum_my[zerospeed_index]
        zerospeedindex = cum_indices[zerospeed_index]
        stop0 = np.array(list(zip(zerospeedx,zerospeedy)))
        _, stops_index = np.unique(stop0, axis=0, return_index=True)
        stops = stop0[stops_index[np.argsort(stops_index)]]
        stop_indices = zerospeedindex[stops_index[np.argsort(stops_index)]]
        distinct_stops = [stops[0]] + [stops[i+1] for i in range(len(stops)-1) if LA.norm(np.array((stops[i+1][0]-stops[i][0], stops[i+1][1]-stops[i][1]))) > 0.6]
      else:
        distinct_stops = []
    else:
      distinct_stops = []
    return distinct_stops
else:
  def num_of_stops(currentTrial):
    duration = [ff_catched_T_sorted[currentTrial-1], ff_catched_T_sorted[currentTrial]]
    cum_indices = np.where((monkey_information['monkey_t'] > duration[0]) & (monkey_information['monkey_t'] <= duration[1]))[0]
    if len(cum_indices) > 5:
      cum_mx, cum_my = monkey_information['monkey_x'][cum_indices], monkey_information['monkey_y'][cum_indices] 
      cum_speeddummy = monkey_information['monkey_speeddummy'][cum_indices]
      zerospeed_index = np.where(cum_speeddummy==0)[0]
      if len(zerospeed_index) > 0 :
        zerospeedx, zerospeedy = cum_mx[zerospeed_index], cum_my[zerospeed_index]
        zerospeedindex = cum_indices[zerospeed_index]
        stop0 = np.array(list(zip(zerospeedx,zerospeedy)))
        _, stops_index = np.unique(stop0, axis=0, return_index=True)
        stops = stop0[stops_index[np.argsort(stops_index)]]
        stop_indices = zerospeedindex[stops_index[np.argsort(stops_index)]]
        distinct_stops = stops
      else:
        distinct_stops = []
    else:
      distinct_stops = []
    return distinct_stops

# To call:
# distinct_stops = num_of_stops(currentTrial)

### num_of_stops_indices
Find the stops the monkey made between currentTrial and currentTrial + 1 and return the indices

In [ ]:
if MONKEY_DATA == True:
  def num_of_stops_indices(currentTrial):
    duration = [ff_catched_T_sorted[currentTrial-1], ff_catched_T_sorted[currentTrial]]
    distinct_stops_indices = []
    cum_indices = np.where((monkey_information['monkey_t'] > duration[0]) & 
                              (monkey_information['monkey_t'] <= duration[1]))[0]
    if len(cum_indices) > 5:
      cum_t, cum_angle = monkey_information['monkey_t'][cum_indices],  monkey_information['monkey_angle'][cum_indices]
      cum_mx, cum_my = monkey_information['monkey_x'][cum_indices], monkey_information['monkey_y'][cum_indices] 
      cum_speeddummy = monkey_information['monkey_speeddummy'][cum_indices]
      zerospeed_index = np.where(cum_speeddummy==0)[0]
      if len(zerospeed_index) > 0 :
        zerospeedx, zerospeedy = cum_mx[zerospeed_index], cum_my[zerospeed_index]
        zerospeedindex = cum_indices[zerospeed_index]
        stop0 = np.array(list(zip(zerospeedx,zerospeedy)))
        _, stops_index = np.unique(stop0, axis=0, return_index=True)
        stops = stop0[stops_index[np.argsort(stops_index)]]
        stop_indices = zerospeedindex[stops_index[np.argsort(stops_index)]]
        if len(stops) > 1: 
          distinct_stops_indices = [stop_indices[0]] + [stop_indices[i+1] for i in range(len(stops)-1) if LA.norm(np.array((stops[i+1][0]-stops[i][0], stops[i+1][1]-stops[i][1]))) > 0.6]
    return distinct_stops_indices
else:
  def num_of_stops_indices(currentTrial):
    duration = [ff_catched_T_sorted[currentTrial-1], ff_catched_T_sorted[currentTrial]]
    distinct_stops_indices = []
    cum_indices = np.where((monkey_information['monkey_t'] > duration[0]) & 
                              (monkey_information['monkey_t'] <= duration[1]))[0]
    if len(cum_indices) > 5:
      cum_mx, cum_my = monkey_information['monkey_x'][cum_indices], monkey_information['monkey_y'][cum_indices] 
      cum_speeddummy = monkey_information['monkey_speeddummy'][cum_indices]
      zerospeed_index = np.where(cum_speeddummy==0)[0]
      if len(zerospeed_index) > 0 :
        zerospeedx, zerospeedy = cum_mx[zerospeed_index], cum_my[zerospeed_index]
        zerospeedindex = cum_indices[zerospeed_index]
        stop0 = np.array(list(zip(zerospeedx,zerospeedy)))
        _, stops_index = np.unique(stop0, axis=0, return_index=True)
        stops = stop0[stops_index[np.argsort(stops_index)]]
        stop_indices = zerospeedindex[stops_index[np.argsort(stops_index)]]
      return stop_indices
# To call:
# distinct_stops = num_of_stops(currentTrial)

### num_of_stops_since_target_last_seen

In [ ]:
def num_of_stops_since_target_last_seen(currentTrial):
  duration = [ff_catched_T_sorted[currentTrial]-t_last_visible[currentTrial-1], ff_catched_T_sorted[currentTrial]]
  if t_last_visible[currentTrial-1] > 50:
    distinct_stops = []
    return distinct_stops
  cum_indices = np.where((monkey_information['monkey_t'] > duration[0]) & (monkey_information['monkey_t'] <= duration[1]))[0]
  if len(cum_indices) > 5:
    cum_t = monkey_information['monkey_t'][cum_indices]
    cum_mx, cum_my = monkey_information['monkey_x'][cum_indices], monkey_information['monkey_y'][cum_indices] 
    cum_speeddummy = monkey_information['monkey_speeddummy'][cum_indices]
    zerospeed_index = np.where(cum_speeddummy==0)[0]
    if len(zerospeed_index) > 0 :
      zerospeedx, zerospeedy = cum_mx[zerospeed_index], cum_my[zerospeed_index]
      zerospeedindex = cum_indices[zerospeed_index]
      stop0 = np.array(list(zip(zerospeedx,zerospeedy)))
      _, stops_index = np.unique(stop0, axis=0, return_index=True)
      stops = stop0[stops_index[np.argsort(stops_index)]]
      stop_indices = zerospeedindex[stops_index[np.argsort(stops_index)]]
      distinct_stops = [stops[0]] + [stops[i+1] for i in range(len(stops)-1) if LA.norm(np.array((stops[i+1][0]-stops[i][0], stops[i+1][1]-stops[i][1]))) > 0.6]
    else:
      distinct_stops = []
  else:
    distinct_stops = []
  return distinct_stops

# To call:
# distinct_stops = num_of_stops(currentTrial)

### find_clusters
Assign each stop with a number that indicates which cluster it belongs to

Note: the algorithm is slightly different from previous notebooks

In [ ]:
def find_clusters(currentTrial, distance_between_points):
  distinct_stops = num_of_stops(currentTrial)
  if len(distinct_stops) == 0: 
    clusters = []
  else:
    distinct_stops2 = np.array([[stop[0], stop[1]] for stop in distinct_stops])
    current_cluster = 1
    clusters = [1]
    for i in range(1, len(distinct_stops2)):
      flag = False
      if LA.norm(distinct_stops2[i]-distinct_stops2[i-1]) < distance_between_points:
          clusters.append(current_cluster)
      else: # Create a new cluster
        current_cluster = current_cluster+1
        clusters.append(current_cluster)
  return clusters

### angle2ff
Calculate the angle from the monkey to the firefly (from the monkey's perspective) at a given time

In [ ]:
def angle2ff(firefly_xy, time):
  num_i = np.where(monkey_information['monkey_t'] == time)[0]
  if len(num_i) == 0:
    time_before = np.where(monkey_information['monkey_t'] <= time)[0]
    if len(time_before) > 0:
      num_i = time_before[-1]
    else:
      num_i = 0

  # Calculate the angle from the monkey to the firefly (overhead view)
  ffradians = math.atan2(firefly_xy[1]-monkey_information['monkey_y'][num_i], firefly_xy[0]-monkey_information['monkey_x'][num_i])
  ffdegrees = math.degrees(ffradians)

  # Find the angle of the firefly from the monkey's perspective
  ff_angle = ffdegrees-monkey_information['monkey_angle'][num_i]
  while abs(ff_angle) > 180:
    if ff_angle > 180:
      ff_angle = ff_angle-360
    elif ff_angle < -180:
      ff_angle = ff_angle + 360
  return ff_angle
# to call:
# ff_angle = angle2ff(firefly_xy, time)

### connect_path_ff
Find the lines to be drawn between the nearby fireflies and the path based on the starting time to flash (excluding the target)

In [ ]:
def connect_path_ff(cum_t, cum_mx, cum_my, cum_angle, currentTrial, max_distance, ff_flash_sorted, ff_real_position_sorted, total_ff_num):
  x = []
  y = []

  if len(cum_t) > 5: # When cum_t is not empty
    # For each firefly
    total_ff_array = np.arange(total_ff_num)
    non_targets_ff_array = np.delete(total_ff_array, currentTrial)
    for i in non_targets_ff_array:
      # For each duration of being visible
      overlapped_intervals = ff_flash_sorted[i][find_intersection(ff_flash_sorted[i], [cum_t[0], cum_t[-1]])]
      for interval in overlapped_intervals:
        overlapped_indices = np.where((cum_t >= interval[0]) & (cum_t <= interval[1]))[0]
        distances_to_monkey = LA.norm(ff_real_position_sorted[i] - np.stack([cum_mx[overlapped_indices], cum_my[overlapped_indices]], axis=1), axis=1)
        valid_distance_indices = overlapped_indices[np.where(distances_to_monkey < max_distance)[0]]
        
        if len(valid_distance_indices) > 0:
          angles_to_monkey = np.arctan2(ff_real_position_sorted[i,1]-cum_my[valid_distance_indices], \
                                        ff_real_position_sorted[i,0]-cum_mx[valid_distance_indices])-cum_angle[valid_distance_indices]
          ## The following lines turn out to be unnecessary because of the range of output for np.arctan2
          angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
          angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi

          angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(25, np.maximum(distances_to_monkey[np.where(distances_to_monkey < max_distance)[0]], 25) ))) # use torch clip to get valid arcsin input
          angles_adjusted = np.clip(angles_adjusted, 0, pi)
          angles_adjusted = np.sign(angles_to_monkey)* angles_adjusted


          overall_valid_indices = valid_distance_indices[np.where(np.absolute(angles_adjusted) <= 2*pi/9)[0]]
          
          x = x + [[ff_real_position_sorted[i][0]] + [cum_mx[index]] for index in overall_valid_indices]
          y = y + [[ff_real_position_sorted[i][1]] + [cum_my[index]] for index in overall_valid_indices]
  return x, y
# To call:
# x, y = connect_path_ff(cum_t, cum_mx, cum_my, cum_angle, currentTrial, 250, ff_flash_sorted, ff_real_position_sorted, total_ff_num) total_ff_num)

### connect_path_ff2
This deals with the case that there are multiple targets to avoid

In [ ]:
def connect_path_ff2(cum_t, cum_mx, cum_my, cum_angle, target_nums, max_distance, ff_flash_sorted, ff_real_position_sorted, total_ff_num):
  x = []
  y = []
  if len(cum_t) > 5: # When cum_t is not empty
    # For each firefly
    total_ff_array = np.arange(total_ff_num)
    non_targets_ff_array = np.delete(total_ff_array, target_nums)
    for i in non_targets_ff_array:
      # For each duration of being visible
      overlapped_intervals = ff_flash_sorted[i][find_intersection(ff_flash_sorted[i], [cum_t[0], cum_t[-1]])]
      for interval in overlapped_intervals:
        overlapped_indices = np.where((cum_t >= interval[0]) & (cum_t <= interval[1]))[0]
        distances_to_monkey = LA.norm(ff_real_position_sorted[i] - np.stack([cum_mx[overlapped_indices], cum_my[overlapped_indices]], axis=1), axis=1)
        valid_distance_indices = overlapped_indices[np.where(distances_to_monkey < max_distance)[0]]
        
        if len(valid_distance_indices) > 0:
          angles_to_monkey = np.arctan2(ff_real_position_sorted[i,1]-cum_my[valid_distance_indices], \
                                        ff_real_position_sorted[i,0]-cum_mx[valid_distance_indices])-cum_angle[valid_distance_indices]
          ## The following lines turn out to be unnecessary because of the range of output for np.arctan2
          angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
          angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi

          angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(25, np.maximum(distances_to_monkey[np.where(distances_to_monkey < max_distance)[0]], 25) ))) # use torch clip to get valid arcsin input
          angles_adjusted = np.clip(angles_adjusted, 0, pi)
          angles_adjusted = np.sign(angles_to_monkey)* angles_adjusted


          overall_valid_indices = valid_distance_indices[np.where(np.absolute(angles_adjusted) <= 2*pi/9)[0]]
          
          x = x + [[ff_real_position_sorted[i][0]] + [cum_mx[index]] for index in overall_valid_indices]
          y = y + [[ff_real_position_sorted[i][1]] + [cum_my[index]] for index in overall_valid_indices]
  return x, y
# To call:
# x, y = connect_path_ff2(cum_t, cum_mx, cum_my, cum_angle, target_nums, 250, ff_flash_sorted, ff_real_position_sorted, total_ff_num)

### connect_path_ff_with_target
No longer exclude target

In [ ]:
def connect_path_ff_with_target(cum_t, cum_mx, cum_my, cum_angle, currentTrial, max_distance, ff_flash_sorted, ff_real_position_sorted, total_ff_num):
  x = []
  y = []

  if len(cum_t) > 5: # When cum_t is not empty
    # For each firefly
    for i in range(total_ff_num):
      # For each duration of being visible
      overlapped_intervals = ff_flash_sorted[i][find_intersection(ff_flash_sorted[i], [cum_t[0], cum_t[-1]])]
      for interval in overlapped_intervals:
        overlapped_indices = np.where((cum_t >= interval[0]) & (cum_t <= interval[1]))[0]
        distances_to_monkey = LA.norm(ff_real_position_sorted[i] - np.stack([cum_mx[overlapped_indices], cum_my[overlapped_indices]], axis=1), axis=1)
        valid_distance_indices = overlapped_indices[np.where(distances_to_monkey < max_distance)[0]]
        
        if len(valid_distance_indices) > 0:
          angles_to_monkey = np.arctan2(ff_real_position_sorted[i,1]-cum_my[valid_distance_indices], \
                                        ff_real_position_sorted[i,0]-cum_mx[valid_distance_indices])-cum_angle[valid_distance_indices]
          ## The following lines turn out to be unnecessary because of the range of output for np.arctan2
          angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
          angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi

          angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(25, np.maximum(distances_to_monkey[np.where(distances_to_monkey < max_distance)[0]], 25) ))) # use torch clip to get valid arcsin input
          angles_adjusted = np.clip(angles_adjusted, 0, pi)
          angles_adjusted = np.sign(angles_to_monkey)* angles_adjusted
          overall_valid_indices = valid_distance_indices[np.where(np.absolute(angles_adjusted) <= 2*pi/9)[0]]
          x = x + [[ff_real_position_sorted[i][0]] + [cum_mx[index]] for index in overall_valid_indices]
          y = y + [[ff_real_position_sorted[i][1]] + [cum_my[index]] for index in overall_valid_indices]
  return x, y
  # To call:
  # x, y = connect_path_ff(cum_t, cum_mx, cum_my, cum_angle, currentTrial, 250, ff_flash_sorted, ff_real_position_sorted, total_ff_num)

### onoff_lines
Find the parts of the path where the target has been on. The angle of the firefly is considered. The firefly is considered visible only when it's at the right angle.

In [ ]:
def onoff_lines(cum_t, cum_mx, cum_my, ff_real_position_sorted, currentTrial, duration):
  target_ff_flash = ff_flash_sorted[currentTrial]
  overlapped_intervals = target_ff_flash[find_intersection(target_ff_flash, duration)]
  x_onoff_lines=np.array([])
  y_onoff_lines=np.array([])
  for interval in overlapped_intervals:
    correspondingIndex_onPath = np.where((cum_t > np.maximum(interval[0], duration[0])) & \
                                          (cum_t < np.minimum(interval[1], duration[1])))[0]
    x_onoff_lines=np.append(x_onoff_lines, cum_mx[correspondingIndex_onPath])
    y_onoff_lines=np.append(y_onoff_lines, cum_my[correspondingIndex_onPath])
  return x_onoff_lines, y_onoff_lines

### onoff_lines_rightangle
Find the parts of the path where the target has been on. The angle of the firefly is considered. The firefly is considered visible only when it's at the right angle.

In [ ]:
def onoff_lines_rightangle(cum_t, cum_mx, cum_my, cum_angle, currentTrial, duration, ff_real_position_sorted, ff_flash_sorted):
  target_ff_flash = ff_flash_sorted[currentTrial]
  overlapped_intervals = target_ff_flash[find_intersection(target_ff_flash, duration)]

  indices_onoff_lines = []
  for interval in overlapped_intervals:
    correspondingIndex_onPath = np.where((cum_t > np.maximum(interval[0], duration[0])) & \
                                          (cum_t < np.minimum(interval[1], duration[1])))[0]
    distances_to_monkey = LA.norm(ff_real_position_sorted[currentTrial] - np.stack([cum_mx[correspondingIndex_onPath], cum_my[correspondingIndex_onPath]], axis=1), axis=1)
    angles_to_monkey = np.arctan2(ff_real_position_sorted[currentTrial,1]-cum_my[correspondingIndex_onPath], \
                                  ff_real_position_sorted[currentTrial,0]-cum_mx[correspondingIndex_onPath]) -cum_angle[correspondingIndex_onPath]                                 
    angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
    angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi
    angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(25, np.maximum(distances_to_monkey, 25) ))) # use torch clip to get valid arcsin input
    angles_adjusted = np.clip(angles_adjusted, 0, pi)
    angles_adjusted = np.sign(angles_to_monkey)* angles_adjusted

    correspondingIndex_onPath_angle = correspondingIndex_onPath[np.where(np.absolute(angles_adjusted) <= 2*pi/9)[0]]
    indices_onoff_lines = indices_onoff_lines + correspondingIndex_onPath_angle.tolist()
  return indices_onoff_lines

# To call:
# indices_onoff_lines = onoff_lines_rightangle(cum_t, cum_mx, cum_my, cum_angle, currentTrial, duration, ff_real_position_sorted, ff_flash_sorted)

### **PlotTrials** (for LSTM)

Among other things, here I eliminate the condition "ff_distance" < 250

In [ ]:
def PlotTrials(currentTrial,
                num_trials, 
                trail_color = "orange", # "orange" or "viridis" or None
                show_reward_boundary = False,
                show_stops = False,
                show_colorbar = False, 
                show_believed_target_positions = False,
                show_connect_path_target = np.array([]), # np.array([]) or target_nums
                show_connect_path_pre_target = np.array([]), # np.array([]) or target_nums
                show_connect_path_ff = np.array([]),
                trial_to_show_cluster = None, # None, 0, or -1
                show_scale_bar = False,
                trial_to_show_cluster_around_target = None,
                cluster_on_off_lines = False,
                show_start = True,
                #target_nums = "default", 
                ):
  


    cum_mxy_rotate = np.matmul(R, np.stack((cum_mx, cum_my)))
    if trail_color == "orange":
      axes.scatter(cum_mxy_rotate[0], cum_mxy_rotate[1],marker = 'o',s = 70, color="orange", zorder=2)
    elif trail_color == "viridis":
      axes.scatter(cum_mxy_rotate[0], cum_mxy_rotate[1],marker = 'o',s = 70, c = cum_speed, zorder=2)
    else:
      axes.scatter(cum_mxy_rotate[0], cum_mxy_rotate[1],marker = 'o',s = 70, color = "yellow", zorder=2)

    if show_start:
      # Plot the start
      axes.scatter(cum_mxy_rotate[0,0], cum_mxy_rotate[1,0],marker = '^',s = 220, color="gold", zorder=3, alpha=0.5)



    if show_stops:
      zerospeed_index = np.where(cum_speeddummy==0)
      zerospeedx, zerospeedy = cum_mx[zerospeed_index], cum_my[zerospeed_index]
      zerospeed_rotate = np.matmul(R, np.stack((zerospeedx, zerospeedy)))
      axes.scatter(zerospeed_rotate[0], zerospeed_rotate[1],marker = '*',s = 160, alpha = 0.7, color="black", zorder=2)

    ff_position_rotate = np.matmul(R, np.stack((ff_position_during_this_trial.T[0], ff_position_during_this_trial.T[1])))
    axes.scatter(ff_position_rotate[0], ff_position_rotate[1], marker='o', s=10, color="magenta",  zorder=2)

    if show_believed_target_positions:
      ff_believed_position_rotate = np.matmul(R, np.stack((ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[0], ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[1])))
      axes.scatter(ff_believed_position_rotate[0], ff_believed_position_rotate[1], marker = '*',s=185, color="red", alpha=0.75, zorder=2)
    
    if show_reward_boundary:
      for i in ff_position_rotate.T:
        circle2 = plt.Circle((i[0], i[1]), 25, facecolor='grey', edgecolor='orange', alpha=0.45, zorder=1)
        axes.add_patch(circle2)

    if trial_to_show_cluster != None:
    # Find the indices of ffs in the cluster
      cluster_indices = np.unique(cluster_dataframe_point[cluster_dataframe_point['target_index']==currentTrial+trial_to_show_cluster].ff_index.to_numpy())
      cluster_ff_positions = ff_real_position_sorted[np.array(cluster_indices)]
      cluster_ff_rotate = np.matmul(R, np.stack((cluster_ff_positions.T[0], cluster_ff_positions.T[1])))
      axes.scatter(cluster_ff_rotate[0], cluster_ff_rotate[1], marker='o', c = "blue", s=25, zorder = 4) 

    if show_connect_path_target.any():
      xy_onoff_lines = ff_dataframe.loc[ff_dataframe['target_index'].isin(show_connect_path_target)]
      xy_onoff_lines = np.array(xy_onoff_lines.loc[(xy_onoff_lines['ff_index']==currentTrial) & (xy_onoff_lines['visible']==1)][['monkey_x', 'monkey_y']])
      onoff_lines_rotate = np.matmul(R, xy_onoff_lines.T)
      axes.scatter(onoff_lines_rotate[0], onoff_lines_rotate[1], s=50, c="green", alpha=0.8, zorder=5) 

    if show_connect_path_pre_target.any():
      xy_onoff_lines = ff_dataframe.loc[ff_dataframe['target_index'].isin(show_connect_path_pre_target)]
      xy_onoff_lines = np.array(xy_onoff_lines.loc[(xy_onoff_lines['ff_index']==currentTrial-1) & (xy_onoff_lines['visible']==1)][['monkey_x', 'monkey_y']])
      onoff_lines_rotate = np.matmul(R, xy_onoff_lines.T)
      axes.scatter(onoff_lines_rotate[0], onoff_lines_rotate[1], s=65, c="aqua",  alpha=0.8, zorder=3)

    if show_connect_path_ff.any():
      temp_dataframe = ff_dataframe.loc[ff_dataframe['target_index'].isin(show_connect_path_ff)]
      temp_dataframe = temp_dataframe.loc[(temp_dataframe['visible']==1)][['ff_x', 'ff_y', 'monkey_x', 'monkey_y']]
      #temp_dataframe = temp_dataframe.loc[~temp_dataframe['ff_index'].isin(target_nums)]
      temp_array = temp_dataframe.to_numpy()
      temp_ff_positions = np.matmul(R, temp_array[:,:2].T)
      temp_monkey_positions = np.matmul(R, temp_array[:,2:].T)
      for j in range(len(temp_array)):
        axes.plot(np.stack([temp_ff_positions[0,j], temp_monkey_positions[0,j]]), np.stack([temp_ff_positions[1,j], temp_monkey_positions[1,j]]), '-', alpha=0.3, linewidth=1.5, c="#a940f5")
        #axes.plot(temp_ff_positions[0,j], temp_ff_positions[1,j], '-', alpha=0.2, marker="o", markersize=5, color="brown")

    # if show_connect_path_ff.any():
    #   if num_trials == 1:
    #     x, y = connect_path_ff(cum_t, cum_mx, cum_my, cum_angle, currentTrial, 400, ff_flash_sorted, ff_real_position_sorted, total_ff_num)
    #   else:
    #     target_nums = np.arange(currentTrial-num_trials+1, currentTrial+1)
    #     x, y = connect_path_ff2(cum_t, cum_mx, cum_my, cum_angle, target_nums, 400, ff_flash_sorted, ff_real_position_sorted, total_ff_num)
    #   for j in range(len(x)):
    #     xy_rotate = np.matmul(R, np.stack((x[j], y[j])))
    #     axes.plot(xy_rotate[0], xy_rotate[1], '-', alpha=0.2, c="#a940f5")
    #     axes.plot(xy_rotate[0][0], xy_rotate[1][0], '-', alpha=0.2, marker="o", markersize=5, color="brown")

    if trial_to_show_cluster_around_target != None:
      cluster_ff_pos = ffs_around_target_positions[currentTrial+trial_to_show_cluster_around_target]
      if len(cluster_ff_pos) > 0:
        ffs_around_target_rotate = np.matmul(R, np.stack((cluster_ff_pos.T[0], cluster_ff_pos.T[1])))
        axes.scatter(ffs_around_target_rotate[0],ffs_around_target_rotate[1], marker='o', s=30, color="blue", zorder=4) 
      if cluster_on_off_lines:
        # Find on_off_lines for ffs in the cluster
        for i in range(len(cluster_ff_pos)):
          index = np.array(ff_dataframe[(np.isclose(np.array(ff_dataframe['ff_x']),cluster_ff_pos[i, 0])) & (np.isclose(np.array(ff_dataframe['ff_y']),cluster_ff_pos[i, 1]))]['ff_index'])
          if len(index) > 0:
            index = index[0]
            #index = ffs_around_target_indices[currentTrial-trial_to_show_cluster_around_target][i]
            xy_onoff_lines = ff_dataframe.loc[(ff_dataframe['time']>= duration[0])&(ff_dataframe['time']<= duration[1])]
            xy_onoff_lines = xy_onoff_lines.loc[(xy_onoff_lines['ff_index']==index) & (xy_onoff_lines['visible']==1)]
            xy_onoff_lines2 = np.array(xy_onoff_lines[['monkey_x', 'monkey_y']])
            onoff_lines_rotate = np.matmul(R, xy_onoff_lines2.T)
            axes.scatter(onoff_lines_rotate[0], onoff_lines_rotate[1], s=80-10*i, color=list_of_colors[i], alpha=0.8, zorder=3+i) 
            # Use corresponding color for that ff
            xy_onoff_lines3 = np.array(xy_onoff_lines[['ff_x', 'ff_y']]) 
            ffs_around_target_rotate = np.matmul(R, xy_onoff_lines3.T)
            axes.scatter(ffs_around_target_rotate[0],ffs_around_target_rotate[1], marker='o', s=140, alpha = 0.8, color=list_of_colors[i], zorder=3) 


    
    
    
    if show_scale_bar:
      scale1 = ScaleBar(  
      dx=1, length_fraction=0.2, fixed_value=100,
          location='upper left',  # in relation to the whole plot
          label_loc='left', scale_loc='bottom'  # in relation to the line
      )
      axes.add_artist(scale1)
      axes.xaxis.set_major_locator(ticker.NullLocator())
      axes.yaxis.set_major_locator(ticker.NullLocator())

    xmin, xmax = np.min(cum_mxy_rotate[0]), np.max(cum_mxy_rotate[0])
    ymin, ymax = np.min(cum_mxy_rotate[1]), np.max(cum_mxy_rotate[1])
    bigger_width = max(xmax-xmin, ymax-ymin)
    xmiddle, ymiddle = (xmin+xmax)/2, (ymin+ymax)/2
    xmin, xmax = xmiddle-bigger_width/2, xmiddle+bigger_width/2
    ymin, ymax = ymiddle-bigger_width/2, ymiddle+bigger_width/2
    margin = max(bigger_width/5, 150)
    axes.set_xlim((xmin-margin, xmax+margin))
    axes.set_ylim((ymin-margin, ymax+margin))
    axes.set_aspect('equal')


    if show_colorbar == True:
      # Make the black and red colorbar
      A = np.reshape([1,2,1,2,2,2], (2,3))# The numbers don't matter much
      norm_bins = np.array([0.5, 1.5, 2.5])
      # Let's also design our color mapping: 1s should be plotted in blue, 2s in red, etc...
      col_dict={1:"black",2:"red"}
      # We create a colormar from our list of colors
      speed_cm = ListedColormap([col_dict[x] for x in col_dict.keys()])
      ## Make normalizer and formatter
      norm = matplotlib.colors.BoundaryNorm(norm_bins, 2, clip=True)
      labels = np.array(["No Reward", "Reward"])
      fmt = matplotlib.ticker.FuncFormatter(lambda x, pos: labels[norm(x)])
      # Plot our figure
      im = axes.imshow(A, cmap=speed_cm, extent=[0,0,0,0], norm=norm)
      cax2 = fig.add_axes([0.95, 0.15, 0.05, 0.2])
      cb = fig.colorbar(im, format=fmt, ticks=np.array([1., 2.]), cax=cax2)
      cb.ax.tick_params(width=0)
      cb.ax.set_title('Stopping Points', ha='left')
      if trail_color == "orange":
        A = np.reshape([1,2,1,2,2,2], (2,3))# The numbers don't matter much
        norm_bins = np.array([0.5, 1.5, 2.5])
        # Let's also design our color mapping: 1s should be plotted in blue, 2s in red, etc...
        col_dict={1:"green",2:"orange"}
        # We create a colormar from our list of colors
        speed_cm = ListedColormap([col_dict[x] for x in col_dict.keys()])
        ## Make normalizer and formatter
        norm = matplotlib.colors.BoundaryNorm(norm_bins, 2, clip=True)
        labels = np.array(["Top Target Visible", "Top Target Not Visible"])
        fmt = matplotlib.ticker.FuncFormatter(lambda x, pos: labels[norm(x)])
        # Plot our figure
        im = axes.imshow(A, cmap=speed_cm, extent=[0,0,0,0], norm=norm)
        cax2 = fig.add_axes([0.95, 0.5, 0.05, 0.2])
        cb = fig.colorbar(im, format=fmt, ticks=np.array([1., 2.]), cax=cax2)
        cb.ax.tick_params(width=0)
        cb.ax.set_title('Path Colors', ha='left', y=1.04)
      elif trail_color == "viridis":
        cmap = cm.viridis
        norm = matplotlib.colors.Normalize(vmin=0, vmax=200)
        cax = fig.add_axes([0.95, 0.4, 0.05, 0.43])
        cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),
                  cax=cax, orientation='vertical')
        cbar.ax.set_title('Speed(cm/s)', ha='left', y=1.04)
        cbar.ax.tick_params(axis='y', color='white', direction="in", right=True,length=5, width=1.5)
        cbar.outline.remove()
      global Show_Colorbar 
      Show_Colorbar = False


### **PlotTrials** 

In [ ]:
def PlotTrials(currentTrial,
                num_trials, 
                trail_color = "orange", # "orange" or "viridis" or None
                show_reward_boundary = False,
                show_stops = False,
                show_colorbar = False, 
                show_believed_target_positions = False,
                show_connect_path_target = np.array([]), # np.array([]) or target_nums
                show_connect_path_pre_target = np.array([]), # np.array([]) or target_nums
                show_connect_path_ff = np.array([]),
                trial_to_show_cluster = None, # None, 0, or -1
                show_scale_bar = False,
                show_start = False,
                trial_to_show_cluster_around_target = None,
                cluster_on_off_lines = False,
                #target_nums = "default", 
                ):
   

    cum_mxy_rotate = np.matmul(R, np.stack((cum_mx, cum_my)))
    if trail_color == "orange":
      axes.scatter(cum_mxy_rotate[0], cum_mxy_rotate[1],marker = 'o',s = 10, color="orange", zorder=2)
    elif trail_color == "viridis":
      axes.scatter(cum_mxy_rotate[0], cum_mxy_rotate[1],marker = 'o',s = 10, c = cum_speed, zorder=2)
    else:
      axes.scatter(cum_mxy_rotate[0], cum_mxy_rotate[1],marker = 'o',s = 10, color = "yellow", zorder=2)

    if show_start:
      # Plot the start
      axes.scatter(cum_mxy_rotate[0,0], cum_mxy_rotate[1,0],marker = 'o',s = 100, color="purple", zorder=3, alpha=0.5)

    if show_stops:
      zerospeed_index = np.where(cum_speeddummy==0)
      zerospeedx, zerospeedy = cum_mx[zerospeed_index], cum_my[zerospeed_index]
      zerospeed_rotate = np.matmul(R, np.stack((zerospeedx, zerospeedy)))
      axes.scatter(zerospeed_rotate[0], zerospeed_rotate[1],marker = '*',s = 150, color="black", zorder=2)

    ff_position_rotate = np.matmul(R, np.stack((ff_position_during_this_trial.T[0], ff_position_during_this_trial.T[1])))
    axes.scatter(ff_position_rotate[0], ff_position_rotate[1], marker='o', s=10, color="magenta",  zorder=2)

    if show_believed_target_positions:
      ff_believed_position_rotate = np.matmul(R, np.stack((ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[0], ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[1])))
      axes.scatter(ff_believed_position_rotate[0], ff_believed_position_rotate[1], marker = '*',s=120, color="red", alpha=0.75, zorder=2)
    
    if show_reward_boundary:
      for i in ff_position_rotate.T:
        circle2 = plt.Circle((i[0], i[1]), 25, facecolor='grey', edgecolor='orange', alpha=0.45, zorder=1)
        axes.add_patch(circle2)

    if trial_to_show_cluster != None:
    # Find the indices of ffs in the cluster
      cluster_indices = np.unique(cluster_dataframe_point[cluster_dataframe_point['target_index']==currentTrial+trial_to_show_cluster].ff_index.to_numpy())
      cluster_ff_positions = ff_real_position_sorted[np.array(cluster_indices)]
      cluster_ff_rotate = np.matmul(R, np.stack((cluster_ff_positions.T[0], cluster_ff_positions.T[1])))
      axes.scatter(cluster_ff_rotate[0], cluster_ff_rotate[1], marker='o', c = "blue", s=25, zorder = 4) 

    if show_connect_path_target.any():
      xy_onoff_lines = ff_dataframe.loc[ff_dataframe['target_index'].isin(show_connect_path_target)]
      xy_onoff_lines = np.array(xy_onoff_lines.loc[(xy_onoff_lines['ff_index']==currentTrial) & (xy_onoff_lines['visible']==1)&(xy_onoff_lines['ff_distance']<250)][['monkey_x', 'monkey_y']])
      onoff_lines_rotate = np.matmul(R, xy_onoff_lines.T)
      axes.scatter(onoff_lines_rotate[0], onoff_lines_rotate[1], s=30, c="green", alpha=0.4, zorder=4) 

    if show_connect_path_pre_target.any():
      xy_onoff_lines = ff_dataframe.loc[ff_dataframe['target_index'].isin(show_connect_path_pre_target)]
      xy_onoff_lines = np.array(xy_onoff_lines.loc[(xy_onoff_lines['ff_index']==currentTrial-1) & (xy_onoff_lines['visible']==1)&(xy_onoff_lines['ff_distance']<250)][['monkey_x', 'monkey_y']])
      onoff_lines_rotate = np.matmul(R, xy_onoff_lines.T)
      axes.scatter(onoff_lines_rotate[0], onoff_lines_rotate[1], s=40, c="aqua",  alpha=0.6, zorder=3)

    if show_connect_path_ff.any():
      target_nums = np.arange(currentTrial-num_trials+1, currentTrial+1)
      temp_dataframe = ff_dataframe.loc[ff_dataframe['target_index'].isin(target_nums)]
      temp_dataframe = temp_dataframe.loc[(temp_dataframe['ff_distance']<250) & (temp_dataframe['visible']==1)]
      temp_dataframe = temp_dataframe.loc[~temp_dataframe['ff_index'].isin(target_nums)][['ff_x', 'ff_y', 'monkey_x', 'monkey_y']]
      temp_array = temp_dataframe.to_numpy()
      temp_ff_positions = np.matmul(R, temp_array[:,:2].T)
      temp_monkey_positions = np.matmul(R, temp_array[:,2:].T)
      for j in range(len(temp_array)):
        axes.plot(np.stack([temp_ff_positions[0,j], temp_monkey_positions[0,j]]), np.stack([temp_ff_positions[1,j], temp_monkey_positions[1,j]]), '-', alpha=0.2, c="#a940f5")
        axes.plot(temp_ff_positions[0,j], temp_ff_positions[1,j], '-', alpha=0.2, marker="o", markersize=5, color="brown")

    # if show_connect_path_ff.any():
    #   if num_trials == 1:
    #     x, y = connect_path_ff(cum_t, cum_mx, cum_my, cum_angle, currentTrial, 250, ff_flash_sorted, ff_real_position_sorted, total_ff_num)
    #   else:
    #     target_nums = np.arange(currentTrial-num_trials+1, currentTrial+1)
    #     x, y = connect_path_ff2(cum_t, cum_mx, cum_my, cum_angle, target_nums, 250, ff_flash_sorted, ff_real_position_sorted, total_ff_num)
    #   for j in range(len(x)):
    #     xy_rotate = np.matmul(R, np.stack((x[j], y[j])))
    #     axes.plot(xy_rotate[0], xy_rotate[1], '-', alpha=0.2, c="#a940f5")
    #     axes.plot(xy_rotate[0][0], xy_rotate[1][0], '-', alpha=0.2, marker="o", markersize=5, color="brown")


    if trial_to_show_cluster_around_target != None:
      cluster_ff_pos = ffs_around_target_positions[currentTrial+trial_to_show_cluster_around_target]
      if len(cluster_ff_pos) > 0:
        ffs_around_target_rotate = np.matmul(R, np.stack((cluster_ff_pos.T[0], cluster_ff_pos.T[1])))
        axes.scatter(ffs_around_target_rotate[0],ffs_around_target_rotate[1], marker='o', s=30, color="blue", zorder=4) 
      if cluster_on_off_lines:
        # Find on_off_lines for ffs in the cluster
        for i in range(len(cluster_ff_pos)):
          index = np.array(ff_dataframe[(np.isclose(np.array(ff_dataframe['ff_x']),cluster_ff_pos[i, 0])) & (np.isclose(np.array(ff_dataframe['ff_y']),cluster_ff_pos[i, 1]))]['ff_index'])
          if len(index) > 0:
            index = index[0]
            #index = ffs_around_target_indices[currentTrial-trial_to_show_cluster_around_target][i]
            xy_onoff_lines = ff_dataframe.loc[ff_dataframe['target_index']==currentTrial]
            xy_onoff_lines2 = np.array(xy_onoff_lines.loc[(xy_onoff_lines['ff_index']==index) & (xy_onoff_lines['visible']==1)][['monkey_x', 'monkey_y']])
            onoff_lines_rotate = np.matmul(R, xy_onoff_lines2.T)
            axes.scatter(onoff_lines_rotate[0], onoff_lines_rotate[1], s=15-3*i, color=list_of_colors[i], alpha=0.4, zorder=3+i) 
            # Use corresponding color for that ff
            xy_onoff_lines3 = np.array(xy_onoff_lines[['ff_x', 'ff_y']]) 
            ffs_around_target_rotate = np.matmul(R, xy_onoff_lines3.T)
            axes.scatter(ffs_around_target_rotate[0],ffs_around_target_rotate[1], marker='o', s=100, alpha = 0.5, color=list_of_colors[i], zorder=3) 



    if show_scale_bar:
      scale1 = ScaleBar(  
      dx=1, length_fraction=0.2, fixed_value=100,
          location='upper left',  # in relation to the whole plot
          label_loc='left', scale_loc='bottom'  # in relation to the line
      )
      axes.add_artist(scale1)
      axes.xaxis.set_major_locator(ticker.NullLocator())
      axes.yaxis.set_major_locator(ticker.NullLocator())

    xmin, xmax = np.min(cum_mxy_rotate[0]), np.max(cum_mxy_rotate[0])
    ymin, ymax = np.min(cum_mxy_rotate[1]), np.max(cum_mxy_rotate[1])
    bigger_width = max(xmax-xmin, ymax-ymin)
    xmiddle, ymiddle = (xmin+xmax)/2, (ymin+ymax)/2
    xmin, xmax = xmiddle-bigger_width/2, xmiddle+bigger_width/2
    ymin, ymax = ymiddle-bigger_width/2, ymiddle+bigger_width/2
    margin = max(bigger_width/5, 150)
    axes.set_xlim((xmin-margin, xmax+margin))
    axes.set_ylim((ymin-margin, ymax+margin))
    axes.set_aspect('equal')


    if show_colorbar == True:
      # Make the black and red colorbar
      A = np.reshape([1,2,1,2,2,2], (2,3))# The numbers don't matter much
      norm_bins = np.array([0.5, 1.5, 2.5])
      # Let's also design our color mapping: 1s should be plotted in blue, 2s in red, etc...
      col_dict={1:"black",2:"red"}
      # We create a colormar from our list of colors
      speed_cm = ListedColormap([col_dict[x] for x in col_dict.keys()])
      ## Make normalizer and formatter
      norm = matplotlib.colors.BoundaryNorm(norm_bins, 2, clip=True)
      labels = np.array(["No Reward", "Reward"])
      fmt = matplotlib.ticker.FuncFormatter(lambda x, pos: labels[norm(x)])
      # Plot our figure
      im = axes.imshow(A, cmap=speed_cm, extent=[0,0,0,0], norm=norm)
      cax2 = fig.add_axes([0.95, 0.15, 0.05, 0.2])
      cb = fig.colorbar(im, format=fmt, ticks=np.array([1., 2.]), cax=cax2)
      cb.ax.tick_params(width=0)
      cb.ax.set_title('Stopping Points', ha='left')
      if trail_color == "orange":
        A = np.reshape([1,2,1,2,2,2], (2,3))# The numbers don't matter much
        norm_bins = np.array([0.5, 1.5, 2.5])
        # Let's also design our color mapping: 1s should be plotted in blue, 2s in red, etc...
        col_dict={1:"green",2:"orange"}
        # We create a colormar from our list of colors
        speed_cm = ListedColormap([col_dict[x] for x in col_dict.keys()])
        ## Make normalizer and formatter
        norm = matplotlib.colors.BoundaryNorm(norm_bins, 2, clip=True)
        labels = np.array(["Top Target Visible", "Top Target Not Visible"])
        fmt = matplotlib.ticker.FuncFormatter(lambda x, pos: labels[norm(x)])
        # Plot our figure
        im = axes.imshow(A, cmap=speed_cm, extent=[0,0,0,0], norm=norm)
        cax2 = fig.add_axes([0.95, 0.5, 0.05, 0.2])
        cb = fig.colorbar(im, format=fmt, ticks=np.array([1., 2.]), cax=cax2)
        cb.ax.tick_params(width=0)
        cb.ax.set_title('Path Colors', ha='left', y=1.04)
      elif trail_color == "viridis":
        cmap = cm.viridis
        norm = matplotlib.colors.Normalize(vmin=0, vmax=200)
        cax = fig.add_axes([0.95, 0.4, 0.05, 0.43])
        cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),
                  cax=cax, orientation='vertical')
        cbar.ax.set_title('Speed(cm/s)', ha='left', y=1.04)
        cbar.ax.tick_params(axis='y', color='white', direction="in", right=True,length=5, width=1.5)
        cbar.outline.remove()
      global Show_Colorbar 
      Show_Colorbar = False


### PlotPoints

In [ ]:
def PlotPoints(point,
                total_time, 
                show_all_ff,
                show_flash_on_ff,
                show_visible_ff,
                show_in_memory_ff, 
                show_target,
                show_reward_boundary,
                show_legend,
                show_colorbar, 
                show_scale_bar,
                trial_num=None,
                **kwargs):


          
  
    alive_ff_indices= np.array([i for i,value in 
                               enumerate(ff_life_sorted) if (value[-1]>=time) and (value[0]<time)]) 
    alive_ff_positions = ff_real_position_sorted[alive_ff_indices]
    if show_all_ff:
      axes.scatter(alive_ff_positions.T[0], alive_ff_positions.T[1], color="grey", s=30)

    if show_flash_on_ff:
      on_ff_indices = [] # Gives the indices of the ffs that are on at this point
      # For each firefly in ff_flash_sorted:
      for index, firefly in enumerate(ff_flash_sorted):
        # If the firefly has flashed during that trial:
        if index in alive_ff_indices: 
            # Let's see if the firefly has flashed at that exact moment
            for interval in firefly:
              if interval[0] <= time <= interval[1]:
                on_ff_indices.append(index) 
                break  
      on_ff_indices = np.array(on_ff_indices) 
      on_ff_positions = ff_real_position_sorted[on_ff_indices]
      axes.scatter(on_ff_positions.T[0], on_ff_positions.T[1], color="red", s=120, marker = '*', alpha = 0.7)
    
    if show_visible_ff:
      visible_ffs = ff_dataframe[(ff_dataframe['point_index']==point)&(ff_dataframe['visible']==1)][['ff_x', 'ff_y']]
      axes.scatter(visible_ffs['ff_x'], visible_ffs['ff_y'], color="orange", s=40)

    if show_in_memory_ff:
      in_memory_ffs = ff_dataframe[(ff_dataframe['point_index']==point)&(ff_dataframe['visible']==0)][['ff_x', 'ff_y']]
      axes.scatter(in_memory_ffs['ff_x'], in_memory_ffs['ff_y'], color="green", s=40)
    
    if show_target:
      if trial_num == None:
        raise ValueError("If show_target, then trial_num cannot be None")
      target_num  = distance_dataframe['trial'].iloc[point]
      target_position = ff_real_position_sorted[trial_num]
      axes.scatter(target_position[0], target_position[1], marker = '*', s=200, color="grey", alpha=0.35)


    if show_legend == True:
      legend_names = []
      if show_all_ff:
        legend_names.append("Invisible")
      if show_flash_on_ff:
        legend_names.append("Flash On")
      if show_visible_ff:
        legend_names.append("Visible")
      if show_in_memory_ff:
        legend_names.append("In memory")
      if show_target:
        legend_names.append("Target")
      axes.legend(legend_names, loc='upper right')
      global Show_Legend
      Show_Legend = False

    if show_reward_boundary:
      if show_all_ff:
        for i in range(len(alive_ff_positions)):
          circle2 = plt.Circle((alive_ff_positions[i, 0], alive_ff_positions[i, 1]), 20, facecolor='grey', edgecolor='orange', alpha=0.25, zorder=1)
          axes.add_patch(circle2)
      elif show_flash_on_ff:
        if show_flash_on_ff:
          for i in range(len(on_ff_positions)):
            circle2 = plt.Circle((on_ff_positions[i, 0], on_ff_positions[i, 1]), 20, facecolor='grey', edgecolor='orange', alpha=0.25, zorder=1)
            axes.add_patch(circle2)
        if show_in_memory_ff:
          for i in range(len(in_memory_ffs)):
            circle2 = plt.Circle((in_memory_ffs['ff_x'].iloc[i], in_memory_ffs['ff_y'].iloc[i]), 20, facecolor='grey', edgecolor='orange', alpha=0.25, zorder=1)
            axes.add_patch(circle2) 
      elif show_visible_ff:
        for i in range(len(visible_ffs)):
          circle2 = plt.Circle((visible_ffs['ff_x'].iloc[i], visible_ffs['ff_y'].iloc[i]), 20, facecolor='grey', edgecolor='orange', alpha=0.25, zorder=1)
          axes.add_patch(circle2)  
        if show_in_memory_ff:
          for i in range(len(in_memory_ffs)):
            circle2 = plt.Circle((in_memory_ffs['ff_x'].iloc[i], in_memory_ffs['ff_y'].iloc[i]), 20, facecolor='grey', edgecolor='orange', alpha=0.25, zorder=1)
            axes.add_patch(circle2)  

    axes.scatter(cum_mx, cum_my, s=15, c=index_temp, cmap="Blues") 

    xmin, xmax = np.min(cum_mx), np.max(cum_mx)
    ymin, ymax = np.min(cum_my), np.max(cum_my)
    bigger_width = max(xmax-xmin, ymax-ymin)
    xmiddle, ymiddle = (xmin+xmax)/2, (ymin+ymax)/2
    xmin, xmax = xmiddle-bigger_width/2, xmiddle+bigger_width/2
    ymin, ymax = ymiddle-bigger_width/2, ymiddle+bigger_width/2
    margin = max(bigger_width/5, 250)
    axes.set_xlim((xmin-margin, xmax+margin))
    axes.set_ylim((ymin-margin, ymax+margin))
    axes.set_aspect('equal')

    if show_scale_bar == True:
      scale1 = ScaleBar(  
      dx=1, length_fraction=0.2, fixed_value=100,
          location='upper left',  # in relation to the whole plot
          label_loc='left', scale_loc='bottom'  # in relation to the line
      )
      axes.add_artist(scale1)

    axes.xaxis.set_major_locator(ticker.NullLocator())
    axes.yaxis.set_major_locator(ticker.NullLocator()) 

    if show_colorbar == True:
      cmap = cm.Blues
      cax = fig.add_axes([0.95, 0.25, 0.05, 0.52]) #[left, bottom, width, height] 
      cbar = fig.colorbar(cm.ScalarMappable(cmap=cmap),ticks=[0, 1],
                cax=cax, orientation='vertical')
      cbar.ax.set_title('Trajectory', ha='left', y=1.07)
      cbar.ax.tick_params(size = 0)
      cbar.outline.remove()
      cbar.ax.set_yticklabels(['Least recent',  'Most recent']) 
      global Show_Colorbar 
      Show_Colorbar = False











### make ff_dataframe

In [ ]:
def MakeFFDataframe(monkey_information, ff_catched_T_sorted, ff_flash_sorted,  ff_real_position_sorted, max_distance = 400, data_folder_name = None, num_missed_index = -1):
  # Let's use data from monkey_information. But we shall cut off portion that is before the time of capturing the first target
  monkey_t_array0 = np.array(monkey_information['monkey_t'])
  monkey_x_array0 = np.array(monkey_information['monkey_x'])
  monkey_y_array0 = np.array(monkey_information['monkey_y'])
  monkey_angle_array0 = np.array(monkey_information['monkey_angle'])
  if num_missed_index < 0:
    valid_index = np.where(monkey_t_array0 > ff_catched_T_sorted[0])[0]
    num_missed_index = valid_index[0]
  monkey_t_array = monkey_t_array0[num_missed_index:]
  monkey_x_array = monkey_x_array0[num_missed_index:]
  monkey_y_array = monkey_y_array0[num_missed_index:]
  monkey_angle_array = monkey_angle_array0[num_missed_index:]
  index_array = np.array(range(len(monkey_t_array)))

  #max_distance = 400
  reward_boundary = 25
  #max_time = 400
  #max_time = ff_catched_T_sorted[1251] #replace with ff_catched_T_sorted[-1]
  ff_index = []
  point_index = []
  time = []
  target_index = []
  ff_x = []
  ff_y = []
  monkey_x = []
  monkey_y = []
  visible = []
  memory = []
  ff_distance = []
  ff_angle = []
  ff_angle_boundary = []
  left_right = []
  num_captures = []
  distance_closest_ff =[]
  distance_2ndclosest_ff = []
  num_ff_within = []
  reward_distance = []
  reward_angle = []
  catched_ff_num = len(ff_catched_T_sorted) - 200
  total_ff_num = len(ff_life_sorted)


  for i in range(total_ff_num):
    if i % 100 == 0:
      print(i," out of ", total_ff_num)

    # visible_indices contains the indices of the points when the ff is visible (within a suitable distance & at the right angle)
    visible_indices = []
    # Go through every visible duration of the same ff
    ff_flash = ff_flash_sorted[i]
    for j in range(len(ff_flash)):
      visible_duration = ff_flash[j]
      ## if visible_duration[0] < max_time:
      # Find the corresponding monkey information:
      cum_indices = np.where((monkey_t_array >= visible_duration[0]) & 
                          (monkey_t_array <= visible_duration[1]))[0]
      cum_t, cum_angle = monkey_t_array[cum_indices], monkey_angle_array[cum_indices]
      cum_mx, cum_my = monkey_x_array[cum_indices], monkey_y_array[cum_indices]
      distances_to_monkey = LA.norm(np.stack([cum_mx, cum_my], axis=1)-ff_real_position_sorted[i], axis = 1)
      valid_distance_indices = np.where(distances_to_monkey < max_distance)[0]
      if len(valid_distance_indices) > 0:
        angles_to_monkey = np.arctan2(ff_real_position_sorted[i,1]-cum_my[valid_distance_indices], ff_real_position_sorted[i,0]-cum_mx[valid_distance_indices])-cum_angle[valid_distance_indices]
        angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
        angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi
        angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(reward_boundary, np.maximum(distances_to_monkey[valid_distance_indices], reward_boundary) ))) # use torch clip to get valid arcsin input
        angles_adjusted = np.clip(angles_adjusted, 0, pi)
        angles_to_monkey = np.sign(angles_to_monkey)* angles_adjusted
        overall_valid_indices = valid_distance_indices[np.where(np.absolute(angles_to_monkey) <= 2*pi/9)[0]]
        visible_indices = visible_indices + cum_indices[overall_valid_indices].tolist()
    visible_indices = np.array(visible_indices)



    if len(visible_indices) > 0:
      # Make a numpy array of points to denote memory, with 0 means being invisible. We also append 99 extra points after the last point    
      memory_indices0 = np.zeros(visible_indices[-1]+100, dtype=int)
      memory_indices0[visible_indices] = 100

      if len(ff_catched_T_sorted)-1 >= i:
        # Find the index of the time at which the ff is captured
        last_live_time = np.where(monkey_t_array <= ff_catched_T_sorted[i])[0][-1]
        # Truncate memory_indices0 based on that 
        memory_indices0 = memory_indices0[:last_live_time+1]

      
      # Iterate through memory_indices0 to make a new list to denote memory (replacing some 0s with other numbers based on time)
      # We preserve the first element of memory_indices0. We also separate memory_indices and point_indices (denoted as final_indices)
      memory_indices = [memory_indices0[0]]

      for k in range(1, len(memory_indices0)):
        if memory_indices0[k] == 0:
          memory_indices.append(memory_indices[k-1]-1)
        else: # Else, preserve the current value
          memory_indices.append(memory_indices0[k])
      memory_indices_array = np.array(memory_indices)

      index_array0 = np.arange(len(memory_indices0))
      if len(index_array0) > len(monkey_t_array):
        max_index = len(monkey_t_array)
        index_array0 = index_array0[:max_index]
        memory_indices_array = memory_indices_array[index_array0]

      in_memory_indices = np.where(memory_indices_array > 0)[0]
      memory_indices_array = memory_indices_array[in_memory_indices]
      index_array = index_array0[in_memory_indices]
      in_memory_length = len(memory_indices_array)
      memory_indices = memory_indices_array.tolist()
      final_indices = index_array.tolist()




      # Append the values for this ff; Using list is faster than np.append
      ff_index = ff_index + [i]*in_memory_length
      point_index = point_index+ [point + num_missed_index for point in final_indices]
      relevant_time = monkey_t_array[index_array]
      time = time+relevant_time.tolist()
      target_index = target_index + np.digitize(relevant_time, ff_catched_T_sorted).tolist()
      ff_x = ff_x + [(ff_real_position_sorted[i][0])]*in_memory_length
      ff_y = ff_y + [(ff_real_position_sorted[i][1])]*in_memory_length
      monkey_x = monkey_x + monkey_x_array[index_array].tolist()
      monkey_y = monkey_y + monkey_y_array[index_array].tolist()
      visible = visible + [1 if point ==100 else 0 for point in memory_indices]
      memory = memory + memory_indices
      monkey_xy_relevant = np.stack([monkey_x_array[index_array], monkey_y_array[index_array]],axis=1)
      monkey_angle_relevant = monkey_angle_array[index_array]
      ff_distance_relevant = LA.norm(monkey_xy_relevant-ff_real_position_sorted[i], axis=1)
      ff_distance = ff_distance + ff_distance_relevant.tolist()
      angles_to_monkey = np.arctan2(ff_real_position_sorted[i,1]-monkey_xy_relevant[:,1], ff_real_position_sorted[i,0]-monkey_xy_relevant[:,0]) - monkey_angle_relevant
      angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
      angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi
      angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(reward_boundary, np.maximum(ff_distance_relevant, reward_boundary) ))) # use torch clip to get valid arcsin input
      angles_adjusted = np.clip(angles_adjusted, 0, pi)
      angles_adjusted = np.sign(angles_to_monkey)* angles_adjusted
      ff_angle = ff_angle + angles_to_monkey.tolist()
      ff_angle_boundary = ff_angle_boundary + angles_adjusted.tolist()
      left_right = left_right + (np.array(angles_to_monkey) > 0).astype(int).tolist()
      # num_captures.append
      # distance_closest_ff.append
      # distance_2ndclosest_ff.append
      # num_ff_within.append
      # reward_distance.append
      # reward_angle.append

        
  # Now let's create a dictionary of the lists
  ff_dict = {'ff_index':ff_index, 'point_index':point_index, 'time':time, 'target_index':target_index,
              'ff_x':ff_x, 'ff_y':ff_y, 'monkey_x':monkey_x, 'monkey_y':monkey_y, 'visible':visible,
              'memory':memory, 'ff_distance':ff_distance, 'ff_angle':ff_angle, 'ff_angle_boundary': ff_angle_boundary, 'left_right':left_right}
  ff_dataframe = pd.DataFrame(ff_dict)

  if len(ff_catched_T_sorted) > 800:
    ff_dataframe = ff_dataframe[ff_dataframe['time'] < ff_catched_T_sorted[-200]]

  if data_folder_name:
    filepath = 'gdrive/MyDrive/fireflies_data/' + data_folder_name + '/ff_dataframe.csv'
    os.makedirs('gdrive/MyDrive/fireflies_data/' + data_folder_name, exist_ok = True)
    ff_dataframe.to_csv(filepath) 

  return ff_dataframe


### make ff_dataframe (agent)
(only include ffs that are in obs space)

In [ ]:
def MakeFFDataframe(monkey_information, ff_catched_T_sorted, ff_flash_sorted,  ff_real_position_sorted, max_distance = 400, data_folder_name = None, num_missed_index = -1):
  # Let's use data from monkey_information. But we shall cut off portion that is before the time of capturing the first target
  monkey_t_array0 = np.array(monkey_information['monkey_t'])
  monkey_x_array0 = np.array(monkey_information['monkey_x'])
  monkey_y_array0 = np.array(monkey_information['monkey_y'])
  monkey_angle_array0 = np.array(monkey_information['monkey_angle'])
  valid_index = np.where(monkey_t_array0 > ff_catched_T_sorted[0])[0]
  num_missed_index = valid_index[0]
  monkey_t_array = monkey_t_array0[valid_index]
  monkey_x_array = monkey_x_array0[valid_index]
  monkey_y_array = monkey_y_array0[valid_index]
  monkey_angle_array = monkey_angle_array0[valid_index]
  #index_array = np.array(range(len(monkey_t_array))) this overlaps with another variable later

  max_distance = 400
  reward_boundary = 25
  #max_time = 400
  #max_time = ff_catched_T_sorted[1251] #replace with ff_catched_T_sorted[-1]
  ff_index = []
  point_index = []
  time = []
  target_index = []
  ff_x = []
  ff_y = []
  monkey_x = []
  monkey_y = []
  visible = []
  memory = []
  ff_distance = []
  ff_angle = []
  ff_angle_boundary = []
  left_right = []
  num_captures = []
  distance_closest_ff =[]
  distance_2ndclosest_ff = []
  num_ff_within = []
  reward_distance = []
  reward_angle = []
  catched_ff_num = len(ff_catched_T_sorted) - 200
  total_ff_num = len(ff_life_sorted)


  for i in range(total_ff_num):

    if i % 100 == 0:
      print(i," out of ", total_ff_num)


    # Go through every visible duration of the same ff
    ff_flash = ff_flash_sorted[i]
    whether_in_obs = []
    original_index = sort_indices_all[i]
    for index in valid_index:
      obs_ff_indices = obs_ff_overall_indices_all[index]
      if original_index in obs_ff_indices:
        whether_in_obs.append(True) 
      else:
        whether_in_obs.append(False)

    
    cum_indices = np.array(whether_in_obs).nonzero()[0]
    if len(cum_indices) > 0:

      updated_t_array = monkey_t_array[cum_indices]
      visible_indices = cum_indices
      index_array = cum_indices
      in_memory_length = len(index_array)
        
      cum_t, cum_angle = monkey_t_array[cum_indices], monkey_angle_array[cum_indices]
      cum_mx, cum_my = monkey_x_array[cum_indices], monkey_y_array[cum_indices]
      distances_to_monkey = LA.norm(np.stack([cum_mx, cum_my], axis=1)-ff_real_position_sorted[i], axis = 1)
      angles_to_monkey = np.arctan2(ff_real_position_sorted[i,1]-cum_my, ff_real_position_sorted[i,0]-cum_mx)-cum_angle
      angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
      angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi
      angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(reward_boundary, np.maximum(distances_to_monkey, reward_boundary) ))) # use torch clip to get valid arcsin input
      angles_adjusted = np.clip(angles_adjusted, 0, pi)
      angles_to_monkey = np.sign(angles_to_monkey)* angles_adjusted



      # Make a numpy array of points to denote memory, with 0 means being invisible. We also append 99 extra points after the last point    
      memory_indices0 = np.zeros(visible_indices[-1]+100, dtype=int)
      memory_indices0[visible_indices] = 100

      if len(ff_catched_T_sorted)-1 >= i:
        # Find the index of the time at which the ff is captured
        last_live_time = np.where(monkey_t_array <= ff_catched_T_sorted[i])[0][-1]
        # Truncate memory_indices0 based on that 
        memory_indices0 = memory_indices0[:last_live_time+1]

      
      # Iterate through memory_indices0 to make a new list to denote memory (replacing some 0s with other numbers based on time)
      # We preserve the first element of memory_indices0. We also separate memory_indices and point_indices (denoted as final_indices)
      memory_indices = [memory_indices0[0]]

      for k in range(1, len(memory_indices0)):
        if memory_indices0[k] == 0:
          memory_indices.append(memory_indices[k-1]-1)
        else: # Else, preserve the current value
          memory_indices.append(memory_indices0[k])
      memory_indices_array = np.array(memory_indices)

      index_array0 = np.arange(len(memory_indices0))
      if len(index_array0) > len(monkey_t_array):
        max_index = len(monkey_t_array)
        index_array0 = index_array0[:max_index]
        memory_indices_array = memory_indices_array[index_array0]

      in_memory_indices = np.where(memory_indices_array > 0)[0]
      memory_indices_array = memory_indices_array[in_memory_indices]
      index_array = index_array0[in_memory_indices]
      in_memory_length = len(memory_indices_array)
      memory_indices = memory_indices_array.tolist()
      final_indices = index_array.tolist()

      ff_index = ff_index + [i]*in_memory_length
      point_index = point_index+ [point + num_missed_index for point in final_indices]
      relevant_time = monkey_t_array[index_array]
      time = time+relevant_time.tolist()
      target_index = target_index + np.digitize(relevant_time, ff_catched_T_sorted).tolist()
      ff_x = ff_x + [(ff_real_position_sorted[i][0])]*in_memory_length
      ff_y = ff_y + [(ff_real_position_sorted[i][1])]*in_memory_length
      monkey_x = monkey_x + monkey_x_array[index_array].tolist()
      monkey_y = monkey_y + monkey_y_array[index_array].tolist()
      visible = visible + [1 if point ==100 else 0 for point in memory_indices]
      memory = memory + memory_indices
      monkey_xy_relevant = np.stack([monkey_x_array[index_array], monkey_y_array[index_array]],axis=1)
      monkey_angle_relevant = monkey_angle_array[index_array]
      ff_distance_relevant = LA.norm(monkey_xy_relevant-ff_real_position_sorted[i], axis=1)
      ff_distance = ff_distance + ff_distance_relevant.tolist()
      angles_to_monkey = np.arctan2(ff_real_position_sorted[i,1]-monkey_xy_relevant[:,1], ff_real_position_sorted[i,0]-monkey_xy_relevant[:,0]) - monkey_angle_relevant
      angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
      angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi
      angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(reward_boundary, np.maximum(ff_distance_relevant, reward_boundary) ))) # use torch clip to get valid arcsin input
      angles_adjusted = np.clip(angles_adjusted, 0, pi)
      angles_adjusted = np.sign(angles_to_monkey)* angles_adjusted
      ff_angle = ff_angle + angles_to_monkey.tolist()
      ff_angle_boundary = ff_angle_boundary + angles_adjusted.tolist()
      left_right = left_right + (np.array(angles_to_monkey) > 0).astype(int).tolist()



          
  # Now let's create a dictionary of the lists
  ff_dict = {'ff_index':ff_index, 'point_index':point_index, 'time':time, 'target_index':target_index,
              'ff_x':ff_x, 'ff_y':ff_y, 'monkey_x':monkey_x, 'monkey_y':monkey_y, 'visible':visible,
              'memory':memory, 'ff_distance':ff_distance, 'ff_angle':ff_angle, 'ff_angle_boundary': ff_angle_boundary, 'left_right':left_right}
  ff_dataframe = pd.DataFrame(ff_dict)

  filepath = 'gdrive/MyDrive/fireflies_data/' + data_folder_name + '/ff_dataframe.csv'
  os.makedirs('gdrive/MyDrive/fireflies_data/' + data_folder_name, exist_ok = True)
  ff_dataframe.to_csv(filepath) 


## Data from agent (SB3)

### import packages

In [ ]:
data_folder_name = "agent_3"
data_num = 3


import random
import gym
from gym import spaces, Env
from gym.spaces import Dict, Box
import torch
from numpy import pi
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.linalg import vector_norm


import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

!pip install stable-baselines3
from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.evaluation import evaluate_policy
from torch.utils.data.dataset import Dataset, random_split
# For animation

torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

from PIL import Image, ImageDraw, ImageOps
from IPython.display import Image as Image2

import os

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback

!pip install optuna
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler

from typing import Any
from typing import Dict


In [ ]:
if MONKEY_DATA == True:
  raise ValueError("Need to skip the codes for RL")

### c10.2: dt = 0.25, noise = 1, memory = 3

discount factor = 0.995


In [ ]:
retrieve_dir = "/content/gdrive/MyDrive/fireflies_agent/July_20_6/" # discount factor = 0.995  
data_folder_name = "July_20_6"

In [ ]:
# obs = [angle, distance, memory, angle2, distance2, memory2, ...]
# before = [angle, angle2 ..., distance, distance2, ... memory, memory2]

class MultiFF(Env):
  def __init__(self):
      super(MultiFF, self).__init__()
      self.num_ff = 200
      self.arena_radius = 1000
      self.episode_len = 15000 
      self.dt = 0.25 
      #self.current_episode = 0
      #self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.obs_ff = 2
      self.observation_space = spaces.Box(low=-1., high=1., shape=(self.obs_ff*4,),dtype=np.float32)
      self.terminal_vel = 0.01
      #self.pro_noise_std = 0.005
      self.vgain = 200
      self.wgain = pi/2
      self.reward_per_ff = 100
      #self.time_cost = 0.005
      #self.total_time = 0
      #self.zero_action = False
      #self.target_update_counter_num = 20
      #self.reward_per_episode = []
      #self.update_slots = True
      #self.closest_ff_distance = 200
      self.pro_noise_std = 0.005
      self.epi_num = -1
      self.has_sped_up_before = False
      self.full_memory = 3
      self.internal_noise_factor = 1
      self.ff_memory_all = torch.ones([self.num_ff,])*self.full_memory
      self.invisible_distance = 400
      self.invisible_angle = 2*pi/9
      self.reward_boundary = 25
      






  def reset(self):
    self.epi_num += 1
    print("\n episode: ", self.epi_num)
    self.num_targets = 0
    #self.past_speeds = []
    self.time = 0
    #self.counter = 0
    #self.current_target_index = torch.tensor([999], dtype=torch.int32)
    #self.previous_target_index = self.current_target_index
    self.ff_flash = []
    self.has_sped_up_before = False
    
    for i in range(self.num_ff):
      num_intervals = 2500
      first_flash = torch.rand(1)
      intervals = torch.poisson(torch.ones(num_intervals-1)*3)
      t0 = torch.cat((first_flash, first_flash+torch.cumsum(intervals, dim=0)+torch.cumsum(torch.ones(num_intervals-1)*0.3, dim=0)))
      t1 = t0 + torch.ones(num_intervals)*0.3
      self.ff_flash.append(torch.stack((t0, t1), dim=1))

    self.ffr = torch.sqrt(torch.rand(self.num_ff))*self.arena_radius # The radius of the arena changed from 1000 cm/s to 200 cm/s
    self.fftheta = torch.rand(self.num_ff)*2*pi
    #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
    self.ffx = torch.cos(self.fftheta) * self.ffr
    self.ffy = torch.sin(self.fftheta) * self.ffr
    self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
    self.ffx2 = self.ffx.clone()
    self.ffy2 = self.ffy.clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    #self.ff_info={}
    #self.update_slots = True
    self.agentx = torch.tensor([0])
    self.agenty = torch.tensor([0])
    self.agentr = torch.zeros(1)
    self.agentxy = torch.tensor([0, 0])
    self.agentheading = torch.zeros(1).uniform_(0, 2*pi)
    self.dv = torch.zeros(1).uniform_(-0.05, 0.05) 
    self.dw = torch.zeros(1)
    self.end_episode = False 
    self.obs = self.beliefs().numpy()
    #self.chunk_50s = 1
    self.episode_reward = 0
    #self.stop_rewarding_speed = False
    #self.current_obs_steps = 0
    self.ff_memory_all = torch.ones([self.num_ff,])
    return self.obs






  def calculate_reward(self):
      #action_cost=((self.previous_action[1]-self.action[1])**2+(self.previous_action[0]-self.action[0])**2)*self.mag_cost
      # To incorporate action_cost, we need to incorporate previous_action into decision_info
      #self.total_time += 1
      # In addition to rewarding the monkey for capturing the firefly, we also use different phases of rewards to teach monkey specific behaviours
      #reward = -self.time_cost
      reward = 0
      # Reward shaping
      # Phase I: reward the agent for learning to stop 
      # Always: reward the agent for capturing fireflies
      self.num_targets = 0
      if abs(self.sys_vel[1]) <= self.terminal_vel:
        captured_ff_index = (self.ff_distance_all <= self.reward_boundary).nonzero().reshape(-1).tolist()
        self.captured_ff_index = captured_ff_index
        num_targets = len(captured_ff_index)
        self.num_targets = num_targets
        if num_targets > 0: # If the monkey hs captured at least 1 ff
          # Calculate reward
          reward = reward + self.reward_per_ff * num_targets
          # Replace the captured ffs with ffs of new locations
          self.ffr[captured_ff_index]= torch.sqrt(torch.rand(num_targets))*self.arena_radius
          self.fftheta[captured_ff_index]= torch.rand(num_targets)*2*pi
          #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
          self.ffx[captured_ff_index] = torch.cos(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffy[captured_ff_index] = torch.sin(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
          self.ffx2[captured_ff_index] = self.ffx[captured_ff_index].clone()
          self.ffy2[captured_ff_index] = self.ffy[captured_ff_index].clone()
          self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
          # Delete the information from self.ff_info
          #[self.ff_info.pop(key) for key in captured_ff_index if (key in self.ff_info)]
          #self.current_target_index = torch.tensor([999], dtype=torch.int32)
          #self.previous_target_index = self.current_target_index
          # Reward the firefly based on the average speed before capturing the firefly
          ##reward = reward + sum(self.past_speeds)/len(self.past_speeds)
          #print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "obs: ", list(np.round(self.obs, decimals = 2)), "n_targets: ",  num_targets)
          print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "n_targets: ",  num_targets)
          #self.update_slots = True
          #self.stop_rewarding_speed = True
        #elif self.has_sped_up_before == True:
        # based on Ruiyi's formula, using the distance of the closest ff in obs
        #reward += math.exp(-((self.ff_current[1, 0]**2)*(25/1.5)**2)/2)
        #self.has_sped_up_before = False
      return reward







  def step(self, action):
    self.time += self.dt
    action=torch.tensor(action)
    action[1] = action[1]/2+0.5
    self.sys_vel=action.clone()
    self.state_step(action)
    self.obs = self.beliefs().numpy()
    reward=self.calculate_reward()
    self.episode_reward += reward
    
    if self.time >= self.episode_len*self.dt:
      self.end_episode = True
      #self.current_episode += 1
      print("Reward for the episode: ", self.episode_reward)
    #print("action: ", torch.round(action, decimals = 3), "obs: ", np.round(self.obs, decimals = 3), "Reward: ", round(reward, 3))
    return self.obs, reward, self.end_episode, {}




  def state_step(self,action):
    vnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std 
    wnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std
    self.dw_normed = (action[0]+wnoise)
    self.dv_normed = (action[1]+vnoise)
    self.dw = (action[0]+wnoise)*self.wgain*self.dt
    self.agentheading = self.agentheading + self.dw.item()
    self.dv =  (action[1]+vnoise)*self.vgain*self.dt
    self.dx= torch.cos(self.agentheading)*self.dv
    self.dy= torch.sin(self.agentheading)*self.dv
    self.agentx = self.agentx + self.dx.item()
    self.agenty = self.agenty + self.dy.item()
    self.agentxy = torch.cat((self.agentx, self.agenty))
    self.agentr = vector_norm(self.agentxy)
    self.agenttheta = torch.tensor(torch.atan2(self.agenty, self.agentx))  
                               
    if self.agentr >= self.arena_radius:
      self.agentr = 2*self.arena_radius-self.agentr
      self.agenttheta = self.agenttheta + pi
      self.agentx = (self.agentr*torch.cos(self.agenttheta)).reshape(1,)
      self.agenty = (self.agentr*torch.sin(self.agenttheta)).reshape(1,)
      self.agentxy = torch.cat((self.agentx, self.agenty))
      self.agentheading = self.agenttheta - pi
    while self.agentheading >= 2*pi:
      self.agentheading = self.agentheading - 2*pi
    while self.agentheading < 0:
      self.agentheading = self.agentheading + 2*pi   








  def beliefs(self): 

    # Make a tensor containing the relative distance of all fireflies to the agent
    self.ff_distance_all = vector_norm(self.ffxy - self.agentxy, dim=1)
    # Make a tensor containing the relative (real) angle of all fireflies to the agent
    ffradians = torch.atan2(self.ffy-self.agenty, self.ffx-self.agentx)
    angle0 = ffradians - self.agentheading
    angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
    angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
    # Adjust the angle based on reward boundary
    angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip( self.ff_distance_all, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
    angle2 = torch.clip(angle1,0,pi)
    ff_angle_all = torch.sign(angle0)* angle2
    # Update the tensor containing the uncertainties of all fireflies to the agent
    visible_ff = torch.logical_and( self.ff_distance_all < self.invisible_distance, torch.abs(ff_angle_all) < self.invisible_angle)
    self.visible_ff_indices0 = visible_ff.nonzero().reshape(-1)
    for index in self.visible_ff_indices0:
      ff = self.ff_flash[index]
      if not torch.any(torch.logical_and(ff[:, 0] <= self.time, ff[:, 1] >= self.time)):
        visible_ff[index] = False
    self.visible_ff_indices = visible_ff.nonzero().reshape(-1)
    # Update memory
    self.ff_memory_all -= 1
    self.ff_memory_all[self.visible_ff_indices] = self.full_memory
    self.ff_memory_all = torch.clamp(self.ff_memory_all, 0, self.full_memory)
    # Calculate the uncertainties that will be added to relative distance and angle based on memory
    ff_uncertainty_all = (self.full_memory-self.ff_memory_all)*self.internal_noise_factor
    self.ffx2 = self.ffx2 + torch.normal(torch.zeros([self.num_ff,]), ff_uncertainty_all)
    self.ffy2 = self.ffy2 + torch.normal(torch.zeros([self.num_ff,]), ff_uncertainty_all)
    self.ffx2[self.visible_ff_indices] = self.ffx[self.visible_ff_indices].clone()
    self.ffy2[self.visible_ff_indices] = self.ffy[self.visible_ff_indices].clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    # find ffs that are in memory
    self.ff_in_memory_indices = (self.ff_memory_all > 0).nonzero().reshape(-1)
    # Consider the case where there are fewer than self.obs_ff fireflies that are in memory
    

   
    if torch.numel(self.ff_in_memory_indices) >= self.obs_ff:
      # Rank the ff whose "memory" is creater than 0 based on distance
      sorted_indices =  torch.topk(-self.ff_distance_all[self.ff_in_memory_indices], self.obs_ff).indices
      self.topk_indices = self.ff_in_memory_indices[sorted_indices]
      self.ffxy2_topk = self.ffxy2[self.topk_indices]
      self.ff_distance_topk = vector_norm(self.ffxy2_topk - self.agentxy, dim=1)
      # Calculate relative angles 
      ffradians = torch.atan2(self.ffxy2_topk[:,1]-self.agenty, self.ffxy2_topk[:,0]-self.agentx)
      angle0 = ffradians - self.agentheading
      angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      self.ff_angle_topk_2 = angle0.clone()
      # Calculate relative angles of all ffs based on reward boundaries
      # Adjust the angle based on reward boundary
      angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      angle2 = torch.clip(angle1,0,pi)
      ff_angle_topk_3 = torch.sign(angle0)* angle2
      # Concatenate distance, angle, and memory
      ff_array = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk,  self.ff_memory_all[self.topk_indices]), dim=0)
    

    elif torch.numel(self.ff_in_memory_indices) == 0:
      ff_array = torch.tensor([[0], [0], [self.invisible_distance], [0]]).repeat([1, self.obs_ff])
      self.topk_indices = torch.tensor([])

    else:
      sorted_distance, sorted_indices = torch.sort(-self.ff_distance_all[self.ff_in_memory_indices])
      self.topk_indices = self.ff_in_memory_indices[sorted_indices]
      self.ffxy2_topk = self.ffxy2[self.topk_indices ]
      self.ff_distance_topk = vector_norm(self.ffxy2_topk - self.agentxy, dim=1)
      # Calculate relative angles 
      ffradians = torch.atan2(self.ffxy2_topk[:,1]-self.agenty, self.ffxy2_topk[:,0]-self.agentx)
      angle0 = ffradians - self.agentheading
      angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      self.ff_angle_topk_2 = angle0.clone()
      # Calculate relative angles of all ffs based on reward boundaries
      # Adjust the angle based on reward boundary
      angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      angle2 = torch.clip(angle1,0,pi)
      ff_angle_topk_3 = torch.sign(angle0)* angle2
      # Concatenate distance, angle, and memory
      ff_array0 = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk,  self.ff_memory_all[self.topk_indices]), dim=0)
      needed_ff = self.obs_ff - torch.numel(self.ff_in_memory_indices)
      ff_array = torch.stack([ff_array0.reshape([4,-1]), torch.tensor([[0], [0], [self.invisible_distance], [0]]).repeat([1, needed_ff])], dim=1)
      

    # ff_array[0:2,:] = ff_array[0:2,:]/pi
    # ff_array[2,:] = (ff_array[2,:]/self.invisible_distance-0.5)*2
    # ff_array[3,:] = (ff_array[3,:]/20-0.5)*2
    self.ff_array = ff_array.clone()
    return torch.flatten(ff_array.transpose(0, 1))


env = MultiFF() 
env.reset() 

sac_model = SAC("MlpPolicy", 
            env,
            buffer_size=int(1e6),
            batch_size=1024,
            device='auto',
            verbose=False,
            train_freq=100,
            learning_starts = int(10),
            target_update_interval=20,
            learning_rate=1e-2,
            gamma=0.9999,
            policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[64, 64])
                )


### c11: dt = 0.25, noise = 1.2, memory = 4

discount factor = 0.995


In [ ]:
retrieve_dir = "/content/gdrive/MyDrive/fireflies_agent/July_30/" # discount factor = 0.995  
# # Last mean reward per episode: 84256.00
# path = os.path.join(retrieve_dir, 'best_model.zip')
# path2 = os.path.join(retrieve_dir, 'buffer.pkl')
# sac_model = sac_model.load(path,env=env)  
# sac_model.load_replay_buffer(path2)

In [ ]:
# obs = [angle, distance, memory, angle2, distance2, memory2, ...]
# before = [angle, angle2 ..., distance, distance2, ... memory, memory2]

class MultiFF(Env):
  def __init__(self):
      super(MultiFF, self).__init__()
      self.num_ff = 200
      self.arena_radius = 1000
      self.episode_len = 10000 
      self.dt = 0.25 
      #self.current_episode = 0
      #self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.obs_ff = 2
      self.observation_space = spaces.Box(low=-1., high=1., shape=(self.obs_ff*4,),dtype=np.float32)
      self.terminal_vel = 0.01
      #self.pro_noise_std = 0.005
      self.vgain = 200
      self.wgain = pi/2
      self.reward_per_ff = 100
      #self.time_cost = 0.005
      #self.total_time = 0
      #self.zero_action = False
      #self.target_update_counter_num = 20
      #self.reward_per_episode = []
      #self.update_slots = True
      #self.closest_ff_distance = 200
      self.pro_noise_std = 0.005
      self.epi_num = -1
      self.has_sped_up_before = False
      self.full_memory = 4
      self.internal_noise_factor = 1.2
      self.ff_memory_all = torch.ones([self.num_ff,])*self.full_memory
      self.invisible_distance = 400
      self.invisible_angle = 2*pi/9
      self.reward_boundary = 25
      






  def reset(self):
    self.epi_num += 1
    print("\n episode: ", self.epi_num)
    self.num_targets = 0
    #self.past_speeds = []
    self.time = 0
    #self.counter = 0
    #self.current_target_index = torch.tensor([999], dtype=torch.int32)
    #self.previous_target_index = self.current_target_index
    self.ff_flash = []
    self.has_sped_up_before = False
    
    for i in range(self.num_ff):
      num_intervals = 1500
      first_flash = torch.rand(1)
      intervals = torch.poisson(torch.ones(num_intervals-1)*3)
      t0 = torch.cat((first_flash, first_flash+torch.cumsum(intervals, dim=0)+torch.cumsum(torch.ones(num_intervals-1)*0.3, dim=0)))
      t1 = t0 + torch.ones(num_intervals)*0.3
      self.ff_flash.append(torch.stack((t0, t1), dim=1))

    self.ffr = torch.sqrt(torch.rand(self.num_ff))*self.arena_radius # The radius of the arena changed from 1000 cm/s to 200 cm/s
    self.fftheta = torch.rand(self.num_ff)*2*pi
    #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
    self.ffx = torch.cos(self.fftheta) * self.ffr
    self.ffy = torch.sin(self.fftheta) * self.ffr
    self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
    self.ffx2 = self.ffx.clone()
    self.ffy2 = self.ffy.clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    #self.ff_info={}
    #self.update_slots = True
    self.agentx = torch.tensor([0])
    self.agenty = torch.tensor([0])
    self.agentr = torch.zeros(1)
    self.agentxy = torch.tensor([0, 0])
    self.agentheading = torch.zeros(1).uniform_(0, 2*pi)
    self.dv = torch.zeros(1).uniform_(-0.05, 0.05) 
    self.dw = torch.zeros(1)
    self.end_episode = False 
    self.obs = self.beliefs().numpy()
    #self.chunk_50s = 1
    self.episode_reward = 0
    #self.stop_rewarding_speed = False
    #self.current_obs_steps = 0
    self.ff_memory_all = torch.ones([self.num_ff,])
    return self.obs






  def calculate_reward(self):
      #action_cost=((self.previous_action[1]-self.action[1])**2+(self.previous_action[0]-self.action[0])**2)*self.mag_cost
      # To incorporate action_cost, we need to incorporate previous_action into decision_info
      #self.total_time += 1
      # In addition to rewarding the monkey for capturing the firefly, we also use different phases of rewards to teach monkey specific behaviours
      #reward = -self.time_cost
      reward = 0
      # Reward shaping
      # Phase I: reward the agent for learning to stop 
      # Always: reward the agent for capturing fireflies
      self.num_targets = 0
      if abs(self.sys_vel[1]) <= self.terminal_vel:
        captured_ff_index = (self.ff_distance_all <= self.reward_boundary).nonzero().reshape(-1).tolist()
        self.captured_ff_index = captured_ff_index
        num_targets = len(captured_ff_index)
        self.num_targets = num_targets
        if num_targets > 0: # If the monkey hs captured at least 1 ff
          # Calculate reward
          reward = reward + self.reward_per_ff * num_targets
          # Replace the captured ffs with ffs of new locations
          self.ffr[captured_ff_index]= torch.sqrt(torch.rand(num_targets))*self.arena_radius
          self.fftheta[captured_ff_index]= torch.rand(num_targets)*2*pi
          #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
          self.ffx[captured_ff_index] = torch.cos(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffy[captured_ff_index] = torch.sin(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
          self.ffx2[captured_ff_index] = self.ffx[captured_ff_index].clone()
          self.ffy2[captured_ff_index] = self.ffy[captured_ff_index].clone()
          self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
          # Delete the information from self.ff_info
          #[self.ff_info.pop(key) for key in captured_ff_index if (key in self.ff_info)]
          #self.current_target_index = torch.tensor([999], dtype=torch.int32)
          #self.previous_target_index = self.current_target_index
          # Reward the firefly based on the average speed before capturing the firefly
          ##reward = reward + sum(self.past_speeds)/len(self.past_speeds)
          #print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "obs: ", list(np.round(self.obs, decimals = 2)), "n_targets: ",  num_targets)
          print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "n_targets: ",  num_targets)
          #self.update_slots = True
          #self.stop_rewarding_speed = True
        #elif self.has_sped_up_before == True:
        # based on Ruiyi's formula, using the distance of the closest ff in obs
        #reward += math.exp(-((self.ff_current[1, 0]**2)*(25/1.5)**2)/2)
        #self.has_sped_up_before = False
      return reward







  def step(self, action):
    self.time += self.dt
    action=torch.tensor(action)
    action[1] = action[1]/2+0.5
    self.sys_vel=action.clone()
    self.state_step(action)
    self.obs = self.beliefs().numpy()
    reward=self.calculate_reward()
    self.episode_reward += reward
    
    if self.time >= self.episode_len*self.dt:
      self.end_episode = True
      #self.current_episode += 1
      print("Reward for the episode: ", self.episode_reward)
    #print("action: ", torch.round(action, decimals = 3), "obs: ", np.round(self.obs, decimals = 3), "Reward: ", round(reward, 3))
    return self.obs, reward, self.end_episode, {}




  def state_step(self,action):
    vnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std 
    wnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std
    self.dw_normed = (action[0]+wnoise)
    self.dv_normed = (action[1]+vnoise)
    self.dw = (action[0]+wnoise)*self.wgain*self.dt
    self.agentheading = self.agentheading + self.dw.item()
    self.dv =  (action[1]+vnoise)*self.vgain*self.dt
    self.dx= torch.cos(self.agentheading)*self.dv
    self.dy= torch.sin(self.agentheading)*self.dv
    self.agentx = self.agentx + self.dx.item()
    self.agenty = self.agenty + self.dy.item()
    self.agentxy = torch.cat((self.agentx, self.agenty))
    self.agentr = vector_norm(self.agentxy)
    self.agenttheta = torch.tensor(torch.atan2(self.agenty, self.agentx))  
                               
    if self.agentr >= self.arena_radius:
      self.agentr = 2*self.arena_radius-self.agentr
      self.agenttheta = self.agenttheta + pi
      self.agentx = (self.agentr*torch.cos(self.agenttheta)).reshape(1,)
      self.agenty = (self.agentr*torch.sin(self.agenttheta)).reshape(1,)
      self.agentxy = torch.cat((self.agentx, self.agenty))
      self.agentheading = self.agenttheta - pi
    while self.agentheading >= 2*pi:
      self.agentheading = self.agentheading - 2*pi
    while self.agentheading < 0:
      self.agentheading = self.agentheading + 2*pi   








  def beliefs(self): 

    # Make a tensor containing the relative distance of all fireflies to the agent
    self.ff_distance_all = vector_norm(self.ffxy - self.agentxy, dim=1)
    # Make a tensor containing the relative (real) angle of all fireflies to the agent
    ffradians = torch.atan2(self.ffy-self.agenty, self.ffx-self.agentx)
    angle0 = ffradians - self.agentheading
    angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
    angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
    # Adjust the angle based on reward boundary
    angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip( self.ff_distance_all, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
    angle2 = torch.clip(angle1,0,pi)
    ff_angle_all = torch.sign(angle0)* angle2
    # Update the tensor containing the uncertainties of all fireflies to the agent
    visible_ff = torch.logical_and( self.ff_distance_all < self.invisible_distance, torch.abs(ff_angle_all) < self.invisible_angle)
    self.visible_ff_indices0 = visible_ff.nonzero().reshape(-1)
    for index in self.visible_ff_indices0:
      ff = self.ff_flash[index]
      if not torch.any(torch.logical_and(ff[:, 0] <= self.time, ff[:, 1] >= self.time)):
        visible_ff[index] = False
    self.visible_ff_indices = visible_ff.nonzero().reshape(-1)
    # Update memory
    self.ff_memory_all -= 1
    self.ff_memory_all[self.visible_ff_indices] = self.full_memory
    self.ff_memory_all = torch.clamp(self.ff_memory_all, 0, self.full_memory)
    # Calculate the uncertainties that will be added to relative distance and angle based on memory
    ff_uncertainty_all = (self.full_memory-self.ff_memory_all)*self.internal_noise_factor
    self.ffx2 = self.ffx2 + torch.normal(torch.zeros([self.num_ff,]), ff_uncertainty_all)
    self.ffy2 = self.ffy2 + torch.normal(torch.zeros([self.num_ff,]), ff_uncertainty_all)
    self.ffx2[self.visible_ff_indices] = self.ffx[self.visible_ff_indices].clone()
    self.ffy2[self.visible_ff_indices] = self.ffy[self.visible_ff_indices].clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    # find ffs that are in memory
    self.ff_in_memory_indices = (self.ff_memory_all > 0).nonzero().reshape(-1)
    # Consider the case where there are fewer than self.obs_ff fireflies that are in memory
    

   
    if torch.numel(self.ff_in_memory_indices) >= self.obs_ff:
      # Rank the ff whose "memory" is creater than 0 based on distance
      sorted_indices =  torch.topk(-self.ff_distance_all[self.ff_in_memory_indices], self.obs_ff).indices
      self.topk_indices = self.ff_in_memory_indices[sorted_indices]
      self.ffxy2_topk = self.ffxy2[self.topk_indices]
      self.ff_distance_topk = vector_norm(self.ffxy2_topk - self.agentxy, dim=1)
      # Calculate relative angles 
      ffradians = torch.atan2(self.ffxy2_topk[:,1]-self.agenty, self.ffxy2_topk[:,0]-self.agentx)
      angle0 = ffradians - self.agentheading
      angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      self.ff_angle_topk_2 = angle0.clone()
      # Calculate relative angles of all ffs based on reward boundaries
      # Adjust the angle based on reward boundary
      angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      angle2 = torch.clip(angle1,0,pi)
      ff_angle_topk_3 = torch.sign(angle0)* angle2
      # Concatenate distance, angle, and memory
      ff_array = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk,  self.ff_memory_all[self.topk_indices]), dim=0)
    

    elif torch.numel(self.ff_in_memory_indices) == 0:
      ff_array = torch.tensor([[0], [0], [self.invisible_distance], [0]]).repeat([1, self.obs_ff])
      self.topk_indices = torch.tensor([])

    else:
      sorted_distance, sorted_indices = torch.sort(-self.ff_distance_all[self.ff_in_memory_indices])
      self.topk_indices = self.ff_in_memory_indices[sorted_indices]
      self.ffxy2_topk = self.ffxy2[self.topk_indices ]
      self.ff_distance_topk = vector_norm(self.ffxy2_topk - self.agentxy, dim=1)
      # Calculate relative angles 
      ffradians = torch.atan2(self.ffxy2_topk[:,1]-self.agenty, self.ffxy2_topk[:,0]-self.agentx)
      angle0 = ffradians - self.agentheading
      angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      self.ff_angle_topk_2 = angle0.clone()
      # Calculate relative angles of all ffs based on reward boundaries
      # Adjust the angle based on reward boundary
      angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      angle2 = torch.clip(angle1,0,pi)
      ff_angle_topk_3 = torch.sign(angle0)* angle2
      # Concatenate distance, angle, and memory
      ff_array0 = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk,  self.ff_memory_all[self.topk_indices]), dim=0)
      needed_ff = self.obs_ff - torch.numel(self.ff_in_memory_indices)
      ff_array = torch.stack([ff_array0.reshape([4,-1]), torch.tensor([[0], [0], [self.invisible_distance], [0]]).repeat([1, needed_ff])], dim=1)
      

    # ff_array[0:2,:] = ff_array[0:2,:]/pi
    # ff_array[2,:] = (ff_array[2,:]/self.invisible_distance-0.5)*2
    # ff_array[3,:] = (ff_array[3,:]/20-0.5)*2
    self.ff_array = ff_array.clone()
    return torch.flatten(ff_array.transpose(0, 1))


env = MultiFF() 
env.reset() 

sac_model = SAC("MlpPolicy", 
            env,
            buffer_size=int(1e6),
            batch_size=1024,
            device='auto',
            verbose=False,
            train_freq=100,
            learning_starts = int(10),
            target_update_interval=20,
            learning_rate=1e-2,
            gamma=0.9999,
            policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[64, 64])
                )


### c12: noise = 1.5, memory = 4

self.dt = 0.25
discount factor = 0.995


In [ ]:
retrieve_dir = "/content/gdrive/MyDrive/fireflies_agent/July_20_6/"

In [ ]:
# obs = [angle, distance, memory, angle2, distance2, memory2, ...]
# before = [angle, angle2 ..., distance, distance2, ... memory, memory2]

class MultiFF(Env):
  def __init__(self):
      super(MultiFF, self).__init__()
      self.num_ff = 200
      self.arena_radius = 1000
      self.episode_len = 10000 
      self.dt = 0.25 
      #self.current_episode = 0
      #self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.obs_ff = 2
      self.observation_space = spaces.Box(low=-1., high=1., shape=(self.obs_ff*4,),dtype=np.float32)
      self.terminal_vel = 0.01
      #self.pro_noise_std = 0.005
      self.vgain = 200
      self.wgain = pi/2
      self.reward_per_ff = 100
      #self.time_cost = 0.005
      #self.total_time = 0
      #self.zero_action = False
      #self.target_update_counter_num = 20
      #self.reward_per_episode = []
      #self.update_slots = True
      #self.closest_ff_distance = 200
      self.pro_noise_std = 0.005
      self.epi_num = -1
      self.has_sped_up_before = False
      self.full_memory = 4
      self.internal_noise_factor = 1.5
      self.ff_memory_all = torch.ones([self.num_ff,])*self.full_memory
      self.invisible_distance = 400
      self.invisible_angle = 2*pi/9
      self.reward_boundary = 25
      






  def reset(self):
    self.epi_num += 1
    print("\n episode: ", self.epi_num)
    self.num_targets = 0
    #self.past_speeds = []
    self.time = 0
    #self.counter = 0
    #self.current_target_index = torch.tensor([999], dtype=torch.int32)
    #self.previous_target_index = self.current_target_index
    self.ff_flash = []
    self.has_sped_up_before = False
    
    for i in range(self.num_ff):
      num_intervals = 1500
      first_flash = torch.rand(1)
      intervals = torch.poisson(torch.ones(num_intervals-1)*3)
      t0 = torch.cat((first_flash, first_flash+torch.cumsum(intervals, dim=0)+torch.cumsum(torch.ones(num_intervals-1)*0.3, dim=0)))
      t1 = t0 + torch.ones(num_intervals)*0.3
      self.ff_flash.append(torch.stack((t0, t1), dim=1))

    self.ffr = torch.sqrt(torch.rand(self.num_ff))*self.arena_radius # The radius of the arena changed from 1000 cm/s to 200 cm/s
    self.fftheta = torch.rand(self.num_ff)*2*pi
    #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
    self.ffx = torch.cos(self.fftheta) * self.ffr
    self.ffy = torch.sin(self.fftheta) * self.ffr
    self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
    self.ffx2 = self.ffx.clone()
    self.ffy2 = self.ffy.clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    #self.ff_info={}
    #self.update_slots = True
    self.agentx = torch.tensor([0])
    self.agenty = torch.tensor([0])
    self.agentr = torch.zeros(1)
    self.agentxy = torch.tensor([0, 0])
    self.agentheading = torch.zeros(1).uniform_(0, 2*pi)
    self.dv = torch.zeros(1).uniform_(-0.05, 0.05) 
    self.dw = torch.zeros(1)
    self.end_episode = False 
    self.obs = self.beliefs().numpy()
    #self.chunk_50s = 1
    self.episode_reward = 0
    #self.stop_rewarding_speed = False
    #self.current_obs_steps = 0
    self.ff_memory_all = torch.ones([self.num_ff,])
    return self.obs






  def calculate_reward(self):
      #action_cost=((self.previous_action[1]-self.action[1])**2+(self.previous_action[0]-self.action[0])**2)*self.mag_cost
      # To incorporate action_cost, we need to incorporate previous_action into decision_info
      #self.total_time += 1
      # In addition to rewarding the monkey for capturing the firefly, we also use different phases of rewards to teach monkey specific behaviours
      #reward = -self.time_cost
      reward = 0
      # Reward shaping
      # Phase I: reward the agent for learning to stop 
      # Always: reward the agent for capturing fireflies
      self.num_targets = 0
      if abs(self.sys_vel[1]) <= self.terminal_vel:
        captured_ff_index = (self.ff_distance_all <= self.reward_boundary).nonzero().reshape(-1).tolist()
        self.captured_ff_index = captured_ff_index
        num_targets = len(captured_ff_index)
        self.num_targets = num_targets
        if num_targets > 0: # If the monkey hs captured at least 1 ff
          # Calculate reward
          reward = reward + self.reward_per_ff * num_targets
          # Replace the captured ffs with ffs of new locations
          self.ffr[captured_ff_index]= torch.sqrt(torch.rand(num_targets))*self.arena_radius
          self.fftheta[captured_ff_index]= torch.rand(num_targets)*2*pi
          #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
          self.ffx[captured_ff_index] = torch.cos(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffy[captured_ff_index] = torch.sin(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
          self.ffx2[captured_ff_index] = self.ffx[captured_ff_index].clone()
          self.ffy2[captured_ff_index] = self.ffy[captured_ff_index].clone()
          self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
          # Delete the information from self.ff_info
          #[self.ff_info.pop(key) for key in captured_ff_index if (key in self.ff_info)]
          #self.current_target_index = torch.tensor([999], dtype=torch.int32)
          #self.previous_target_index = self.current_target_index
          # Reward the firefly based on the average speed before capturing the firefly
          ##reward = reward + sum(self.past_speeds)/len(self.past_speeds)
          #print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "obs: ", list(np.round(self.obs, decimals = 2)), "n_targets: ",  num_targets)
          print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "n_targets: ",  num_targets)
          #self.update_slots = True
          #self.stop_rewarding_speed = True
        #elif self.has_sped_up_before == True:
        # based on Ruiyi's formula, using the distance of the closest ff in obs
        #reward += math.exp(-((self.ff_current[1, 0]**2)*(25/1.5)**2)/2)
        #self.has_sped_up_before = False
      return reward







  def step(self, action):
    self.time += self.dt
    action=torch.tensor(action)
    action[1] = action[1]/2+0.5
    self.sys_vel=action.clone()
    self.state_step(action)
    self.obs = self.beliefs().numpy()
    reward=self.calculate_reward()
    self.episode_reward += reward
    
    if self.time >= self.episode_len*self.dt:
      self.end_episode = True
      #self.current_episode += 1
      print("Reward for the episode: ", self.episode_reward)
    #print("action: ", torch.round(action, decimals = 3), "obs: ", np.round(self.obs, decimals = 3), "Reward: ", round(reward, 3))
    return self.obs, reward, self.end_episode, {}




  def state_step(self,action):
    vnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std 
    wnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std
    self.dw_normed = (action[0]+wnoise)
    self.dv_normed = (action[1]+vnoise)
    self.dw = (action[0]+wnoise)*self.wgain*self.dt
    self.agentheading = self.agentheading + self.dw.item()
    self.dv =  (action[1]+vnoise)*self.vgain*self.dt
    self.dx= torch.cos(self.agentheading)*self.dv
    self.dy= torch.sin(self.agentheading)*self.dv
    self.agentx = self.agentx + self.dx.item()
    self.agenty = self.agenty + self.dy.item()
    self.agentxy = torch.cat((self.agentx, self.agenty))
    self.agentr = vector_norm(self.agentxy)
    self.agenttheta = torch.tensor(torch.atan2(self.agenty, self.agentx))  
                               
    if self.agentr >= self.arena_radius:
      self.agentr = 2*self.arena_radius-self.agentr
      self.agenttheta = self.agenttheta + pi
      self.agentx = (self.agentr*torch.cos(self.agenttheta)).reshape(1,)
      self.agenty = (self.agentr*torch.sin(self.agenttheta)).reshape(1,)
      self.agentxy = torch.cat((self.agentx, self.agenty))
      self.agentheading = self.agenttheta - pi
    while self.agentheading >= 2*pi:
      self.agentheading = self.agentheading - 2*pi
    while self.agentheading < 0:
      self.agentheading = self.agentheading + 2*pi   








  def beliefs(self): 

    # Make a tensor containing the relative distance of all fireflies to the agent
    self.ff_distance_all = vector_norm(self.ffxy - self.agentxy, dim=1)
    # Make a tensor containing the relative (real) angle of all fireflies to the agent
    ffradians = torch.atan2(self.ffy-self.agenty, self.ffx-self.agentx)
    angle0 = ffradians - self.agentheading
    angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
    angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
    # Adjust the angle based on reward boundary
    angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip( self.ff_distance_all, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
    angle2 = torch.clip(angle1,0,pi)
    ff_angle_all = torch.sign(angle0)* angle2
    # Update the tensor containing the uncertainties of all fireflies to the agent
    visible_ff = torch.logical_and( self.ff_distance_all < self.invisible_distance, torch.abs(ff_angle_all) < self.invisible_angle)
    self.visible_ff_indices0 = visible_ff.nonzero().reshape(-1)
    for index in self.visible_ff_indices0:
      ff = self.ff_flash[index]
      if not torch.any(torch.logical_and(ff[:, 0] <= self.time, ff[:, 1] >= self.time)):
        visible_ff[index] = False
    self.visible_ff_indices = visible_ff.nonzero().reshape(-1)
    # Update memory
    self.ff_memory_all -= 1
    self.ff_memory_all[self.visible_ff_indices] = self.full_memory
    self.ff_memory_all = torch.clamp(self.ff_memory_all, 0, self.full_memory)
    # Calculate the uncertainties that will be added to relative distance and angle based on memory
    ff_uncertainty_all = (self.full_memory-self.ff_memory_all)*self.internal_noise_factor
    self.ffx2 = self.ffx2 + torch.normal(torch.zeros([self.num_ff,]), ff_uncertainty_all)
    self.ffy2 = self.ffy2 + torch.normal(torch.zeros([self.num_ff,]), ff_uncertainty_all)
    self.ffx2[self.visible_ff_indices] = self.ffx[self.visible_ff_indices].clone()
    self.ffy2[self.visible_ff_indices] = self.ffy[self.visible_ff_indices].clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    # find ffs that are in memory
    self.ff_in_memory_indices = (self.ff_memory_all > 0).nonzero().reshape(-1)
    # Consider the case where there are fewer than self.obs_ff fireflies that are in memory
    

   
    if torch.numel(self.ff_in_memory_indices) >= self.obs_ff:
      # Rank the ff whose "memory" is creater than 0 based on distance
      sorted_indices =  torch.topk(-self.ff_distance_all[self.ff_in_memory_indices], self.obs_ff).indices
      self.topk_indices = self.ff_in_memory_indices[sorted_indices]
      self.ffxy2_topk = self.ffxy2[self.topk_indices]
      self.ff_distance_topk = vector_norm(self.ffxy2_topk - self.agentxy, dim=1)
      # Calculate relative angles 
      ffradians = torch.atan2(self.ffxy2_topk[:,1]-self.agenty, self.ffxy2_topk[:,0]-self.agentx)
      angle0 = ffradians - self.agentheading
      angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      self.ff_angle_topk_2 = angle0.clone()
      # Calculate relative angles of all ffs based on reward boundaries
      # Adjust the angle based on reward boundary
      angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      angle2 = torch.clip(angle1,0,pi)
      ff_angle_topk_3 = torch.sign(angle0)* angle2
      # Concatenate distance, angle, and memory
      ff_array = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk,  self.ff_memory_all[self.topk_indices]), dim=0)
    

    elif torch.numel(self.ff_in_memory_indices) == 0:
      ff_array = torch.tensor([[0], [0], [self.invisible_distance], [0]]).repeat([1, self.obs_ff])
      self.topk_indices = torch.tensor([])

    else:
      sorted_distance, sorted_indices = torch.sort(-self.ff_distance_all[self.ff_in_memory_indices])
      self.topk_indices = self.ff_in_memory_indices[sorted_indices]
      self.ffxy2_topk = self.ffxy2[self.topk_indices ]
      self.ff_distance_topk = vector_norm(self.ffxy2_topk - self.agentxy, dim=1)
      # Calculate relative angles 
      ffradians = torch.atan2(self.ffxy2_topk[:,1]-self.agenty, self.ffxy2_topk[:,0]-self.agentx)
      angle0 = ffradians - self.agentheading
      angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      self.ff_angle_topk_2 = angle0.clone()
      # Calculate relative angles of all ffs based on reward boundaries
      # Adjust the angle based on reward boundary
      angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      angle2 = torch.clip(angle1,0,pi)
      ff_angle_topk_3 = torch.sign(angle0)* angle2
      # Concatenate distance, angle, and memory
      ff_array0 = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk,  self.ff_memory_all[self.topk_indices]), dim=0)
      needed_ff = self.obs_ff - torch.numel(self.ff_in_memory_indices)
      ff_array = torch.stack([ff_array0.reshape([4,-1]), torch.tensor([[0], [0], [self.invisible_distance], [0]]).repeat([1, needed_ff])], dim=1)
      

    # ff_array[0:2,:] = ff_array[0:2,:]/pi
    # ff_array[2,:] = (ff_array[2,:]/self.invisible_distance-0.5)*2
    # ff_array[3,:] = (ff_array[3,:]/20-0.5)*2
    self.ff_array = ff_array.clone()
    return torch.flatten(ff_array.transpose(0, 1))


env = MultiFF() 
env.reset() 

sac_model = SAC("MlpPolicy", 
            env,
            buffer_size=int(1e6),
            batch_size=1024,
            device='auto',
            verbose=False,
            train_freq=100,
            learning_starts = int(10),
            target_update_interval=20,
            learning_rate=1e-2,
            gamma=0.9999,
            policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[64, 64])
                )


### c16: c15 + noise = 2, memory = 3

(so that its epi_len is the same as LSTM)

self.dt = 0.25
discount factor = 0.995


In [ ]:
retrieve_dir = "/content/gdrive/MyDrive/fireflies_agent/Aug_10_2/" # discount factor = 0.995  
# eval rewards: 6720
path = os.path.join(retrieve_dir, 'best_model.zip')
path2 = os.path.join(retrieve_dir, 'buffer.pkl')
sac_model = sac_model.load(path,env=env)  
sac_model.load_replay_buffer(path2)

In [ ]:
# obs = [angle, distance, memory, angle2, distance2, memory2, ...]
# before = [angle, angle2 ..., distance, distance2, ... memory, memory2]

class MultiFF(Env):
  def __init__(self):
      super(MultiFF, self).__init__()
      self.num_ff = 200
      self.arena_radius = 1000
      self.episode_len = 10000 
      self.dt = 0.25 
      #self.current_episode = 0
      #self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.obs_ff = 2
      self.observation_space = spaces.Box(low=-1., high=1., shape=(self.obs_ff*4,),dtype=np.float32)
      self.terminal_vel = 0.01
      #self.pro_noise_std = 0.005
      self.vgain = 200
      self.wgain = pi/2
      self.reward_per_ff = 100
      #self.time_cost = 0.005
      #self.total_time = 0
      #self.zero_action = False
      #self.target_update_counter_num = 20
      #self.reward_per_episode = []
      #self.update_slots = True
      #self.closest_ff_distance = 200
      self.pro_noise_std = 0.005
      self.epi_num = -1
      self.has_sped_up_before = False
      self.full_memory = 3
      self.internal_noise_factor = 2
      self.ff_memory_all = torch.ones([self.num_ff,])*self.full_memory
      self.invisible_distance = 400
      self.invisible_angle = 2*pi/9
      self.reward_boundary = 25
      






  def reset(self):
    self.epi_num += 1
    print("\n episode: ", self.epi_num)
    self.num_targets = 0
    #self.past_speeds = []
    self.time = 0
    #self.counter = 0
    #self.current_target_index = torch.tensor([999], dtype=torch.int32)
    #self.previous_target_index = self.current_target_index
    self.ff_flash = []
    self.has_sped_up_before = False
    
    for i in range(self.num_ff):
      num_intervals = 1500
      first_flash = torch.rand(1)
      intervals = torch.poisson(torch.ones(num_intervals-1)*3)
      t0 = torch.cat((first_flash, first_flash+torch.cumsum(intervals, dim=0)+torch.cumsum(torch.ones(num_intervals-1)*0.3, dim=0)))
      t1 = t0 + torch.ones(num_intervals)*0.3
      self.ff_flash.append(torch.stack((t0, t1), dim=1))

    self.ffr = torch.sqrt(torch.rand(self.num_ff))*self.arena_radius # The radius of the arena changed from 1000 cm/s to 200 cm/s
    self.fftheta = torch.rand(self.num_ff)*2*pi
    #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
    self.ffx = torch.cos(self.fftheta) * self.ffr
    self.ffy = torch.sin(self.fftheta) * self.ffr
    self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
    self.ffx2 = self.ffx.clone()
    self.ffy2 = self.ffy.clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    #self.ff_info={}
    #self.update_slots = True
    self.agentx = torch.tensor([0])
    self.agenty = torch.tensor([0])
    self.agentr = torch.zeros(1)
    self.agentxy = torch.tensor([0, 0])
    self.agentheading = torch.zeros(1).uniform_(0, 2*pi)
    self.dv = torch.zeros(1).uniform_(-0.05, 0.05) 
    self.dw = torch.zeros(1)
    self.end_episode = False 
    self.obs = self.beliefs().numpy()
    #self.chunk_50s = 1
    self.episode_reward = 0
    #self.stop_rewarding_speed = False
    #self.current_obs_steps = 0
    self.ff_memory_all = torch.ones([self.num_ff,])
    return self.obs






  def calculate_reward(self):
      #action_cost=((self.previous_action[1]-self.action[1])**2+(self.previous_action[0]-self.action[0])**2)*self.mag_cost
      # To incorporate action_cost, we need to incorporate previous_action into decision_info
      #self.total_time += 1
      # In addition to rewarding the monkey for capturing the firefly, we also use different phases of rewards to teach monkey specific behaviours
      #reward = -self.time_cost
      reward = 0
      # Reward shaping
      # Phase I: reward the agent for learning to stop 
      # Always: reward the agent for capturing fireflies
      self.num_targets = 0
      if abs(self.sys_vel[1]) <= self.terminal_vel:
        captured_ff_index = (self.ff_distance_all <= self.reward_boundary).nonzero().reshape(-1).tolist()
        self.captured_ff_index = captured_ff_index
        num_targets = len(captured_ff_index)
        self.num_targets = num_targets
        if num_targets > 0: # If the monkey hs captured at least 1 ff
          # Calculate reward
          reward = reward + self.reward_per_ff * num_targets
          # Replace the captured ffs with ffs of new locations
          self.ffr[captured_ff_index]= torch.sqrt(torch.rand(num_targets))*self.arena_radius
          self.fftheta[captured_ff_index]= torch.rand(num_targets)*2*pi
          #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
          self.ffx[captured_ff_index] = torch.cos(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffy[captured_ff_index] = torch.sin(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
          self.ffx2[captured_ff_index] = self.ffx[captured_ff_index].clone()
          self.ffy2[captured_ff_index] = self.ffy[captured_ff_index].clone()
          self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
          # Delete the information from self.ff_info
          #[self.ff_info.pop(key) for key in captured_ff_index if (key in self.ff_info)]
          #self.current_target_index = torch.tensor([999], dtype=torch.int32)
          #self.previous_target_index = self.current_target_index
          # Reward the firefly based on the average speed before capturing the firefly
          ##reward = reward + sum(self.past_speeds)/len(self.past_speeds)
          #print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "obs: ", list(np.round(self.obs, decimals = 2)), "n_targets: ",  num_targets)
          print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "n_targets: ",  num_targets)
          #self.update_slots = True
          #self.stop_rewarding_speed = True
        #elif self.has_sped_up_before == True:
        # based on Ruiyi's formula, using the distance of the closest ff in obs
        #reward += math.exp(-((self.ff_current[1, 0]**2)*(25/1.5)**2)/2)
        #self.has_sped_up_before = False
      return reward







  def step(self, action):
    self.time += self.dt
    action=torch.tensor(action)
    action[1] = action[1]/2+0.5
    self.sys_vel=action.clone()
    self.state_step(action)
    self.obs = self.beliefs().numpy()
    reward=self.calculate_reward()
    self.episode_reward += reward
    
    if self.time >= self.episode_len*self.dt:
      self.end_episode = True
      #self.current_episode += 1
      print("Reward for the episode: ", self.episode_reward)
    #print("action: ", torch.round(action, decimals = 3), "obs: ", np.round(self.obs, decimals = 3), "Reward: ", round(reward, 3))
    return self.obs, reward, self.end_episode, {}




  def state_step(self,action):
    vnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std 
    wnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std
    self.dw_normed = (action[0]+wnoise)
    self.dv_normed = (action[1]+vnoise)
    self.dw = (action[0]+wnoise)*self.wgain*self.dt
    self.agentheading = self.agentheading + self.dw.item()
    self.dv =  (action[1]+vnoise)*self.vgain*self.dt
    self.dx= torch.cos(self.agentheading)*self.dv
    self.dy= torch.sin(self.agentheading)*self.dv
    self.agentx = self.agentx + self.dx.item()
    self.agenty = self.agenty + self.dy.item()
    self.agentxy = torch.cat((self.agentx, self.agenty))
    self.agentr = vector_norm(self.agentxy)
    self.agenttheta = torch.tensor(torch.atan2(self.agenty, self.agentx))  
                               
    if self.agentr >= self.arena_radius:
      self.agentr = 2*self.arena_radius-self.agentr
      self.agenttheta = self.agenttheta + pi
      self.agentx = (self.agentr*torch.cos(self.agenttheta)).reshape(1,)
      self.agenty = (self.agentr*torch.sin(self.agenttheta)).reshape(1,)
      self.agentxy = torch.cat((self.agentx, self.agenty))
      self.agentheading = self.agenttheta - pi
    while self.agentheading >= 2*pi:
      self.agentheading = self.agentheading - 2*pi
    while self.agentheading < 0:
      self.agentheading = self.agentheading + 2*pi   








  def beliefs(self): 

    # Make a tensor containing the relative distance of all fireflies to the agent
    self.ff_distance_all = vector_norm(self.ffxy - self.agentxy, dim=1)
    # Make a tensor containing the relative (real) angle of all fireflies to the agent
    ffradians = torch.atan2(self.ffy-self.agenty, self.ffx-self.agentx)
    angle0 = ffradians - self.agentheading
    angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
    angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
    # Adjust the angle based on reward boundary
    angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip( self.ff_distance_all, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
    angle2 = torch.clip(angle1,0,pi)
    ff_angle_all = torch.sign(angle0)* angle2
    # Update the tensor containing the uncertainties of all fireflies to the agent
    visible_ff = torch.logical_and( self.ff_distance_all < self.invisible_distance, torch.abs(ff_angle_all) < self.invisible_angle)
    self.visible_ff_indices0 = visible_ff.nonzero().reshape(-1)
    for index in self.visible_ff_indices0:
      ff = self.ff_flash[index]
      if not torch.any(torch.logical_and(ff[:, 0] <= self.time, ff[:, 1] >= self.time)):
        visible_ff[index] = False
    self.visible_ff_indices = visible_ff.nonzero().reshape(-1)
    # Update memory
    self.ff_memory_all -= 1
    self.ff_memory_all[self.visible_ff_indices] = self.full_memory
    self.ff_memory_all = torch.clamp(self.ff_memory_all, 0, self.full_memory)
    # Calculate the uncertainties that will be added to relative distance and angle based on memory
    ff_uncertainty_all = (self.full_memory-self.ff_memory_all)*self.internal_noise_factor
    self.ffx2 = self.ffx2 + torch.normal(torch.zeros([self.num_ff,]), ff_uncertainty_all)
    self.ffy2 = self.ffy2 + torch.normal(torch.zeros([self.num_ff,]), ff_uncertainty_all)
    self.ffx2[self.visible_ff_indices] = self.ffx[self.visible_ff_indices].clone()
    self.ffy2[self.visible_ff_indices] = self.ffy[self.visible_ff_indices].clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    # find ffs that are in memory
    self.ff_in_memory_indices = (self.ff_memory_all > 0).nonzero().reshape(-1)
    # Consider the case where there are fewer than self.obs_ff fireflies that are in memory
    

   
    if torch.numel(self.ff_in_memory_indices) >= self.obs_ff:
      # Rank the ff whose "memory" is creater than 0 based on distance
      sorted_indices =  torch.topk(-self.ff_distance_all[self.ff_in_memory_indices], self.obs_ff).indices
      self.topk_indices = self.ff_in_memory_indices[sorted_indices]
      self.ffxy2_topk = self.ffxy2[self.topk_indices]
      self.ff_distance_topk = vector_norm(self.ffxy2_topk - self.agentxy, dim=1)
      # Calculate relative angles 
      ffradians = torch.atan2(self.ffxy2_topk[:,1]-self.agenty, self.ffxy2_topk[:,0]-self.agentx)
      angle0 = ffradians - self.agentheading
      angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      self.ff_angle_topk_2 = angle0.clone()
      # Calculate relative angles of all ffs based on reward boundaries
      # Adjust the angle based on reward boundary
      angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      angle2 = torch.clip(angle1,0,pi)
      ff_angle_topk_3 = torch.sign(angle0)* angle2
      # Concatenate distance, angle, and memory
      ff_array = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk,  self.ff_memory_all[self.topk_indices]), dim=0)
    

    elif torch.numel(self.ff_in_memory_indices) == 0:
      ff_array = torch.tensor([[0], [0], [self.invisible_distance], [0]]).repeat([1, self.obs_ff])
      self.topk_indices = torch.tensor([])

    else:
      sorted_distance, sorted_indices = torch.sort(-self.ff_distance_all[self.ff_in_memory_indices])
      self.topk_indices = self.ff_in_memory_indices[sorted_indices]
      self.ffxy2_topk = self.ffxy2[self.topk_indices ]
      self.ff_distance_topk = vector_norm(self.ffxy2_topk - self.agentxy, dim=1)
      # Calculate relative angles 
      ffradians = torch.atan2(self.ffxy2_topk[:,1]-self.agenty, self.ffxy2_topk[:,0]-self.agentx)
      angle0 = ffradians - self.agentheading
      angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      self.ff_angle_topk_2 = angle0.clone()
      # Calculate relative angles of all ffs based on reward boundaries
      # Adjust the angle based on reward boundary
      angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      angle2 = torch.clip(angle1,0,pi)
      ff_angle_topk_3 = torch.sign(angle0)* angle2
      # Concatenate distance, angle, and memory
      ff_array0 = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk,  self.ff_memory_all[self.topk_indices]), dim=0)
      needed_ff = self.obs_ff - torch.numel(self.ff_in_memory_indices)
      ff_array = torch.stack([ff_array0.reshape([4,-1]), torch.tensor([[0], [0], [self.invisible_distance], [0]]).repeat([1, needed_ff])], dim=1)
      

    # ff_array[0:2,:] = ff_array[0:2,:]/pi
    # ff_array[2,:] = (ff_array[2,:]/self.invisible_distance-0.5)*2
    # ff_array[3,:] = (ff_array[3,:]/20-0.5)*2
    self.ff_array = ff_array.clone()
    return torch.flatten(ff_array.transpose(0, 1))


env = MultiFF() 
env.reset() 

sac_model = SAC("MlpPolicy", 
            env,
            buffer_size=int(1e6),
            batch_size=1024,
            device='auto',
            verbose=False,
            train_freq=100,
            learning_starts = int(10),
            target_update_interval=20,
            learning_rate=1e-2,
            gamma=0.9999,
            policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[64, 64])
                )


### Load

In [ ]:
class CollectInformation(MultiFF):  # Note when using this wrapper, the number of steps cannot exceed one episode
  def __init__(self):
      super().__init__()
      self.ff_information = np.ones([self.num_ff, 8])*(-9999)   #[index, x, y, time_start, time_captured, mx(when_captured), my(when_captured), index_in_flash]

  def reset(self):
      self.obs = super().reset()
      self.ff_information[:,0] = np.arange(self.num_ff)
      self.ff_information[:,7] = np.arange(self.num_ff)
      self.ff_information[:,1] = self.ffx.numpy()
      self.ff_information[:,2] = self.ffy.numpy()
      self.ff_information[:,3] = 0
      return self.obs

  def calculate_reward(self):
      reward = super().calculate_reward()    
      if self.num_targets > 0: 
        for index in self.captured_ff_index:
          overall_index = int(self.ff_information[:,0][np.where(self.ff_information[:,-1]==index)[0][-1]])
          self.ff_information[overall_index, 4] = self.time
          self.ff_information[overall_index, 5] = self.agentx.item()
          self.ff_information[overall_index, 6] = self.agenty.item()
        self.new_ff_info = np.ones([self.num_targets, 8])*(-9999)
        self.new_ff_info[:,0] = np.arange(len(self.ff_information), len(self.ff_information)+self.num_targets)
        self.new_ff_info[:,7] = np.array(self.captured_ff_index)
        self.new_ff_info[:,1] = self.ffx[self.captured_ff_index].numpy()
        self.new_ff_info[:,2] = self.ffy[self.captured_ff_index].numpy()
        self.new_ff_info[:,3] = self.time
        self.ff_information = np.concatenate([self.ff_information, self.new_ff_info], axis = 0)
      return(reward)
  #def reset_wrapper(self):



env = CollectInformation()


# Create and wrap the environment
log_dir = "/content/gdrive/MyDrive/fireflies_agent/July_20_6/" # discount factor = 0.999
os.makedirs(log_dir, exist_ok=True)

env = Monitor(env, log_dir)
env.reset() 

env.flash_on_interval = 0.3
env.distance2center_cost = 0

# For direct training
sac_model = SAC("MlpPolicy", 
            env,
            gamma=0.995,
            learning_rate=0.0015,
            batch_size=1024,
            target_update_interval=50,
            buffer_size=1000000,
            learning_starts=10000,
            train_freq=10,
            ent_coef=0.00083,
            policy_kwargs=dict(activation_fn=nn.Tanh, net_arch=[128, 128])
                )


path = os.path.join(retrieve_dir, 'best_model.zip')
path2 = os.path.join(retrieve_dir, 'buffer.pkl')
sac_model = sac_model.load(path,env=env)  
sac_model.load_replay_buffer(path2)

### Test (SB3)

note: num_steps cannot exceed episode_length

In [ ]:
# Test the trained agent
obs = env.reset()
n_steps = 9900
mx = []
my = []
monkey_t = []
monkey_speed_info = []
mheading = [] # in radians
ffxy_all = []
ffxy_visible = []
ffxy2_all = []
time_rl = []
reward_log = []
mx_rewarded = []
my_rewarded = []
captured_ff = []
current_target = []
action_reward = []
num_targets = []
env_obs = []
#captured_ff_cum_x = []
#captured_ff_cum_y = []
all_captured_ff_x = []
all_captured_ff_y = []
visible_ff_indices_all = []
memory_ff_indices_all = []
obs_ff_indices_all = []
obs_ff_overall_indices_all = []
memory_all = []
count_targets = 0
ff_angles2 = []
ff_distances2 = []


for step in range(n_steps):
  action, _ = sac_model.predict(obs, deterministic=True)
  #print("Step {}".format(step + 1))
  #print("distance, action, reward", np.concatenate([np.array([env.closest_ff_distance]), np.array([action[0]]), np.array([action[1]]), np.array([reward])]))
  previous_ffxy = env.ffxy
  prev_ff_information = env.ff_information.copy()
  obs, reward, done, info = env.step(action)
  print(step, "Action: ", action)
  reward_log.append(reward)
  #print(env.num_targets)
  #action_reward.append(np.concatenate([action, np.array([reward])]))
  #print(step, np.concatenate([np.round(action, 4), np.round(obs, 4)]), reward)
  num_targets.append(env.num_targets)
  memory_all.append(env.ff_memory_all)
  #if reward > 99:
    #for i in env.captured_ff_index:
      #captured_ff.append([env.time, previous_ffxy[i,0].item(), previous_ffxy[i,1].item()])
  if env.num_targets >0:
    count_targets += env.num_targets
    captured_ff.append(env.captured_ff_index)
    all_captured_ff_x = all_captured_ff_x + previous_ffxy[env.captured_ff_index][:,0].tolist()
    all_captured_ff_y = all_captured_ff_y + previous_ffxy[env.captured_ff_index][:,1].tolist()
  else:
    captured_ff.append(0)
  #captured_ff_cum_x.append(all_captured_ff_x)
  #captured_ff_cum_y.append(all_captured_ff_y)
  mx.append(env.agentx.item())
  my.append(env.agenty.item())
  monkey_t.append(env.time)
  monkey_speed_info.append(env.dv.item())
  mheading.append(env.agentheading.item())
  time_rl.append(env.time)
  ffxy_all.append(env.ffxy.clone())
  ffxy2_all.append(env.ffxy2.clone())
  ffxy_visible.append(env.ffxy[env.visible_ff_indices].clone())
  env_obs.append(obs)
  visible_ff_indices_all.append(env.visible_ff_indices)
  memory_ff_indices_all.append(env.ff_in_memory_indices)
  obs_ff_indices_all.append(env.topk_indices)
  real_indices = []
  for index in env.topk_indices:
    real_indices.append(int(prev_ff_information[:,0][np.where(prev_ff_information[:,7]==index.item())[0][-1]].copy()))
  obs_ff_overall_indices_all.append(real_indices)
  if len(env.topk_indices) > 0:
    ff_angles2.append(env.ff_angle_topk_2)
    ff_distances2.append(env.ff_distance_topk)
  else:
    ff_angles2.append(torch.tensor([]))
    ff_distances2.append(torch.tensor([]))
  if done:
    obs = env.reset()
  #print(step, ffxy_visible[-1])

### monkey & ff information

In [ ]:
monkey_speed = np.array(monkey_speed_info)

monkey_information = {
'monkey_x': np.array(mx),
'monkey_y': np.array(my),
'monkey_t': np.array(monkey_t),
'monkey_speed': monkey_speed,
'monkey_angle': np.array(mheading),
}


monkey_information['monkey_speeddummy'] = (monkey_speed>200*0.01*env.dt).astype(int) 
monkey_speeddummy = (monkey_speed>200*0.01*env.dt).astype(int) 
delta_time = np.delete(monkey_information['monkey_t'], 0) - np.delete(monkey_information['monkey_t'], -1)
monkey_dw = np.diff(np.array(mheading), prepend=mheading[0])/env.dt
monkey_information['monkey_dw'] = monkey_dw


#ff: 
#[index, x, y, time_start, time_captured, mx(when_captured), my(when_captured), index_in_flash]
# Note: when collecting firefly data, no more than one episode can be run. Otherwise the data might be messed up.
# A solution is to elongate an episode


# sort the time of capture for ff (if they have been captured)
ff_information = env.ff_information.copy()
ff_time_captured_all = ff_information[:,4]
captured_ff_indices = np.where(ff_time_captured_all != -9999)[0]
not_captured_ff_indices = np.where(ff_time_captured_all == -9999)[0]
num_captured_ff = len(captured_ff_indices)
sorted_indices_captured = captured_ff_indices[np.argsort(ff_time_captured_all[captured_ff_indices])]
sort_indices_all = np.concatenate([sorted_indices_captured, not_captured_ff_indices])
ff_information_sorted = ff_information[sort_indices_all]




# make ff_flash_sorted
ff_flash_sorted = []
env_end_time = env.time
for ff in ff_information[sorted_indices_captured]:
  flash = env.ff_flash[int(ff[7])].numpy()
  replace_start = False
  replace_end = False

  # first flatten flash, and then find the elements that are before ff start time
  # if the number of elements is even, then the flashing_start_time is the start time of the next interval
  before_start = np.where(flash.flatten() <= ff[3])[0]
  if len(before_start) > 0:
    if len(before_start)%2 == 0:
      start_flash_index = int(len(before_start)/2)
    else:
      start_flash_index = int((len(before_start)-1)/2)
      replace_start = True
  else:
      start_flash_index = 0


  after_finish = np.where(flash.flatten() >= ff[4])[0]
  if len(after_finish) > 0:
    num_indices_before_finish = after_finish[0]
    if num_indices_before_finish%2 == 0:
      end_flash_index = int(num_indices_before_finish/2) - 1
    else: 
      end_flash_index = int((num_indices_before_finish+1)/2) - 1
      replace_end = True
  else:
    end_flash_index = len(flash)-1


  
  ff_flash = flash[start_flash_index:(end_flash_index+1)]

  if len(ff_flash) < 2:
    ff_flash = flash[end_flash_index-1: end_flash_index+1]
    if len(ff_flash) < 2:
      ff_flash = np.array([[-1,-1]])


  if replace_start == True:
    ff_flash[0, 0] = ff[3]
  if replace_end == True:
    ff_flash[-1, 1] = ff[4]

  if len(ff_flash) == 0:
    ff_flash = np.array([[-1,-1]])

  ff_flash_sorted.append(ff_flash)



# For the ffs that have never been captured, the end_flash_time is evaluated not in relation to the 
# time of captrue, but to the time that the env ends
for ff in ff_information[not_captured_ff_indices]:
  flash = env.ff_flash[int(ff[7])].numpy()
  replace_start = False


  # first flatten flash, and then find the elements that are before ff start time
  # if the number of elements is even, then the flashing_start_time is the start time of the next interval
  before_start = np.where(flash.flatten() <= ff[3])[0]
  if len(before_start)%2 == 0:
    start_flash_index = int(len(before_start)/2)
  else:
    start_flash_index = int((len(before_start)-1)/2)
    replace_start = True


# # if we only want the part before the testing ends
#   after_finish = np.where(flash.flatten() >= env_end_time)[0] # differing from captured ffs
#   num_indices_before_finish = after_finish[0]
#   if num_indices_before_finish%2 == 0:
#     end_flash_index = int(num_indices_before_finish/2) - 1
#   else: 
#     end_flash_index = int((num_indices_before_finish+1)/2) - 1
#     replace_end = True
#   ff_flash = flash[start_flash_index:(end_flash_index+1)]


  ff_flash = flash[start_flash_index:]

  if len(ff_flash) < 2:
    ff_flash = flash[end_flash_index-1: end_flash_index+1]
    if len(ff_flash) < 2:
      ff_flash = np.array([[-1,-1]])


  if replace_start == True:
    ff_flash[0, 0] = ff[3]

  # # if we only want the part before the testing ends
  # if replace_end == True:
  #   ff_flash[-1, 1] = ff[4]
  if len(ff_flash) == 0:
    ff_flash = np.array([[-1,-1]])

  ff_flash_sorted.append(ff_flash)


ff_catched_T_sorted = ff_time_captured_all[sorted_indices_captured] # Note that these two will be shorter than the other arrays
ff_believed_position_sorted = ff_information[:,5:7][sorted_indices_captured]

ff_real_position_sorted = ff_information[:,1:3][sort_indices_all]
ff_life_sorted = ff_information[:,3:5][sort_indices_all]
ff_life_sorted[:,1][np.where(ff_life_sorted[:,1]==-9999)[0]] = env.time
ff_flash_end_sorted = [flash[-1,1] if len(flash) > 0 else env.time for flash in ff_flash_sorted]
ff_flash_end_sorted = np.array(ff_flash_end_sorted)

catched_ff_num = len(ff_catched_T_sorted) - 200
total_ff_num = len(ff_life_sorted)

## Data from agent (LSTM)

### env (LSTM)



#### e1: one firefly env, no noise


In [ ]:
# self.distance_noise_factor = 1/6, self.angle_noise_factor  = 1/8
model_path = "/content/gdrive/MyDrive/fireflies_agent/lstm/July_26" 

# best_avg_rewards = 7124

# A sample of actions:
# 95 Action:  [ 0.31841782 -0.999913  ]
# 96 Action:  [-0.07380565  0.53944504]
# 97 Action:  [-0.22972749 -0.9984572 ]
# 98 Action:  [-0.7427281   0.40428725]
# 99 Action:  [0.7400373  0.76522696]
# 100 Action:  [-0.34833822 -0.99916583]
# 101 Action:  [0.41152725 0.41533998]
# 102 Action:  [ 0.940351   -0.99330115]
# 103 Action:  [0.9275105  0.40295646]
# 104 Action:  [ 0.17494562 -0.9831464 ]
# 26.0 sys_vel:  [0.1749, 0.0084] n_targets:  1

In [ ]:
gamma= 0.995
soft_q_lr = 0.0015
policy_lr = 0.003  
alpha_lr = 0.002 
update_itr= 1
hidden_dim= 128
reward_scale= 10
target_entropy= -2
soft_tau= 0.015

In [ ]:
# obs = [angle, distance, memory, angle2, distance2, memory2, ...]
# before = [angle, angle2 ..., distance, distance2, ... memory, memory2]


reward_per_episode = 0

class MultiFF(Env):
  def __init__(self):
      super(MultiFF, self).__init__()
      self.num_ff = 200
      self.arena_radius = 1000
      self.episode_len = 15000
      self.dt = 0.25 
      #self.current_episode = 0
      #self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.obs_ff = 1
      self.observation_space = spaces.Box(low=-1., high=1., shape=(self.obs_ff*3,),dtype=np.float32)
      self.terminal_vel = 0.01
      #self.pro_noise_std = 0.005
      self.vgain = 200
      self.wgain = pi/4
      self.reward_per_ff = 100
      #self.time_cost = 0.005
      #self.total_time = 0
      #self.zero_action = False
      #self.target_update_counter_num = 20
      #self.reward_per_episode = []
      #self.update_slots = True
      #self.closest_ff_distance = 200
      self.pro_noise_std = 0.005
      self.epi_num = -1
      self.has_sped_up_before = False
      self.invisible_distance = 400
      self.invisible_angle = 2*pi/9
      self.reward_boundary = 25
      # self.dev_v_cost = 1
      self.flash_on_interval = 2.1
      # self.distance_noise_factor = 1/6
      # self.distance_noise_constant = 5
      # self.angle_noise_factor = 1/8
      # self.angle_noise_constant = 1/8
      self.num_intervals = 25000
      self.distance2center_cost = 2
      






  def reset(self):
    self.epi_num += 1
    print("\n episode: ", self.epi_num)
    self.num_targets = 0
    #self.past_speeds = []
    self.time = 0
    #self.counter = 0
    #self.current_target_index = torch.tensor([999], dtype=torch.int32)
    #self.previous_target_index = self.current_target_index
    self.ff_flash = []
    self.has_sped_up_before = False
    

    if self.flash_on_interval > 0.31:
      global reward_per_episode
      if reward_per_episode >= 1000:
        self.flash_on_interval-=0.3 
        reward_per_episode = 800
        global best_cum_rewards
        best_cum_rewards = best_cum_rewards/2
    else:
      self.distance2center_cost = 0
    print("flash_on_interval: ",   self.flash_on_interval)

    for i in range(self.num_ff):
      
      first_flash = torch.rand(1)
      intervals = torch.poisson(torch.ones(self.num_intervals-1)*3)
      t0 = torch.cat((first_flash, first_flash+torch.cumsum(intervals, dim=0)+torch.cumsum(torch.ones(self.num_intervals-1)*0.3, dim=0)))
      t1 = t0 + torch.ones(self.num_intervals)*self.flash_on_interval
      self.ff_flash.append(torch.stack((t0, t1), dim=1))

    self.ffr = torch.sqrt(torch.rand(self.num_ff))*self.arena_radius # The radius of the arena changed from 1000 cm/s to 200 cm/s
    self.fftheta = torch.rand(self.num_ff)*2*pi
    #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
    self.ffx = torch.cos(self.fftheta) * self.ffr
    self.ffy = torch.sin(self.fftheta) * self.ffr
    self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
    self.ffx2 = self.ffx.clone()
    self.ffy2 = self.ffy.clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    #self.ff_info={}
    #self.update_slots = True
    self.agentx = torch.tensor([0])
    self.agenty = torch.tensor([0])
    self.agentr = torch.zeros(1)
    self.agentxy = torch.tensor([0, 0])
    self.agentheading = torch.zeros(1).uniform_(0, 2*pi)
    self.dv = torch.zeros(1).uniform_(-0.05, 0.05) 
    self.dw = torch.zeros(1)
    self.end_episode = False 
    self.obs = self.beliefs().numpy()
    #self.chunk_50s = 1
    self.episode_reward = 0
    #self.stop_rewarding_speed = False
    #self.current_obs_steps = 0
    return self.obs






  def calculate_reward(self):
      #action_cost=((self.previous_action[1]-self.action[1])**2+(self.previous_action[0]-self.action[0])**2)*self.mag_cost
      # To incorporate action_cost, we need to incorporate previous_action into decision_info
      #self.total_time += 1
      # In addition to rewarding the monkey for capturing the firefly, we also use different phases of rewards to teach monkey specific behaviours
      #reward = -self.time_cost
      reward = 0
      # Reward shaping
      # Phase I: reward the agent for learning to stop 
      # Always: reward the agent for capturing fireflies
      self.num_targets = 0
      if abs(self.sys_vel[1]) <= self.terminal_vel:
        captured_ff_index0 = (self.ff_distance_all <= self.reward_boundary).nonzero().reshape(-1)
        total_deviated_distance = torch.sum(self.ff_distance_all[captured_ff_index0]).item()
        captured_ff_index = captured_ff_index0.tolist()
        self.captured_ff_index = captured_ff_index
        num_targets = len(captured_ff_index)
        self.num_targets = num_targets
        if num_targets > 0: # If the monkey hs captured at least 1 ff
          # Calculate reward
          reward = reward + self.reward_per_ff * num_targets - total_deviated_distance*self.distance2center_cost 
          # Replace the captured ffs with ffs of new locations
          self.ffr[captured_ff_index]= torch.sqrt(torch.rand(num_targets))*self.arena_radius
          self.fftheta[captured_ff_index]= torch.rand(num_targets)*2*pi
          #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
          self.ffx[captured_ff_index] = torch.cos(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffy[captured_ff_index] = torch.sin(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
          # Delete the information from self.ff_info
          #[self.ff_info.pop(key) for key in captured_ff_index if (key in self.ff_info)]
          #self.current_target_index = torch.tensor([999], dtype=torch.int32)
          #self.previous_target_index = self.current_target_index
          # Reward the firefly based on the average speed before capturing the firefly
          ##reward = reward + sum(self.past_speeds)/len(self.past_speeds)
          #print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "obs: ", list(np.round(self.obs, decimals = 2)), "n_targets: ",  num_targets)
          print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "n_targets: ",  num_targets)
          #self.update_slots = True
          #self.stop_rewarding_speed = True
        #elif self.has_sped_up_before == True:
        # based on Ruiyi's formula, using the distance of the closest ff in obs
        #reward += math.exp(-((self.ff_current[1, 0]**2)*(25/1.5)**2)/2)
        #self.has_sped_up_before = False
      self.prev_action = self.action.clone()

      return reward







  def step(self, action):
    # For a small possibility, the agent will just stop
    self.time += self.dt
    action=torch.tensor(action)
    self.action = action.clone()
    action[1] = action[1]/2+0.5
    self.sys_vel=action.clone()
    self.state_step(action)
    self.obs = self.beliefs().numpy()
    reward=self.calculate_reward()
    self.episode_reward += reward
    if self.time >= self.episode_len*self.dt:
      self.end_episode = True
      #self.current_episode += 1
      print("Reward for the episode: ", self.episode_reward)
    #print("action: ", torch.round(action, decimals = 3), "obs: ", np.round(self.obs, decimals = 3), "Reward: ", round(reward, 3))
    return self.obs, reward, self.end_episode, {}




  def state_step(self,action):
    #vnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std 
    #wnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std
    vnoise = 0
    wnoise = 0
    self.dw_normed = (action[0]+wnoise)
    self.dv_normed = (action[1]+vnoise)
    self.dw = (action[0]+wnoise)*self.wgain*self.dt
    self.agentheading = self.agentheading + self.dw.item()
    self.dv =  (action[1]+vnoise)*self.vgain*self.dt
    self.dx= torch.cos(self.agentheading)*self.dv
    self.dy= torch.sin(self.agentheading)*self.dv
    self.agentx = self.agentx + self.dx.item()
    self.agenty = self.agenty + self.dy.item()
    self.agentxy = torch.cat((self.agentx, self.agenty))
    self.agentr = vector_norm(self.agentxy)
    self.agenttheta = torch.tensor(torch.atan2(self.agenty, self.agentx))  
                               
    if self.agentr >= self.arena_radius:
      self.agentr = 2*self.arena_radius-self.agentr
      self.agenttheta = self.agenttheta + pi
      self.agentx = (self.agentr*torch.cos(self.agenttheta)).reshape(1,)
      self.agenty = (self.agentr*torch.sin(self.agenttheta)).reshape(1,)
      self.agentxy = torch.cat((self.agentx, self.agenty))
      self.agentheading = self.agenttheta - pi
    while self.agentheading >= 2*pi:
      self.agentheading = self.agentheading - 2*pi
    while self.agentheading < 0:
      self.agentheading = self.agentheading + 2*pi   








  def beliefs(self): 
    torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std 


    # Make a tensor containing the relative distance of all fireflies to the agent
    self.ff_distance_all = vector_norm(self.ffxy - self.agentxy, dim=1)
    # distance_noise = torch.distributions.Normal(0,self.ff_distance_all*self.distance_noise_factor+ self.distance_noise_constant).sample() 
    # self.ff_distance_all = self.ff_distance_all + distance_noise
    # Make a tensor containing the relative (real) angle of all fireflies to the agent
    ffradians = torch.atan2(self.ffy-self.agenty, self.ffx-self.agentx)
    angle0 = ffradians - self.agentheading
    angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
    angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi

    # angle_noise = torch.distributions.Normal(0,torch.abs(angle0)*self.angle_noise_factor+ self.angle_noise_constant).sample() 
    # angle0 = angle0+angle_noise
    # Adjust the angle based on reward boundary
    angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip( self.ff_distance_all, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
    angle2 = torch.clip(angle1,0,pi)
    ff_angle_all = torch.sign(angle0)* angle2
    # Update the tensor containing the uncertainties of all fireflies to the agent
    visible_ff = torch.logical_and(self.ff_distance_all < self.invisible_distance, torch.abs(angle0) < self.invisible_angle)
    self.visible_ff_indices0 = visible_ff.nonzero().reshape(-1)
    for index in self.visible_ff_indices0:
      ff = self.ff_flash[index]
      if not torch.any(torch.logical_and(ff[:, 0] <= self.time, ff[:, 1] >= self.time)):
        visible_ff[index] = False
    self.visible_ff_indices = visible_ff.nonzero().reshape(-1)

    # Consider the case where there are fewer than self.obs_ff fireflies that are in memory
    if torch.numel(self.visible_ff_indices) >= self.obs_ff:
      # Rank the ff whose "memory" is creater than 0 based on distance
      sorted_indices =  torch.topk(-self.ff_distance_all[self.visible_ff_indices], self.obs_ff).indices
      self.topk_indices = self.visible_ff_indices[sorted_indices]
      self.ffxy_topk = self.ffxy[self.topk_indices]
      #self.ff_distance_topk = vector_norm(self.ffxy_topk - self.agentxy, dim=1)
      self.ff_distance_topk = self.ff_distance_all[self.topk_indices]
      self.ff_angle_topk_2 = angle0[self.topk_indices]
      ff_angle_topk_3 = ff_angle_all[self.topk_indices]

      # # Calculate relative angles 
      # ffradians = torch.atan2(self.ffxy_topk[:,1]-self.agenty, self.ffxy_topk[:,0]-self.agentx)
      # angle0 = ffradians - self.agentheading
      # angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      # angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      # self.ff_angle_topk_2 = angle0.clone()
     
      # # Calculate relative angles of all ffs based on reward boundaries
      # # Adjust the angle based on reward boundary
      # angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      # angle2 = torch.clip(angle1,0,pi)
      # ff_angle_topk_3 = torch.sign(angle0)* angle2
      
      
      # Concatenate distance, angle, and memory
      ff_array = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk), dim=0)
    

    elif torch.numel(self.visible_ff_indices) == 0:
      ff_array = torch.tensor([[0], [0], [self.invisible_distance]]).repeat([1, self.obs_ff])
      self.topk_indices = torch.tensor([])

    else:
      sorted_distance, sorted_indices = torch.sort(-self.ff_distance_all[self.visible_ff_indices])
      self.topk_indices = self.visible_ff_indices[sorted_indices]
      self.ffxy_topk = self.ffxy[self.topk_indices ]
      
      self.ff_distance_topk = self.ff_distance_all[self.topk_indices]
      self.ff_angle_topk_2 = angle0[self.topk_indices]
      ff_angle_topk_3 = ff_angle_all[self.topk_indices]
      # self.ff_distance_topk = vector_norm(self.ffxy_topk - self.agentxy, dim=1)
      # # Calculate relative angles 
      # ffradians = torch.atan2(self.ffxy_topk[:,1]-self.agenty, self.ffxy_topk[:,0]-self.agentx)
      # angle0 = ffradians - self.agentheading
      # angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      # angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      # self.ff_angle_topk_2 = angle0.clone()
      # # Calculate relative angles of all ffs based on reward boundaries
      # # Adjust the angle based on reward boundary
      # angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      # angle2 = torch.clip(angle1,0,pi)
      # ff_angle_topk_3 = torch.sign(angle0)* angle2
      
      # Concatenate distance, angle, and memory
      ff_array0 = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk), dim=0)
      needed_ff = self.obs_ff - torch.numel(self.visible_ff_indices)
      ff_array = torch.stack([ff_array0.reshape([3,-1]), torch.tensor([[0], [0], [self.invisible_distance]]).repeat([1, needed_ff])], dim=1)
    self.real_indices = []
    for index in env.topk_indices:
      self.real_indices.append(int(env.ff_information[:,0][np.where(env.ff_information[:,-1]==index.item())[0][-1]].copy()))
    # ff_array[0:2,:] = ff_array[0:2,:]/pi
    # ff_array[2,:] = (ff_array[2,:]/self.invisible_distance-0.5)*2
    ff_array[2,:] = ff_array[2,:]/self.invisible_distance
    self.ff_array = ff_array.clone()
    return torch.flatten(ff_array.transpose(0, 1))


env = MultiFF() 
env.reset() 

sac_model = SAC("MlpPolicy", 
            env,
            buffer_size=int(1e6),
            batch_size=1024,
            device='auto',
            verbose=False,
            train_freq=100,
            learning_starts = int(10),
            target_update_interval=20,
            learning_rate=1e-2,
            gamma=0.9999,
            policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[64, 64])
                )


#### e3: 2nd slot is always placeholder


In [ ]:
# self.distance_noise_factor = 1/6, self.angle_noise_factor  = 1/8
model_path = "/content/gdrive/MyDrive/fireflies_agent/lstm/July_29" 

# test rewards = 11080



In [ ]:
model_path = "/content/gdrive/MyDrive/fireflies_agent/lstm/July_26_2" 

## Trained from d22


## Reward: 12480 after training for 3000 + episodes

## Reward: 12100 for e3 env. Lol....

In [ ]:
gamma= 0.995
soft_q_lr = 0.0015
policy_lr = 0.003  
alpha_lr = 0.002 
update_itr= 1
hidden_dim= 128
reward_scale= 10
target_entropy= -2
soft_tau= 0.015

In [ ]:
# obs = [angle, distance, memory, angle2, distance2, memory2, ...]
# before = [angle, angle2 ..., distance, distance2, ... memory, memory2]


reward_per_episode = 0

class MultiFF(Env):
  def __init__(self):
      super(MultiFF, self).__init__()
      self.num_ff = 200
      self.arena_radius = 1000
      self.episode_len = 10000 
      self.dt = 0.25 
      #self.current_episode = 0
      #self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.obs_ff = 2
      self.observation_space = spaces.Box(low=-1., high=1., shape=(self.obs_ff*3,),dtype=np.float32)
      self.terminal_vel = 0.01
      #self.pro_noise_std = 0.005
      self.vgain = 200
      self.wgain = pi/4
      self.reward_per_ff = 100
      #self.time_cost = 0.005
      #self.total_time = 0
      #self.zero_action = False
      #self.target_update_counter_num = 20
      #self.reward_per_episode = []
      #self.update_slots = True
      #self.closest_ff_distance = 200
      self.pro_noise_std = 0.005
      self.epi_num = -1
      self.has_sped_up_before = False
      self.invisible_distance = 400
      self.invisible_angle = 2*pi/9
      self.reward_boundary = 25
      # self.dev_v_cost = 1
      self.flash_on_interval = 2.1
      # self.distance_noise_factor = 1/6
      # self.distance_noise_constant = 5
      # self.angle_noise_factor = 1/8
      # self.angle_noise_constant = 1/8
      self.num_intervals = 25000
      self.distance2center_cost = 2
      






  def reset(self):
    self.epi_num += 1
    print("\n episode: ", self.epi_num)
    self.num_targets = 0
    #self.past_speeds = []
    self.time = 0
    #self.counter = 0
    #self.current_target_index = torch.tensor([999], dtype=torch.int32)
    #self.previous_target_index = self.current_target_index
    self.ff_flash = []
    self.has_sped_up_before = False
    

    if self.flash_on_interval > 0.31:
      global reward_per_episode
      if reward_per_episode >= 1000:
        self.flash_on_interval-=0.3 
        reward_per_episode = 800
        global best_cum_rewards
        best_cum_rewards = best_cum_rewards/2
    else:
      self.distance2center_cost = 0
    print("flash_on_interval: ",   self.flash_on_interval)

    for i in range(self.num_ff):
      
      first_flash = torch.rand(1)
      intervals = torch.poisson(torch.ones(self.num_intervals-1)*3)
      t0 = torch.cat((first_flash, first_flash+torch.cumsum(intervals, dim=0)+torch.cumsum(torch.ones(self.num_intervals-1)*0.3, dim=0)))
      t1 = t0 + torch.ones(self.num_intervals)*self.flash_on_interval
      self.ff_flash.append(torch.stack((t0, t1), dim=1))

    self.ffr = torch.sqrt(torch.rand(self.num_ff))*self.arena_radius # The radius of the arena changed from 1000 cm/s to 200 cm/s
    self.fftheta = torch.rand(self.num_ff)*2*pi
    #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
    self.ffx = torch.cos(self.fftheta) * self.ffr
    self.ffy = torch.sin(self.fftheta) * self.ffr
    self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
    self.ffx2 = self.ffx.clone()
    self.ffy2 = self.ffy.clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    #self.ff_info={}
    #self.update_slots = True
    self.agentx = torch.tensor([0])
    self.agenty = torch.tensor([0])
    self.agentr = torch.zeros(1)
    self.agentxy = torch.tensor([0, 0])
    self.agentheading = torch.zeros(1).uniform_(0, 2*pi)
    self.dv = torch.zeros(1).uniform_(-0.05, 0.05) 
    self.dw = torch.zeros(1)
    self.end_episode = False 
    self.obs = self.beliefs().numpy()
    #self.chunk_50s = 1
    self.episode_reward = 0
    #self.stop_rewarding_speed = False
    #self.current_obs_steps = 0
    return self.obs






  def calculate_reward(self):
      #action_cost=((self.previous_action[1]-self.action[1])**2+(self.previous_action[0]-self.action[0])**2)*self.mag_cost
      # To incorporate action_cost, we need to incorporate previous_action into decision_info
      #self.total_time += 1
      # In addition to rewarding the monkey for capturing the firefly, we also use different phases of rewards to teach monkey specific behaviours
      #reward = -self.time_cost
      reward = 0
      # Reward shaping
      # Phase I: reward the agent for learning to stop 
      # Always: reward the agent for capturing fireflies
      self.num_targets = 0
      if abs(self.sys_vel[1]) <= self.terminal_vel:
        captured_ff_index0 = (self.ff_distance_all <= self.reward_boundary).nonzero().reshape(-1)
        total_deviated_distance = torch.sum(self.ff_distance_all[captured_ff_index0]).item()
        captured_ff_index = captured_ff_index0.tolist()
        self.captured_ff_index = captured_ff_index
        num_targets = len(captured_ff_index)
        self.num_targets = num_targets
        if num_targets > 0: # If the monkey hs captured at least 1 ff
          # Calculate reward
          reward = reward + self.reward_per_ff * num_targets - total_deviated_distance*self.distance2center_cost 
          # Replace the captured ffs with ffs of new locations
          self.ffr[captured_ff_index]= torch.sqrt(torch.rand(num_targets))*self.arena_radius
          self.fftheta[captured_ff_index]= torch.rand(num_targets)*2*pi
          #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
          self.ffx[captured_ff_index] = torch.cos(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffy[captured_ff_index] = torch.sin(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
          # Delete the information from self.ff_info
          #[self.ff_info.pop(key) for key in captured_ff_index if (key in self.ff_info)]
          #self.current_target_index = torch.tensor([999], dtype=torch.int32)
          #self.previous_target_index = self.current_target_index
          # Reward the firefly based on the average speed before capturing the firefly
          ##reward = reward + sum(self.past_speeds)/len(self.past_speeds)
          #print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "obs: ", list(np.round(self.obs, decimals = 2)), "n_targets: ",  num_targets)
          print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "n_targets: ",  num_targets)
          #self.update_slots = True
          #self.stop_rewarding_speed = True
        #elif self.has_sped_up_before == True:
        # based on Ruiyi's formula, using the distance of the closest ff in obs
        #reward += math.exp(-((self.ff_current[1, 0]**2)*(25/1.5)**2)/2)
        #self.has_sped_up_before = False
      self.prev_action = self.action.clone()

      return reward







  def step(self, action):
    self.time += self.dt
    action=torch.tensor(action)
    self.action = action.clone()
    action[1] = action[1]/2+0.5
    self.sys_vel=action.clone()
    self.state_step(action)
    self.obs = self.beliefs().numpy()
    reward=self.calculate_reward()
    self.episode_reward += reward
    
    if self.time >= self.episode_len*self.dt:
      self.end_episode = True
      #self.current_episode += 1
      print("Reward for the episode: ", self.episode_reward)
    #print("action: ", torch.round(action, decimals = 3), "obs: ", np.round(self.obs, decimals = 3), "Reward: ", round(reward, 3))
    return self.obs, reward, self.end_episode, {}




  def state_step(self,action):
    #vnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std 
    #wnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std
    vnoise = 0
    wnoise = 0
    self.dw_normed = (action[0]+wnoise)
    self.dv_normed = (action[1]+vnoise)
    self.dw = (action[0]+wnoise)*self.wgain*self.dt
    self.agentheading = self.agentheading + self.dw.item()
    self.dv =  (action[1]+vnoise)*self.vgain*self.dt
    self.dx= torch.cos(self.agentheading)*self.dv
    self.dy= torch.sin(self.agentheading)*self.dv
    self.agentx = self.agentx + self.dx.item()
    self.agenty = self.agenty + self.dy.item()
    self.agentxy = torch.cat((self.agentx, self.agenty))
    self.agentr = vector_norm(self.agentxy)
    self.agenttheta = torch.tensor(torch.atan2(self.agenty, self.agentx))  
                               
    if self.agentr >= self.arena_radius:
      self.agentr = 2*self.arena_radius-self.agentr
      self.agenttheta = self.agenttheta + pi
      self.agentx = (self.agentr*torch.cos(self.agenttheta)).reshape(1,)
      self.agenty = (self.agentr*torch.sin(self.agenttheta)).reshape(1,)
      self.agentxy = torch.cat((self.agentx, self.agenty))
      self.agentheading = self.agenttheta - pi
    while self.agentheading >= 2*pi:
      self.agentheading = self.agentheading - 2*pi
    while self.agentheading < 0:
      self.agentheading = self.agentheading + 2*pi   








  def beliefs(self): 


    # Make a tensor containing the relative distance of all fireflies to the agent
    self.ff_distance_all = vector_norm(self.ffxy - self.agentxy, dim=1)
    #distance_noise = torch.distributions.Normal(0,self.ff_distance_all*self.distance_noise_factor+ self.distance_noise_constant).sample() 
    #self.ff_distance_all = self.ff_distance_all + distance_noise
    # Make a tensor containing the relative (real) angle of all fireflies to the agent
    ffradians = torch.atan2(self.ffy-self.agenty, self.ffx-self.agentx)
    angle0 = ffradians - self.agentheading
    angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
    angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi

    #angle_noise = torch.distributions.Normal(0,torch.abs(angle0)*self.angle_noise_factor+ self.angle_noise_constant).sample() 
    #angle0 = angle0+angle_noise
    # Adjust the angle based on reward boundary
    angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip( self.ff_distance_all, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
    angle2 = torch.clip(angle1,0,pi)
    ff_angle_all = torch.sign(angle0)* angle2
    # Update the tensor containing the uncertainties of all fireflies to the agent
    visible_ff = torch.logical_and( self.ff_distance_all < self.invisible_distance, torch.abs(angle0) < self.invisible_angle)
    self.visible_ff_indices0 = visible_ff.nonzero().reshape(-1)
    for index in self.visible_ff_indices0:
      ff = self.ff_flash[index]
      if not torch.any(torch.logical_and(ff[:, 0] <= self.time, ff[:, 1] >= self.time)):
        visible_ff[index] = False
    self.visible_ff_indices = visible_ff.nonzero().reshape(-1)

    # Consider the case where there are fewer than self.obs_ff fireflies that are in memory
    if torch.numel(self.visible_ff_indices) >= self.obs_ff-1:
      # Rank the ff whose "memory" is creater than 0 based on distance
      sorted_indices =  torch.topk(-self.ff_distance_all[self.visible_ff_indices], self.obs_ff-1).indices
      self.topk_indices = self.visible_ff_indices[sorted_indices]
      self.ffxy_topk = self.ffxy[self.topk_indices]
      #self.ff_distance_topk = vector_norm(self.ffxy_topk - self.agentxy, dim=1)
      self.ff_distance_topk = self.ff_distance_all[self.topk_indices]
      self.ff_angle_topk_2 = angle0[self.topk_indices]
      ff_angle_topk_3 = ff_angle_all[self.topk_indices]

      # # Calculate relative angles 
      # ffradians = torch.atan2(self.ffxy_topk[:,1]-self.agenty, self.ffxy_topk[:,0]-self.agentx)
      # angle0 = ffradians - self.agentheading
      # angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      # angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      # self.ff_angle_topk_2 = angle0.clone()
     
      # # Calculate relative angles of all ffs based on reward boundaries
      # # Adjust the angle based on reward boundary
      # angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      # angle2 = torch.clip(angle1,0,pi)
      # ff_angle_topk_3 = torch.sign(angle0)* angle2
      # Concatenate distance, angle, and memory
      ff_array0 = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk), dim=0)
      needed_ff = 1
      ff_array = torch.stack([ff_array0.reshape([3,-1]), torch.tensor([[0], [0], [self.invisible_distance]]).repeat([1, needed_ff])], dim=1)
      
    else:
      ff_array = torch.tensor([[0], [0], [self.invisible_distance]]).repeat([1, self.obs_ff])
      self.topk_indices = torch.tensor([])



    # ff_array[0:2,:] = ff_array[0:2,:]/pi
    # ff_array[2,:] = (ff_array[2,:]/self.invisible_distance-0.5)*2
    ff_array[2,:] = ff_array[2,:]/self.invisible_distance
    self.ff_array = ff_array.clone()
    return torch.flatten(ff_array.transpose(0, 1))


env = MultiFF() 
env.reset() 

sac_model = SAC("MlpPolicy", 
            env,
            buffer_size=int(1e6),
            batch_size=1024,
            device='auto',
            verbose=False,
            train_freq=100,
            learning_starts = int(10),
            target_update_interval=20,
            learning_rate=1e-2,
            gamma=0.9999,
            policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[64, 64])
                )


#### d22: no noise; fixes wgain 

(The previous environments are fixdd too)



In [ ]:
# self.distance_noise_factor = 1/6, self.angle_noise_factor  = 1/8
model_path = "/content/gdrive/MyDrive/fireflies_agent/lstm/July_26_2" 


## Reward: 12480 after training for 3000 + episodes



# for agent
NEW_DATASET = True
MONKEY_DATA = False
NO_PLOT_NEEDED = True
data_folder_name = "LSTM_July_29"
data_num = 721
trial_total_num = 30

In [ ]:
gamma= 0.995
soft_q_lr = 0.0015
policy_lr = 0.003  
alpha_lr = 0.002 
update_itr= 1
hidden_dim= 128
reward_scale= 10
target_entropy= -2
soft_tau= 0.015

In [ ]:
# obs = [angle, distance, memory, angle2, distance2, memory2, ...]
# before = [angle, angle2 ..., distance, distance2, ... memory, memory2]


reward_per_episode = 0

class MultiFF(Env):
  def __init__(self):
      super(MultiFF, self).__init__()
      self.num_ff = 200
      self.arena_radius = 1000
      self.episode_len = 15000
      self.dt = 0.25 
      #self.current_episode = 0
      #self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.obs_ff = 2
      self.observation_space = spaces.Box(low=-1., high=1., shape=(self.obs_ff*3,),dtype=np.float32)
      self.terminal_vel = 0.01
      #self.pro_noise_std = 0.005
      self.vgain = 200
      self.wgain = pi/4
      self.reward_per_ff = 100
      #self.time_cost = 0.005
      #self.total_time = 0
      #self.zero_action = False
      #self.target_update_counter_num = 20
      #self.reward_per_episode = []
      #self.update_slots = True
      #self.closest_ff_distance = 200
      self.pro_noise_std = 0.005
      self.epi_num = -1
      self.has_sped_up_before = False
      self.invisible_distance = 400
      self.invisible_angle = 2*pi/9
      self.reward_boundary = 25
      # self.dev_v_cost = 1
      self.flash_on_interval = 2.1
      # self.distance_noise_factor = 1/6
      # self.distance_noise_constant = 5
      # self.angle_noise_factor = 1/8
      # self.angle_noise_constant = 1/8
      self.distance2center_cost = 2
      






  def reset(self):
    self.epi_num += 1
    print("\n episode: ", self.epi_num)
    self.num_targets = 0
    #self.past_speeds = []
    self.time = 0
    #self.counter = 0
    #self.current_target_index = torch.tensor([999], dtype=torch.int32)
    #self.previous_target_index = self.current_target_index
    self.ff_flash = []
    self.has_sped_up_before = False
    

    if self.flash_on_interval > 0.31:
      global reward_per_episode
      if reward_per_episode >= 1000:
        self.flash_on_interval-=0.3 
        reward_per_episode = 800
        global best_cum_rewards
        best_cum_rewards = best_cum_rewards/2
    else:
      self.distance2center_cost = 0
    print("flash_on_interval: ",   self.flash_on_interval)

    self.num_intervals = 25000
    for i in range(self.num_ff):   
      first_flash = torch.rand(1)
      intervals = torch.poisson(torch.ones(self.num_intervals-1)*3)
      t0 = torch.cat((first_flash, first_flash+torch.cumsum(intervals, dim=0)+torch.cumsum(torch.ones(self.num_intervals-1)*0.3, dim=0)))
      t1 = t0 + torch.ones(self.num_intervals)*self.flash_on_interval
      self.ff_flash.append(torch.stack((t0, t1), dim=1))

    self.ffr = torch.sqrt(torch.rand(self.num_ff))*self.arena_radius # The radius of the arena changed from 1000 cm/s to 200 cm/s
    self.fftheta = torch.rand(self.num_ff)*2*pi
    #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
    self.ffx = torch.cos(self.fftheta) * self.ffr
    self.ffy = torch.sin(self.fftheta) * self.ffr
    self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
    self.ffx2 = self.ffx.clone()
    self.ffy2 = self.ffy.clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    #self.ff_info={}
    #self.update_slots = True
    self.agentx = torch.tensor([0])
    self.agenty = torch.tensor([0])
    self.agentr = torch.zeros(1)
    self.agentxy = torch.tensor([0, 0])
    self.agentheading = torch.zeros(1).uniform_(0, 2*pi)
    self.dv = torch.zeros(1).uniform_(-0.05, 0.05) 
    self.dw = torch.zeros(1)
    self.end_episode = False 
    self.obs = self.beliefs().numpy()
    #self.chunk_50s = 1
    self.episode_reward = 0
    #self.stop_rewarding_speed = False
    #self.current_obs_steps = 0
    return self.obs






  def calculate_reward(self):
      #action_cost=((self.previous_action[1]-self.action[1])**2+(self.previous_action[0]-self.action[0])**2)*self.mag_cost
      # To incorporate action_cost, we need to incorporate previous_action into decision_info
      #self.total_time += 1
      # In addition to rewarding the monkey for capturing the firefly, we also use different phases of rewards to teach monkey specific behaviours
      #reward = -self.time_cost
      reward = 0
      # Reward shaping
      # Phase I: reward the agent for learning to stop 
      # Always: reward the agent for capturing fireflies
      self.num_targets = 0
      if abs(self.sys_vel[1]) <= self.terminal_vel:
        captured_ff_index0 = (self.ff_distance_all <= self.reward_boundary).nonzero().reshape(-1)
        total_deviated_distance = torch.sum(self.ff_distance_all[captured_ff_index0]).item()
        captured_ff_index = captured_ff_index0.tolist()
        self.captured_ff_index = captured_ff_index
        num_targets = len(captured_ff_index)
        self.num_targets = num_targets
        if num_targets > 0: # If the monkey hs captured at least 1 ff
          # Calculate reward
          reward = reward + self.reward_per_ff * num_targets - total_deviated_distance*self.distance2center_cost 
          # Replace the captured ffs with ffs of new locations
          self.ffr[captured_ff_index]= torch.sqrt(torch.rand(num_targets))*self.arena_radius
          self.fftheta[captured_ff_index]= torch.rand(num_targets)*2*pi
          #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
          self.ffx[captured_ff_index] = torch.cos(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffy[captured_ff_index] = torch.sin(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
          # Delete the information from self.ff_info
          #[self.ff_info.pop(key) for key in captured_ff_index if (key in self.ff_info)]
          #self.current_target_index = torch.tensor([999], dtype=torch.int32)
          #self.previous_target_index = self.current_target_index
          # Reward the firefly based on the average speed before capturing the firefly
          ##reward = reward + sum(self.past_speeds)/len(self.past_speeds)
          #print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "obs: ", list(np.round(self.obs, decimals = 2)), "n_targets: ",  num_targets)
          print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "n_targets: ",  num_targets)
          #self.update_slots = True
          #self.stop_rewarding_speed = True
        #elif self.has_sped_up_before == True:
        # based on Ruiyi's formula, using the distance of the closest ff in obs
        #reward += math.exp(-((self.ff_current[1, 0]**2)*(25/1.5)**2)/2)
        #self.has_sped_up_before = False
      self.prev_action = self.action.clone()

      return reward







  def step(self, action):
    self.time += self.dt
    action=torch.tensor(action)
    self.action = action.clone()
    action[1] = action[1]/2+0.5
    self.sys_vel=action.clone()
    self.state_step(action)
    self.obs = self.beliefs().numpy()
    reward=self.calculate_reward()
    self.episode_reward += reward
    if self.time >= self.episode_len*self.dt:
      self.end_episode = True
      #self.current_episode += 1
      print("Reward for the episode: ", self.episode_reward)
    #print("action: ", torch.round(action, decimals = 3), "obs: ", np.round(self.obs, decimals = 3), "Reward: ", round(reward, 3))
    return self.obs, reward, self.end_episode, {}




  def state_step(self,action):
    #vnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std 
    #wnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std
    vnoise = 0
    wnoise = 0
    self.dw_normed = (action[0]+wnoise)
    self.dv_normed = (action[1]+vnoise)
    self.dw = (action[0]+wnoise)*self.wgain*self.dt
    self.agentheading = self.agentheading + self.dw.item()
    self.dv =  (action[1]+vnoise)*self.vgain*self.dt
    self.dx= torch.cos(self.agentheading)*self.dv
    self.dy= torch.sin(self.agentheading)*self.dv
    self.agentx = self.agentx + self.dx.item()
    self.agenty = self.agenty + self.dy.item()
    self.agentxy = torch.cat((self.agentx, self.agenty))
    self.agentr = vector_norm(self.agentxy)
    self.agenttheta = torch.tensor(torch.atan2(self.agenty, self.agentx))  
                               
    if self.agentr >= self.arena_radius:
      self.agentr = 2*self.arena_radius-self.agentr
      self.agenttheta = self.agenttheta + pi
      self.agentx = (self.agentr*torch.cos(self.agenttheta)).reshape(1,)
      self.agenty = (self.agentr*torch.sin(self.agenttheta)).reshape(1,)
      self.agentxy = torch.cat((self.agentx, self.agenty))
      self.agentheading = self.agenttheta - pi
    while self.agentheading >= 2*pi:
      self.agentheading = self.agentheading - 2*pi
    while self.agentheading < 0:
      self.agentheading = self.agentheading + 2*pi   








  def beliefs(self): 


    # Make a tensor containing the relative distance of all fireflies to the agent
    self.ff_distance_all = vector_norm(self.ffxy - self.agentxy, dim=1)
    #distance_noise = torch.distributions.Normal(0,self.ff_distance_all*self.distance_noise_factor+ self.distance_noise_constant).sample() 
    #self.ff_distance_all = self.ff_distance_all + distance_noise
    # Make a tensor containing the relative (real) angle of all fireflies to the agent
    ffradians = torch.atan2(self.ffy-self.agenty, self.ffx-self.agentx)
    angle0 = ffradians - self.agentheading
    angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
    angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi

    #angle_noise = torch.distributions.Normal(0,torch.abs(angle0)*self.angle_noise_factor+ self.angle_noise_constant).sample() 
    #angle0 = angle0+angle_noise
    # Adjust the angle based on reward boundary
    angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip( self.ff_distance_all, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
    angle2 = torch.clip(angle1,0,pi)
    ff_angle_all = torch.sign(angle0)* angle2
    # Update the tensor containing the uncertainties of all fireflies to the agent
    visible_ff = torch.logical_and( self.ff_distance_all < self.invisible_distance, torch.abs(angle0) < self.invisible_angle)
    self.visible_ff_indices0 = visible_ff.nonzero().reshape(-1)
    for index in self.visible_ff_indices0:
      ff = self.ff_flash[index]
      if not torch.any(torch.logical_and(ff[:, 0] <= self.time, ff[:, 1] >= self.time)):
        visible_ff[index] = False
    self.visible_ff_indices = visible_ff.nonzero().reshape(-1)

    # Consider the case where there are fewer than self.obs_ff fireflies that are in memory
    if torch.numel(self.visible_ff_indices) >= self.obs_ff:
      # Rank the ff whose "memory" is creater than 0 based on distance
      sorted_indices =  torch.topk(-self.ff_distance_all[self.visible_ff_indices], self.obs_ff).indices
      self.topk_indices = self.visible_ff_indices[sorted_indices]
      self.ffxy_topk = self.ffxy[self.topk_indices]
      #self.ff_distance_topk = vector_norm(self.ffxy_topk - self.agentxy, dim=1)
      self.ff_distance_topk = self.ff_distance_all[self.topk_indices]
      self.ff_angle_topk_2 = angle0[self.topk_indices]
      ff_angle_topk_3 = ff_angle_all[self.topk_indices]

      # # Calculate relative angles 
      # ffradians = torch.atan2(self.ffxy_topk[:,1]-self.agenty, self.ffxy_topk[:,0]-self.agentx)
      # angle0 = ffradians - self.agentheading
      # angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      # angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      # self.ff_angle_topk_2 = angle0.clone()
     
      # # Calculate relative angles of all ffs based on reward boundaries
      # # Adjust the angle based on reward boundary
      # angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      # angle2 = torch.clip(angle1,0,pi)
      # ff_angle_topk_3 = torch.sign(angle0)* angle2
      
      
      # Concatenate distance, angle, and memory
      ff_array = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk), dim=0)
    

    elif torch.numel(self.visible_ff_indices) == 0:
      ff_array = torch.tensor([[0], [0], [self.invisible_distance]]).repeat([1, self.obs_ff])
      self.topk_indices = torch.tensor([])

    else:
      sorted_distance, sorted_indices = torch.sort(-self.ff_distance_all[self.visible_ff_indices])
      self.topk_indices = self.visible_ff_indices[sorted_indices]
      self.ffxy_topk = self.ffxy[self.topk_indices ]
      
      self.ff_distance_topk = self.ff_distance_all[self.topk_indices]
      self.ff_angle_topk_2 = angle0[self.topk_indices]
      ff_angle_topk_3 = ff_angle_all[self.topk_indices]
      # self.ff_distance_topk = vector_norm(self.ffxy_topk - self.agentxy, dim=1)
      # # Calculate relative angles 
      # ffradians = torch.atan2(self.ffxy_topk[:,1]-self.agenty, self.ffxy_topk[:,0]-self.agentx)
      # angle0 = ffradians - self.agentheading
      # angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      # angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      # self.ff_angle_topk_2 = angle0.clone()
      # # Calculate relative angles of all ffs based on reward boundaries
      # # Adjust the angle based on reward boundary
      # angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      # angle2 = torch.clip(angle1,0,pi)
      # ff_angle_topk_3 = torch.sign(angle0)* angle2
      
      # Concatenate distance, angle, and memory
      ff_array0 = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk), dim=0)
      needed_ff = self.obs_ff - torch.numel(self.visible_ff_indices)
      ff_array = torch.stack([ff_array0.reshape([3,-1]), torch.tensor([[0], [0], [self.invisible_distance]]).repeat([1, needed_ff])], dim=1)
      

    # ff_array[0:2,:] = ff_array[0:2,:]/pi
    # ff_array[2,:] = (ff_array[2,:]/self.invisible_distance-0.5)*2
    ff_array[2,:] = ff_array[2,:]/self.invisible_distance
    self.ff_array = ff_array.clone()
    return torch.flatten(ff_array.transpose(0, 1))


env = MultiFF() 
env.reset() 

sac_model = SAC("MlpPolicy", 
            env,
            buffer_size=int(1e6),
            batch_size=1024,
            device='auto',
            verbose=False,
            train_freq=100,
            learning_starts = int(10),
            target_update_interval=20,
            learning_rate=1e-2,
            gamma=0.9999,
            policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[64, 64])
                )


### load LSTM model

#### prep codes

The cell below contains all the codes combined. This version is:decrease hidden layer; change to QNetworkLSTM2

Note that the codes are slightly different from those in the notebook for training the agent because this notebook does not use GPU

In [ ]:
import torch.nn.functional as F


def linear_weights_init(m):
    if isinstance(m, nn.Linear):
        stdv = 1. / math.sqrt(m.weight.size(1))
        m.weight.data.uniform_(-stdv, stdv)
        if m.bias is not None:
            m.bias.data.uniform_(-stdv, stdv)


def conv_weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)


!pip install pygame
import pygame
from gym.spaces.box import Box



class ReplayBufferLSTM:
    """ 
    Replay buffer for agent with LSTM network additionally using previous action, can be used 
    if the hidden states are not stored (arbitrary initialization of lstm for training).
    And each sample contains the whole episode instead of a single step.
    """
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0

    def push(self, state, action, last_action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, last_action, reward, next_state, done)
        self.position = int((self.position + 1) % self.capacity)  # as a ring buffer

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, last_action, reward, next_state, done = map(np.stack,
                                                      zip(*batch))  # stack for each element
        ''' 
        the * serves as unpack: sum(a,b) <=> batch=(a,b), sum(*batch) ;
        zip: a=[1,2], b=[2,3], zip(a,b) => [(1, 2), (2, 3)] ;
        the map serves as mapping the function on each list element: map(square, [2,3]) => [4,9] ;
        np.stack((1,2)) => array([1, 2])
        '''
        return state, action, last_action, reward, next_state, done

    def __len__(
            self):  # cannot work in multiprocessing case, len(replay_buffer) is not available in proxy of manager!
        return len(self.buffer)

    def get_length(self):
        return len(self.buffer)

class ReplayBufferLSTM2:
    """ 
    Replay buffer for agent with LSTM network additionally storing previous action, 
    initial input hidden state and output hidden state of LSTM.
    And each sample contains the whole episode instead of a single step.
    'hidden_in' and 'hidden_out' are only the initial hidden state for each episode, for LSTM initialization.
    """
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0

    def push(self, hidden_in, hidden_out, state, action, last_action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (hidden_in, hidden_out, state, action, last_action, reward, next_state, done)
        self.position = int((self.position + 1) % self.capacity)  # as a ring buffer

    def sample(self, batch_size):
        s_lst, a_lst, la_lst, r_lst, ns_lst, hi_lst, ci_lst, ho_lst, co_lst, d_lst=[],[],[],[],[],[],[],[],[],[]
        batch = random.sample(self.buffer, batch_size)
        for sample in batch:
            (h_in, c_in), (h_out, c_out), state, action, last_action, reward, next_state, done = sample
            s_lst.append(state) 
            a_lst.append(action)
            la_lst.append(last_action)
            r_lst.append(reward)
            ns_lst.append(next_state)
            d_lst.append(done)
            hi_lst.append(h_in)  # h_in: (1, batch_size=1, hidden_size)
            ci_lst.append(c_in)
            ho_lst.append(h_out)
            co_lst.append(c_out)
        hi_lst = torch.cat(hi_lst, dim=-2).detach() # cat along the batch dim
        ho_lst = torch.cat(ho_lst, dim=-2).detach()
        ci_lst = torch.cat(ci_lst, dim=-2).detach()
        co_lst = torch.cat(co_lst, dim=-2).detach()

        hidden_in = (hi_lst, ci_lst)
        hidden_out = (ho_lst, co_lst)

        return hidden_in, hidden_out, s_lst, a_lst, la_lst, r_lst, ns_lst, d_lst

    def __len__(
            self):  # cannot work in multiprocessing case, len(replay_buffer) is not available in proxy of manager!
        return len(self.buffer)

    def get_length(self):
        return len(self.buffer)




import copy


class ValueNetworkBase(nn.Module):
    """ Base network class for value function approximation """
    def __init__(self, state_space, activation):
        super(ValueNetworkBase, self).__init__()
        self._state_space = state_space
        self._state_shape = state_space.shape
        if len(self._state_shape) == 1:
            self._state_dim = self._state_shape[0]
        else:  # high-dim state
            pass  

        self.activation = activation

    def forward(self):
        pass

class QNetworkBase(ValueNetworkBase):
    def __init__(self, state_space, action_space, activation ):
        super().__init__( state_space, activation)
        self._action_space = action_space
        self._action_shape = action_space.shape
        self._action_dim = self._action_shape[0]


class ValueNetwork(ValueNetworkBase):
    def __init__(self, state_space, hidden_dim, activation=F.relu, output_activation=None):
        super().__init__(state_space, activation)

        self.linear1 = nn.Linear(self._state_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, hidden_dim)
        self.linear4 = nn.Linear(hidden_dim, 1)
        # weights initialization
        self.linear4.apply(linear_weights_init)
        
    def forward(self, state):
        x = self.activation(self.linear1(state))
        x = self.activation(self.linear2(x))
        x = self.activation(self.linear3(x))
        x = self.linear4(x)
        return x        


class QNetwork(QNetworkBase):
    def __init__(self, state_space, action_space, hidden_dim, activation=F.relu, output_activation=None):
        super().__init__(state_space, action_space, activation)

        self.linear1 = nn.Linear(self._state_dim+self._action_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, hidden_dim)
        self.linear4 = nn.Linear(hidden_dim, 1)
        # weights initialization
        self.linear4.apply(linear_weights_init)
        
    def forward(self, state, action):
        x = torch.cat([state, action], 1) # the dim 0 is number of samples
        x = self.activation(self.linear1(x))
        x = self.activation(self.linear2(x))
        x = self.activation(self.linear3(x))
        x = self.linear4(x)
        return x        

class QNetworkLSTM(QNetworkBase):
    """
    Q network with LSTM structure.
    The network follows two-branch structure as in paper: 
    Sim-to-Real Transfer of Robotic Control with Dynamics Randomization
    """
    def __init__(self, state_space, action_space, hidden_dim, activation=F.relu, output_activation=None):
        super().__init__(state_space, action_space, activation)
        self.hidden_dim = hidden_dim

        self.linear1 = nn.Linear(self._state_dim+self._action_dim, hidden_dim)
        self.linear2 = nn.Linear(self._state_dim+self._action_dim, hidden_dim)
        self.lstm1 = nn.LSTM(hidden_dim, hidden_dim, dropout = 0.2)
        self.linear3 = nn.Linear(2*hidden_dim, hidden_dim)
        self.linear4 = nn.Linear(hidden_dim, 1)
        # weights initialization
        self.linear4.apply(linear_weights_init)
        
    def forward(self, state, action, last_action, hidden_in):
        """ 
        state shape: (batch_size, sequence_length, state_dim)
        output shape: (batch_size, sequence_length, 1)
        for lstm needs to be permuted as: (sequence_length, batch_size, state_dim)
        """
        state = state.permute(1,0,2)
        action = action.permute(1,0,2)
        last_action = last_action.permute(1,0,2)
        # branch 1
        fc_branch = torch.cat([state, action], -1) 
        fc_branch = self.activation(self.linear1(fc_branch))
        # branch 2
        lstm_branch = torch.cat([state, last_action], -1) 
        lstm_branch = self.activation(self.linear2(lstm_branch))  # linear layer for 3d input only applied on the last dim
        lstm_branch, lstm_hidden = self.lstm1(lstm_branch, hidden_in)  # no activation after lstm
        # merged
        merged_branch=torch.cat([fc_branch, lstm_branch], -1) 

        x = self.activation(self.linear3(merged_branch))
        x = self.linear4(x)
        x = x.permute(1,0,2)  # back to same axes as input    
        return x, lstm_hidden    # lstm_hidden is actually tuple: (hidden, cell)   

class QNetworkLSTM2(QNetworkBase):
    """
    Q network with LSTM structure.
    The network follows single-branch structure as in paper: 
    Memory-based control with recurrent neural networks
    """
    def __init__(self, state_space, action_space, hidden_dim, activation=F.relu, output_activation=None):
        super().__init__(state_space, action_space, activation)
        self.hidden_dim = hidden_dim

        self.linear1 = nn.Linear(self._state_dim+2*self._action_dim, hidden_dim)
        self.lstm1 = nn.LSTM(hidden_dim, hidden_dim, dropout = 0.2)
        self.linear2 = nn.Linear(hidden_dim, 1)
        # weights initialization
        self.linear2.apply(linear_weights_init)
        
    def forward(self, state, action, last_action, hidden_in):
        """ 
        state shape: (batch_size, sequence_length, state_dim)
        output shape: (batch_size, sequence_length, 1)
        for lstm needs to be permuted as: (sequence_length, batch_size, state_dim)
        """
        state = state.permute(1,0,2)
        action = action.permute(1,0,2)
        last_action = last_action.permute(1,0,2)
        # single branch
        x = torch.cat([state, action, last_action], -1) 
        x = self.activation(self.linear1(x))
        x, lstm_hidden = self.lstm1(x, hidden_in)  # no activation after lstm
        x = self.linear2(x)
        x = x.permute(1,0,2)  # back to same axes as input    
        return x, lstm_hidden    # lstm_hidden is actually tuple: (hidden, cell)   





class PolicyNetworkBase(nn.Module):
    """ Base network class for policy function """
    def __init__(self, state_space, action_space, action_range):
        super(PolicyNetworkBase, self).__init__()
        self._state_space = state_space
        self._state_shape = state_space.shape
        if len(self._state_shape) == 1:
            self._state_dim = self._state_shape[0]
        else:  # high-dim state
            pass  
        self._action_space = action_space
        self._action_shape = action_space.shape
        if len(self._action_shape) < 1:  # Discrete space
            self._action_dim = action_space.n
        else:
            self._action_dim = self._action_shape[0]
        self.action_range = action_range

    def forward(self):
        pass
    
    def evaluate(self):
        pass 
    
    def get_action(self):
        pass

    def sample_action(self,):
        a=torch.FloatTensor(self._action_dim).uniform_(-1, 1)
        return self.action_range*a.numpy()







class SAC_PolicyNetworkLSTM(PolicyNetworkBase):
    def __init__(self, state_space, action_space, hidden_size, action_range=1., init_w=3e-3, log_std_min=-20, log_std_max=2):
        super().__init__(state_space, action_space, action_range=action_range)
        
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max
        self.hidden_size = hidden_size
        
        self.linear1 = nn.Linear(self._state_dim, hidden_size)
        self.linear2 = nn.Linear(self._state_dim+self._action_dim, hidden_size)
        self.lstm1 = nn.LSTM(hidden_size, hidden_size, dropout = 0.2)
        self.linear3 = nn.Linear(2*hidden_size, hidden_size)

        self.mean_linear = nn.Linear(hidden_size, self._action_dim)
        self.mean_linear.weight.data.uniform_(-init_w, init_w)
        self.mean_linear.bias.data.uniform_(-init_w, init_w)
        
        self.log_std_linear = nn.Linear(hidden_size, self._action_dim)
        self.log_std_linear.weight.data.uniform_(-init_w, init_w)
        self.log_std_linear.bias.data.uniform_(-init_w, init_w)


    def forward(self, state, last_action, hidden_in):
        """ 
        state shape: (batch_size, sequence_length, state_dim)
        output shape: (batch_size, sequence_length, action_dim)
        for lstm needs to be permuted as: (sequence_length, batch_size, -1)
        """
        state = state.permute(1,0,2)
        last_action = last_action.permute(1,0,2)
        # branch 1
        fc_branch = F.relu(self.linear1(state))
        # branch 2
        lstm_branch = torch.cat([state, last_action], -1)
        lstm_branch = F.relu(self.linear2(lstm_branch))
        lstm_branch, lstm_hidden = self.lstm1(lstm_branch, hidden_in)  # no activation after lstm
        # merged
        merged_branch=torch.cat([fc_branch, lstm_branch], -1) 
        x = torch.tanh(self.linear3(merged_branch))
        x = x.permute(1,0,2)  # permute back

        mean    = self.mean_linear(x)
        # mean    = F.leaky_relu(self.mean_linear(x))
        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max)
        
        return mean, log_std, lstm_hidden
    
    def evaluate(self, state, last_action, hidden_in, epsilon=1e-6):
        '''
        generate sampled action with state as input wrt the policy network;
        '''
        mean, log_std, hidden_out = self.forward(state, last_action, hidden_in)
        std = log_std.exp() # no clip in evaluation, clip affects gradients flow
        
        normal = Normal(0, 1)
        z = normal.sample(mean.shape)
        action_0 = torch.tanh(mean + std * z)  # TanhNormal distribution as actions; reparameterization trick
        action = self.action_range * action_0
        log_prob = Normal(mean, std).log_prob(mean + std * z) - torch.log(
            1. - action_0.pow(2) + epsilon) - np.log(self.action_range)
        # both dims of normal.log_prob and -log(1-a**2) are (N,dim_of_action);
        # the Normal.log_prob outputs the same dim of input features instead of 1 dim probability,
        # needs sum up across the features dim to get 1 dim prob; or else use Multivariate Normal.
        log_prob = log_prob.sum(dim=-1, keepdim=True)
        return action, log_prob, z, mean, log_std, hidden_out

    def get_action(self, state, last_action, hidden_in, deterministic=True):
        state = torch.FloatTensor(state).unsqueeze(0).unsqueeze(0) # increase 2 dims to match with training data
        last_action = torch.FloatTensor(last_action).unsqueeze(0).unsqueeze(0)
        mean, log_std, hidden_out = self.forward(state, last_action, hidden_in)
        std = log_std.exp()
        
        normal = Normal(0, 1)
        z = normal.sample(mean.shape)
        action = self.action_range * torch.tanh(mean + std * z)

        action = self.action_range * torch.tanh(mean).detach().cpu().numpy() if deterministic else \
        action.detach().cpu().numpy()
        return action[0][0], hidden_out






'''
Soft Actor-Critic version 2
using target Q instead of V net: 2 Q net, 2 target Q net, 1 policy net
add alpha loss compared with version 1
paper: https://arxiv.org/pdf/1812.05905.pdf
'''


from torch.distributions import Normal
from IPython.display import clear_output
from matplotlib import animation
from IPython.display import display

GPU = False
device_idx = 0
if GPU:
    device = torch.device("cuda:" + str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)



class SAC_Trainer():
    def __init__(self, replay_buffer, state_space, action_space, hidden_dim, action_range, gamma, soft_q_lr, policy_lr, alpha_lr, batch_size, update_itr, reward_scale, target_entropy, soft_tau, train_freq):


        self.gamma = gamma
        self.batch_size = batch_size
        self.update_itr = update_itr
        self.hidden_dim = hidden_dim
        self.reward_scale = reward_scale
        self.target_entropy = target_entropy
        self.soft_tau = soft_tau
        self.train_freq = train_freq

        self.replay_buffer = replay_buffer
        self.soft_q_net1 = QNetworkLSTM2(state_space, action_space, hidden_dim).to(device)
        self.soft_q_net2 = QNetworkLSTM2(state_space, action_space, hidden_dim).to(device)
        self.target_soft_q_net1 = QNetworkLSTM2(state_space, action_space, hidden_dim).to(device)
        self.target_soft_q_net2 = QNetworkLSTM2(state_space, action_space, hidden_dim).to(device)
        self.policy_net = SAC_PolicyNetworkLSTM(state_space, action_space, hidden_dim, action_range).to(device)
        self.log_alpha = torch.zeros(1, dtype=torch.float32, requires_grad=True, device=device)
        print('Soft Q Network (1,2): ', self.soft_q_net1)
        print('Policy Network: ', self.policy_net)

        for target_param, param in zip(self.target_soft_q_net1.parameters(), self.soft_q_net1.parameters()):
            target_param.data.copy_(param.data)
        for target_param, param in zip(self.target_soft_q_net2.parameters(), self.soft_q_net2.parameters()):
            target_param.data.copy_(param.data)

        self.soft_q_criterion1 = nn.MSELoss()
        self.soft_q_criterion2 = nn.MSELoss()

        # soft_q_lr = 0.0015
        # policy_lr = 0.0015
        # alpha_lr  = 0.0015

        self.soft_q_optimizer1 = optim.Adam(self.soft_q_net1.parameters(), lr=soft_q_lr)
        self.soft_q_optimizer2 = optim.Adam(self.soft_q_net2.parameters(), lr=soft_q_lr)
        self.policy_optimizer = optim.Adam(self.policy_net.parameters(), lr=policy_lr)
        self.alpha_optimizer = optim.Adam([self.log_alpha], lr=alpha_lr)

    
    def update(self, batch_size, reward_scale=10., auto_entropy=True, target_entropy=-2, gamma=0.975,soft_tau=1e-2):
        hidden_in, hidden_out, state, action, last_action, reward, next_state, done = self.replay_buffer.sample(batch_size)
        # print('sample:', state, action,  reward, done)

        batch_size = self.batch_size
        reward_scale = self.reward_scale
        gamma = self.gamma
        target_entropy = self.target_entropy
        soft_tau = self.soft_tau


        state      = torch.FloatTensor(np.array(state)).to(device)
        next_state = torch.FloatTensor(np.array(next_state)).to(device)
        action     = torch.FloatTensor(np.array(action)).to(device)
        last_action     = torch.FloatTensor(np.array(last_action)).to(device)
        reward     = torch.FloatTensor(np.array(reward)).unsqueeze(-1).to(device)  # reward is single value, unsqueeze() to add one dim to be [reward] at the sample dim;
        done       = torch.FloatTensor(np.float32(done)).unsqueeze(-1).to(device)


        # state      = torch.FloatTensor(state).to(device)
        # next_state = torch.FloatTensor(next_state).to(device)
        # action     = torch.FloatTensor(action).to(device)
        # last_action     = torch.FloatTensor(last_action).to(device)
        # reward     = torch.FloatTensor(reward).unsqueeze(-1).to(device)  # reward is single value, unsqueeze() to add one dim to be [reward] at the sample dim;
        # done       = torch.FloatTensor(np.float32(done)).unsqueeze(-1).to(device)

        predicted_q_value1, _ = self.soft_q_net1(state, action, last_action, hidden_in)
        predicted_q_value2, _ = self.soft_q_net2(state, action, last_action, hidden_in)
        new_action, log_prob, z, mean, log_std, _ = self.policy_net.evaluate(state, last_action, hidden_in)
        new_next_action, next_log_prob, _, _, _, _ = self.policy_net.evaluate(next_state, action, hidden_out)
        reward = reward_scale * (reward - reward.mean(dim=0)) / (reward.std(dim=0) + 1e-6) # normalize with batch mean and std; plus a small number to prevent numerical problem
    # Updating alpha wrt entropy
        # alpha = 0.0  # trade-off between exploration (max entropy) and exploitation (max Q) 
        if auto_entropy is True:
            alpha_loss = -(self.log_alpha * (log_prob + target_entropy).detach()).mean()
            # print('alpha loss: ',alpha_loss)
            self.alpha_optimizer.zero_grad()
            alpha_loss.backward()
            self.alpha_optimizer.step()
            self.alpha = self.log_alpha.exp()
        else:
            self.alpha = 1.
            alpha_loss = 0

    # Training Q Function
        predict_target_q1, _ = self.target_soft_q_net1(next_state, new_next_action, action, hidden_out)
        predict_target_q2, _ = self.target_soft_q_net2(next_state, new_next_action, action, hidden_out)
        target_q_min = torch.min(predict_target_q1, predict_target_q2) - self.alpha * next_log_prob
        target_q_value = reward + (1 - done) * gamma * target_q_min # if done==1, only reward
        q_value_loss1 = self.soft_q_criterion1(predicted_q_value1, target_q_value.detach())  # detach: no gradients for the variable
        q_value_loss2 = self.soft_q_criterion2(predicted_q_value2, target_q_value.detach())


        self.soft_q_optimizer1.zero_grad()
        q_value_loss1.backward()
        self.soft_q_optimizer1.step()
        self.soft_q_optimizer2.zero_grad()
        q_value_loss2.backward()
        self.soft_q_optimizer2.step()  

    # Training Policy Function
        predict_q1, _= self.soft_q_net1(state, new_action, last_action, hidden_in)
        predict_q2, _ = self.soft_q_net2(state, new_action, last_action, hidden_in)
        predicted_new_q_value = torch.min(predict_q1, predict_q2)
        policy_loss = (self.alpha * log_prob - predicted_new_q_value).mean()

        self.policy_optimizer.zero_grad()
        policy_loss.backward()
        self.policy_optimizer.step()
        
        # print('q loss: ', q_value_loss1, q_value_loss2)
        # print('policy loss: ', policy_loss )


    # Soft update the target value net
        for target_param, param in zip(self.target_soft_q_net1.parameters(), self.soft_q_net1.parameters()):
            target_param.data.copy_(  # copy data value into target parameters
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )
        for target_param, param in zip(self.target_soft_q_net2.parameters(), self.soft_q_net2.parameters()):
            target_param.data.copy_(  # copy data value into target parameters
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )
        return predicted_new_q_value.mean()

    def save_model(self, path):
        torch.save(self.soft_q_net1.state_dict(), path+'/lstm_q1')
        torch.save(self.soft_q_net2.state_dict(), path+'/lstm_q2')
        torch.save(self.policy_net.state_dict(), path+'/lstm_policy')

    def load_model(self, path):
        self.soft_q_net1.load_state_dict(torch.load(path+'/lstm_q1', map_location=torch.device('cpu')))
        self.soft_q_net2.load_state_dict(torch.load(path+'/lstm_q2', map_location=torch.device('cpu')))
        self.policy_net.load_state_dict(torch.load(path+'/lstm_policy', map_location=torch.device('cpu')))

        self.soft_q_net1.eval()
        self.soft_q_net2.eval()
        self.policy_net.eval()


def plot(rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.plot(rewards)
    plt.savefig('sac_v2_lstm.png')
    # plt.show()




#### make agent

In [ ]:
replay_buffer_size = 100
batch_size = 10
replay_buffer = ReplayBufferLSTM2(replay_buffer_size)

# choose env
action_space = env.action_space
state_space  = env.observation_space
action_range=1.

action_dim = action_space.shape[0]

# hyper-parameters for RL training
max_episodes  = 1000
max_steps   = 1024
frame_idx   = 0
AUTO_ENTROPY=True
DETERMINISTIC=False
rewards     = []
train_freq = 100


# # env includes memory
# gamma=1-0.017155091291799374
# soft_q_lr=0.0010223596792273903
# policy_lr=1.670414519639193e-05
# alpha_lr=0.0005012216457204279
# batch_size=512
# update_itr=4
# hidden_dim=256
# reward_scale=10
# target_entropy=-2
# soft_tau=0.004341682252329602
# train_freq = 10

# # env does not include memory
# gamma = 1-0.0008767920887906205
# soft_q_lr = 3.45249248634654e-05
# policy_lr = 0.003925404720612826
# alpha_lr = 0.03434388145436511
# batch_size = 16
# update_itr = 1
# hidden_dim = 256
# reward_scale = 100
# target_entropy = -2
# soft_tau = 1.2326412869674132e-06
# train_freq = 10


#sac_trainer=SAC_Trainer(replay_buffer, state_space, action_space, hidden_dim=hidden_dim, action_range=action_range  )
sac_trainer=SAC_Trainer(replay_buffer, state_space, action_space, hidden_dim=hidden_dim, action_range=action_range,\
                          gamma = gamma, soft_q_lr = soft_q_lr, policy_lr = policy_lr, alpha_lr = alpha_lr, \
                          batch_size = batch_size, update_itr = update_itr, reward_scale = reward_scale, \
                        target_entropy = target_entropy, soft_tau = soft_tau, train_freq = train_freq)



#### LOAD (not simulated)

In [ ]:
class CollectInformation(MultiFF):  # Note when using this wrapper, the number of steps cannot exceed one episode
  def __init__(self):
      super().__init__()
      self.ff_information = np.ones([self.num_ff, 8])*(-9999)   #[index, x, y, time_start, time_captured, mx(when_captured), my(when_captured), index_in_flash]
      self.episode_len = 15000
      self.num_intervals = 25000
  def reset(self):
      self.obs = super().reset()
      self.ff_information = np.ones([self.num_ff, 8])*(-9999)   #[index, x, y, time_start, time_captured, mx(when_captured), my(when_captured), index_in_flash]
      self.ff_information[:,0] = np.arange(self.num_ff)
      self.ff_information[:,7] = np.arange(self.num_ff)
      self.ff_information[:,1] = self.ffx.clone().numpy()
      self.ff_information[:,2] = self.ffy.clone().numpy()
      self.ff_information[:,3] = 0
      return self.obs

  def calculate_reward(self):
      reward = super().calculate_reward()
      if self.num_targets > 0: 
        for index in self.captured_ff_index:
          overall_index = int(self.ff_information[:,0][np.where(self.ff_information[:,-1]==index)[0][-1]])
          self.ff_information[overall_index, 4] = self.time
          self.ff_information[overall_index, 5] = self.agentx.item()
          self.ff_information[overall_index, 6] = self.agenty.item()
        ## The following codes are not needed because this is for simulation
        self.new_ff_info = np.ones([self.num_targets, 8])*(-9999)
        self.new_ff_info[:,0] = np.arange(len(self.ff_information), len(self.ff_information)+self.num_targets)
        self.new_ff_info[:,7] = np.array(self.captured_ff_index)
        self.new_ff_info[:,1] = self.ffx[self.captured_ff_index].numpy()
        self.new_ff_info[:,2] = self.ffy[self.captured_ff_index].numpy()
        self.new_ff_info[:,3] = self.time + self.dt
        self.ff_information = np.concatenate([self.ff_information, self.new_ff_info], axis = 0)
      return(reward)


env = CollectInformation()

env.flash_on_interval = 0.3
env.distance2center_cost = 0

sac_trainer.load_model(model_path)

### test (LSTM)

In [ ]:
env = CollectInformation()

env.flash_on_interval = 0.3
env.distance2center_cost = 0
sac_trainer.load_model(model_path)


# env.flash_on_interval = 0.3

# model_path = "/content/gdrive/MyDrive/fireflies_agent/lstm/July_17"
# sac_trainer.load_model(model_path)

# Test the trained agent
n_steps = 10000

obs = env.reset()

state = obs

last_action = env.action_space.sample()
hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float), \
        torch.zeros([1, 1, hidden_dim], dtype=torch.float))  # initialize hidden state for lstm, (hidden, cell), each is (layer, batch, dim)             
episode_reward = 0
hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float), \
    torch.zeros([1, 1, hidden_dim], dtype=torch.float))  # initialize hidden state for lstm, (hidden, cell), each is (layer, batch, dim)
mx = []
my = []
monkey_t = []
monkey_speed_info = []
monkey_angle = [] # in radians
ffxy_all = []
ffxy_visible = []
ffxy2_all = []
time_rl = []
reward_log = []
mx_rewarded = []
my_rewarded = []
mheading = []
captured_ff = []
current_target = []
action_reward = []
num_targets = []
env_obs = []
#captured_ff_cum_x = []
#captured_ff_cum_y = []
all_captured_ff_x = []
all_captured_ff_y = []
visible_ff_indices_all = []
memory_ff_indices_all = []
obs_ff_indices_all = []
obs_ff_overall_indices_all = []
memory_ff_indices_all = []
obs_ff_indices_all = []
ff_angles2 = []
ff_distances2 = []
for step in range(n_steps):
  hidden_in = hidden_out
  action, hidden_out = sac_trainer.policy_net.get_action(state, last_action, hidden_in, deterministic = DETERMINISTIC)
  previous_ffxy = env.ffxy
  prev_ff_information = env.ff_information.copy()
  next_state, reward, done, _ = env.step(action)  
  #print(step, "Action: ", action)
  #print(hidden_out)
  last_action = action
  episode_reward += reward
  state=next_state
  reward_log.append(reward)
  num_targets.append(env.num_targets)
  if env.num_targets > 0:
    #print(step, "num_targets:",  env.num_targets)
    captured_ff.append(env.captured_ff_index)
    all_captured_ff_x = all_captured_ff_x + previous_ffxy[env.captured_ff_index][:,0].tolist()
    all_captured_ff_y = all_captured_ff_y + previous_ffxy[env.captured_ff_index][:,1].tolist()
  else:
    captured_ff.append(0)
  #captured_ff_cum_x.append(all_captured_ff_x)
  #captured_ff_cum_y.append(all_captured_ff_y)
  mx.append(env.agentx.item())
  my.append(env.agenty.item())
  monkey_t.append(env.time)
  monkey_speed_info.append(env.dv.item())
  mheading.append(env.agentheading.item())
  time_rl.append(env.time)
  ffxy_all.append(env.ffxy.clone())
  ffxy2_all.append(env.ffxy2.clone())
  ffxy_visible.append(env.ffxy[env.visible_ff_indices].clone())
  env_obs.append(next_state.copy())
  visible_ff_indices_all.append(env.visible_ff_indices)
  obs_ff_indices_all.append(env.topk_indices.numpy())
  real_indices = []
  for index in env.topk_indices:
    real_indices.append(int(prev_ff_information[:,0][np.where(prev_ff_information[:,7]==index.item())[0][-1]].copy()))
  obs_ff_overall_indices_all.append(real_indices)
  if len(env.topk_indices) > 0:
    ff_angles2.append(env.ff_angle_topk_2)
    ff_distances2.append(env.ff_distance_topk)
  else:
    ff_angles2.append(torch.tensor([]))
    ff_distances2.append(torch.tensor([]))
  if done:
    obs = env.reset()
  #print(step, ffxy_visible[-1])

### monkey & ff information

In [ ]:
monkey_speed = np.array(monkey_speed_info)

monkey_information = {
'monkey_x': np.array(mx),
'monkey_y': np.array(my),
'monkey_t': np.array(monkey_t),
'monkey_speed': monkey_speed,
'monkey_angle': np.array(mheading),
}


monkey_information['monkey_speeddummy'] = (monkey_speed>200*0.01*env.dt).astype(int) 
monkey_speeddummy = (monkey_speed>200*0.01*env.dt).astype(int) 
delta_time = np.delete(monkey_information['monkey_t'], 0) - np.delete(monkey_information['monkey_t'], -1)
monkey_dw = np.diff(np.array(mheading), prepend=mheading[0])/env.dt
monkey_information['monkey_dw'] = monkey_dw


#ff: 
#[index, x, y, time_start, time_captured, mx(when_captured), my(when_captured), index_in_flash]
# Note: when collecting firefly data, no more than one episode can be run. Otherwise the data might be messed up.
# A solution is to elongate an episode


# sort the time of capture for ff (if they have been captured)
ff_information = env.ff_information.copy()
ff_time_captured_all = ff_information[:,4]
captured_ff_indices = np.where(ff_time_captured_all != -9999)[0]
not_captured_ff_indices = np.where(ff_time_captured_all == -9999)[0]
num_captured_ff = len(captured_ff_indices)
sorted_indices_captured = captured_ff_indices[np.argsort(ff_time_captured_all[captured_ff_indices])]
sort_indices_all = np.concatenate([sorted_indices_captured, not_captured_ff_indices])
ff_information_sorted = ff_information[sort_indices_all]




# make ff_flash_sorted
ff_flash_sorted = []
env_end_time = env.time
for ff in ff_information[sorted_indices_captured]:
  flash = env.ff_flash[int(ff[7])].numpy()
  replace_start = False
  replace_end = False

  # first flatten flash, and then find the elements that are before ff start time
  # if the number of elements is even, then the flashing_start_time is the start time of the next interval
  before_start = np.where(flash.flatten() <= ff[3])[0]
  if len(before_start) > 0:
    if len(before_start)%2 == 0:
      start_flash_index = int(len(before_start)/2)
    else:
      start_flash_index = int((len(before_start)-1)/2)
      replace_start = True
  else:
      start_flash_index = 0


  after_finish = np.where(flash.flatten() >= ff[4])[0]
  if len(after_finish) > 0:
    num_indices_before_finish = after_finish[0]
    if num_indices_before_finish%2 == 0:
      end_flash_index = int(num_indices_before_finish/2) - 1
    else: 
      end_flash_index = int((num_indices_before_finish+1)/2) - 1
      replace_end = True
  else:
    end_flash_index = len(flash)-1


  
  ff_flash = flash[start_flash_index:(end_flash_index+1)]

  if len(ff_flash) < 2:
    ff_flash = flash[end_flash_index-1: end_flash_index+1]
    if len(ff_flash) < 2:
      ff_flash = np.array([[-1,-1]])


  if replace_start == True:
    ff_flash[0, 0] = ff[3]
  if replace_end == True:
    ff_flash[-1, 1] = ff[4]

  if len(ff_flash) == 0:
    ff_flash = np.array([[-1,-1]])

  ff_flash_sorted.append(ff_flash)



# For the ffs that have never been captured, the end_flash_time is evaluated not in relation to the 
# time of captrue, but to the time that the env ends
for ff in ff_information[not_captured_ff_indices]:
  flash = env.ff_flash[int(ff[7])].numpy()
  replace_start = False


  # first flatten flash, and then find the elements that are before ff start time
  # if the number of elements is even, then the flashing_start_time is the start time of the next interval
  before_start = np.where(flash.flatten() <= ff[3])[0]
  if len(before_start)%2 == 0:
    start_flash_index = int(len(before_start)/2)
  else:
    start_flash_index = int((len(before_start)-1)/2)
    replace_start = True


# # if we only want the part before the testing ends
#   after_finish = np.where(flash.flatten() >= env_end_time)[0] # differing from captured ffs
#   num_indices_before_finish = after_finish[0]
#   if num_indices_before_finish%2 == 0:
#     end_flash_index = int(num_indices_before_finish/2) - 1
#   else: 
#     end_flash_index = int((num_indices_before_finish+1)/2) - 1
#     replace_end = True
#   ff_flash = flash[start_flash_index:(end_flash_index+1)]


  ff_flash = flash[start_flash_index:]

  if len(ff_flash) < 2:
    ff_flash = flash[end_flash_index-1: end_flash_index+1]
    if len(ff_flash) < 2:
      ff_flash = np.array([[-1,-1]])


  if replace_start == True:
    ff_flash[0, 0] = ff[3]

  # # if we only want the part before the testing ends
  # if replace_end == True:
  #   ff_flash[-1, 1] = ff[4]
  if len(ff_flash) == 0:
    ff_flash = np.array([[-1,-1]])

  ff_flash_sorted.append(ff_flash)


ff_catched_T_sorted = ff_time_captured_all[sorted_indices_captured] # Note that these two will be shorter than the other arrays
ff_believed_position_sorted = ff_information[:,5:7][sorted_indices_captured]

ff_real_position_sorted = ff_information[:,1:3][sort_indices_all]
ff_life_sorted = ff_information[:,3:5][sort_indices_all]
ff_life_sorted[:,1][np.where(ff_life_sorted[:,1]==-9999)[0]] = env.time
ff_flash_end_sorted = [flash[-1,1] if len(flash) > 0 else env.time for flash in ff_flash_sorted]
ff_flash_end_sorted = np.array(ff_flash_end_sorted)

catched_ff_num = len(ff_catched_T_sorted) - 200
total_ff_num = len(ff_life_sorted)

## Organize data

#### ff_dataframe

Note: here angels are in radians, whereas in previous stages, angles are in degrees. In addition, here angle_boundary is added. It replaces angle as a standard to select valid points. And because of that, more points are selected into the dataframe.

In [ ]:
if NEW_DATASET == False:
  filepath = 'gdrive/MyDrive/fireflies_data/' + data_folder_name + '/ff_dataframe.csv'
  ff_dataframe = pd.read_csv(filepath)
  if len(ff_catched_T_sorted) > 400:
    ff_dataframe = ff_dataframe[ff_dataframe['time'] < ff_catched_T_sorted[-200]]
  ff_dataframe = ff_dataframe.drop(["Unnamed: 0"], axis=1)
  ff_dataframe.memory = ff_dataframe.memory.astype('int')
  
  # Let's use data from monkey_information. But we shall cut off portion that is before the time of capturing the first target
  monkey_t_array0 = np.array(monkey_information['monkey_t'])
  monkey_x_array0 = np.array(monkey_information['monkey_x'])
  monkey_y_array0 = np.array(monkey_information['monkey_y'])
  monkey_angle_array0 = np.array(monkey_information['monkey_angle'])
  valid_index = np.where(monkey_t_array0 > ff_catched_T_sorted[0])[0]
  num_missed_index = valid_index[0]
  monkey_t_array = monkey_t_array0[valid_index]
  monkey_x_array = monkey_x_array0[valid_index]
  monkey_y_array = monkey_y_array0[valid_index]
  monkey_angle_array = monkey_angle_array0[valid_index]
  index_array = np.array(range(len(monkey_t_array)))
  ff_dataframe['target_x'] = ff_real_position_sorted[np.array(ff_dataframe['target_index'])][:,0]
  ff_dataframe['target_y'] = ff_real_position_sorted[np.array(ff_dataframe['target_index'])][:,1]
  ff_dataframe['ffdistance2target'] = LA.norm(np.array(ff_dataframe[['ff_x', 'ff_y']])-np.array(ff_dataframe[['target_x', 'target_y']]), axis = 1)
  min_point_index = np.min(np.array(ff_dataframe['point_index']))
  max_point_index = np.max(np.array(ff_dataframe['point_index']))

##### for agent
(only include those in obs space)

In [ ]:
if NEW_DATASET == True:
  
  # Let's use data from monkey_information. But we shall cut off portion that is before the time of capturing the first target
  monkey_t_array0 = np.array(monkey_information['monkey_t'])
  monkey_x_array0 = np.array(monkey_information['monkey_x'])
  monkey_y_array0 = np.array(monkey_information['monkey_y'])
  monkey_angle_array0 = np.array(monkey_information['monkey_angle'])
  valid_index = np.where(monkey_t_array0 > ff_catched_T_sorted[0])[0]
  num_missed_index = valid_index[0]
  monkey_t_array = monkey_t_array0[valid_index]
  monkey_x_array = monkey_x_array0[valid_index]
  monkey_y_array = monkey_y_array0[valid_index]
  monkey_angle_array = monkey_angle_array0[valid_index]
  #index_array = np.array(range(len(monkey_t_array))) this overlaps with another variable later


  max_distance = 400
  reward_boundary = 25
  #max_time = 400
  #max_time = ff_catched_T_sorted[1251] #replace with ff_catched_T_sorted[-1]
  ff_index = []
  point_index = []
  time = []
  target_index = []
  ff_x = []
  ff_y = []
  monkey_x = []
  monkey_y = []
  visible = []
  memory = []
  ff_distance = []
  ff_angle = []
  ff_angle_boundary = []
  left_right = []
  num_captures = []
  distance_closest_ff =[]
  distance_2ndclosest_ff = []
  num_ff_within = []
  reward_distance = []
  reward_angle = []
  catched_ff_num = len(ff_catched_T_sorted) - 200 
  total_ff_num = len(ff_life_sorted)


  for i in range(total_ff_num):

    if i % 100 == 0:
      print(i," out of ", total_ff_num)


    # Go through every visible duration of the same ff
    ff_flash = ff_flash_sorted[i]
    whether_in_obs = []
    original_index = sort_indices_all[i]
    for index in valid_index:
      obs_ff_indices = obs_ff_overall_indices_all[index]
      if original_index in obs_ff_indices:
        whether_in_obs.append(True) 
      else:
        whether_in_obs.append(False)

    
    cum_indices = np.array(whether_in_obs).nonzero()[0]
    if len(cum_indices) > 0:

      updated_t_array = monkey_t_array[cum_indices]
      visible_indices = cum_indices
      index_array = cum_indices
      in_memory_length = len(index_array)
        
      cum_t, cum_angle = monkey_t_array[cum_indices], monkey_angle_array[cum_indices]
      cum_mx, cum_my = monkey_x_array[cum_indices], monkey_y_array[cum_indices]
      distances_to_monkey = LA.norm(np.stack([cum_mx, cum_my], axis=1)-ff_real_position_sorted[i], axis = 1)
      
      angles_to_monkey = np.arctan2(ff_real_position_sorted[i,1]-cum_my, ff_real_position_sorted[i,0]-cum_mx)-cum_angle
      angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
      angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi
      angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(reward_boundary, np.maximum(distances_to_monkey, reward_boundary) ))) # use torch clip to get valid arcsin input
      angles_adjusted = np.clip(angles_adjusted, 0, pi)
      angles_to_monkey = np.sign(angles_to_monkey)* angles_adjusted



      # Make a numpy array of points to denote memory, with 0 means being invisible. We also append 99 extra points after the last point    
      memory_indices0 = np.zeros(visible_indices[-1]+100, dtype=int)
      memory_indices0[visible_indices] = 100

      if len(ff_catched_T_sorted)-1 >= i:
        # Find the index of the time at which the ff is captured
        last_live_time_array = np.where(monkey_t_array <= ff_catched_T_sorted[i])[0]
        if len(last_live_time_array) > 0:
          last_live_time = last_live_time_array[-1]
        # Truncate memory_indices0 based on that 
          memory_indices0 = memory_indices0[:last_live_time+1]

      
      # Iterate through memory_indices0 to make a new list to denote memory (replacing some 0s with other numbers based on time)
      # We preserve the first element of memory_indices0. We also separate memory_indices and point_indices (denoted as final_indices)
      memory_indices = [memory_indices0[0]]

      for k in range(1, len(memory_indices0)):
        if memory_indices0[k] == 0:
          memory_indices.append(memory_indices[k-1]-1)
        else: # Else, preserve the current value
          memory_indices.append(memory_indices0[k])
      memory_indices_array = np.array(memory_indices)

      index_array0 = np.arange(len(memory_indices0))
      if len(index_array0) > len(monkey_t_array):
        max_index = len(monkey_t_array)
        index_array0 = index_array0[:max_index]
        memory_indices_array = memory_indices_array[index_array0]

      in_memory_indices = np.where(memory_indices_array > 0)[0]
      memory_indices_array = memory_indices_array[in_memory_indices]
      index_array = index_array0[in_memory_indices]
      in_memory_length = len(memory_indices_array)
      memory_indices = memory_indices_array.tolist()
      final_indices = index_array.tolist()

      ff_index = ff_index + [i]*in_memory_length
      point_index = point_index+ [point + num_missed_index for point in final_indices]
      relevant_time = monkey_t_array[index_array]
      time = time+relevant_time.tolist()
      target_index = target_index + np.digitize(relevant_time, ff_catched_T_sorted).tolist()
      ff_x = ff_x + [(ff_real_position_sorted[i][0])]*in_memory_length
      ff_y = ff_y + [(ff_real_position_sorted[i][1])]*in_memory_length
      monkey_x = monkey_x + monkey_x_array[index_array].tolist()
      monkey_y = monkey_y + monkey_y_array[index_array].tolist()
      visible = visible + [1 if point ==100 else 0 for point in memory_indices]
      memory = memory + memory_indices
      monkey_xy_relevant = np.stack([monkey_x_array[index_array], monkey_y_array[index_array]],axis=1)
      monkey_angle_relevant = monkey_angle_array[index_array]
      ff_distance_relevant = LA.norm(monkey_xy_relevant-ff_real_position_sorted[i], axis=1)
      ff_distance = ff_distance + ff_distance_relevant.tolist()
      angles_to_monkey = np.arctan2(ff_real_position_sorted[i,1]-monkey_xy_relevant[:,1], ff_real_position_sorted[i,0]-monkey_xy_relevant[:,0]) - monkey_angle_relevant
      angles_to_monkey[angles_to_monkey > pi] = angles_to_monkey[angles_to_monkey > pi] - 2*pi
      angles_to_monkey[angles_to_monkey < -pi] = angles_to_monkey[angles_to_monkey < -pi] + 2*pi
      angles_adjusted = np.absolute(angles_to_monkey)-np.abs(np.arcsin(np.divide(reward_boundary, np.maximum(ff_distance_relevant, reward_boundary) ))) # use torch clip to get valid arcsin input
      angles_adjusted = np.clip(angles_adjusted, 0, pi)
      angles_adjusted = np.sign(angles_to_monkey)* angles_adjusted
      ff_angle = ff_angle + angles_to_monkey.tolist()
      ff_angle_boundary = ff_angle_boundary + angles_adjusted.tolist()
      left_right = left_right + (np.array(angles_to_monkey) > 0).astype(int).tolist()



        
  # Now let's create a dictionary of the lists
  ff_dict = {'ff_index':ff_index, 'point_index':point_index, 'time':time, 'target_index':target_index,
              'ff_x':ff_x, 'ff_y':ff_y, 'monkey_x':monkey_x, 'monkey_y':monkey_y, 'visible':visible,
              'memory':memory, 'ff_distance':ff_distance, 'ff_angle':ff_angle, 'ff_angle_boundary': ff_angle_boundary, 'left_right':left_right}
  ff_dataframe = pd.DataFrame(ff_dict)

  filepath = 'gdrive/MyDrive/fireflies_data/' + data_folder_name + '/ff_dataframe.csv'
  os.makedirs('gdrive/MyDrive/fireflies_data/' + data_folder_name, exist_ok = True)
  ff_dataframe.to_csv(filepath) 

  ff_dataframe['target_x'] = ff_real_position_sorted[np.array(ff_dataframe['target_index'])][:,0]
  ff_dataframe['target_y'] = ff_real_position_sorted[np.array(ff_dataframe['target_index'])][:,1]
  ff_dataframe['ffdistance2target'] = LA.norm(np.array(ff_dataframe[['ff_x', 'ff_y']])-np.array(ff_dataframe[['target_x', 'target_y']]), axis = 1)

  #ff_dataframe = ff_dataframe[ff_dataframe['time'] < ff_catched_T_sorted[-200]]

  min_point_index = np.min(np.array(ff_dataframe['point_index']))
  max_point_index = np.max(np.array(ff_dataframe['point_index']))


# Categorize all trials



## Capture more than 1 in a cluster

A change: instead of using ff_real_position_sorted, I now use ff_believed_position_sorted

In [ ]:
n_ff_in_a_row = [1]
distance_between_ff = 50
count = 1
# n_ff_in_a_row[k] will denote the number of ff that the monkey has captured in a row at trial k
for i in range(1, catched_ff_num):
  if LA.norm(ff_believed_position_sorted[i]-ff_believed_position_sorted[i-1]) < distance_between_ff:
    count += 1
  else:
    count = 1
  n_ff_in_a_row.append(count)


n_ff_in_a_row = np.array(n_ff_in_a_row)


two_in_a_row= np.where(n_ff_in_a_row==2)[0]
dif_time = ff_catched_T_sorted[two_in_a_row] - ff_catched_T_sorted[two_in_a_row-1]
two_in_a_row_simul = two_in_a_row[np.where(dif_time <= 0.1)[0]]
two_in_a_row_non_simul = two_in_a_row[np.where(dif_time > 0.1)[0]]

In [ ]:
two_in_a_row_non_simul

## on before last one

In the future, a sub-category can be created where the target is far from any other ff that has been visible for a while

In [ ]:
on_before_last_one_trials = [] # Note, the formula has been changed from the previous stages
for i in range(1, catched_ff_num):
  # Evaluate whether the last flash of the current ff finishes before the capture of the previous ff
  if ff_flash_end_sorted[i] < ff_catched_T_sorted[i-1]:
    # Then we consider special situations
    # Make sure that the index is not out of bound
    if i > 1: 
      # If the monkey captures 2 fireflies at the same time, then the trial does not count as "on_before_last_one"
      if ff_catched_T_sorted[i] == ff_catched_T_sorted[i-1]:
        continue
    # Append the index into the list
    on_before_last_one_trials.append(i)

on_before_last_one_trials = np.array(on_before_last_one_trials)


dif_time2 = ff_catched_T_sorted[on_before_last_one_trials] - ff_catched_T_sorted[on_before_last_one_trials-1]
on_before_last_one_simul = on_before_last_one_trials[np.where(dif_time2 <= 0.1)[0]]
on_before_last_one_non_simul = on_before_last_one_trials[np.where(dif_time2 > 0.1)[0]]
print(len(on_before_last_one_non_simul))
len(on_before_last_one_simul)

## visible before last one

In [ ]:
temp_dataframe = ff_dataframe[(ff_dataframe['target_index']==ff_dataframe['ff_index']) & (ff_dataframe['visible'] == 1)]
trials_not_to_select = np.unique(np.array(temp_dataframe['target_index']))
all_trials = np.unique(np.array(ff_dataframe['target_index']))
visible_before_last_one_trials = np.setdiff1d(all_trials, trials_not_to_select)

## used clusters

In [ ]:
used_cluster = np.intersect1d(two_in_a_row_non_simul, on_before_last_one_trials)

## disappear_latest

In [ ]:
#By trial
ff_dataframe_visible = ff_dataframe[(ff_dataframe['visible']==1)]
# For each trial, find out the point index where the monkey last sees a ff
last_visible_index = ff_dataframe_visible[['point_index','target_index']].groupby('target_index').max()
# Take out all the rows corresponding to these points
last_visible_ffs = pd.merge(last_visible_index, ff_dataframe_visible, how="left")
# Select trials where the target disappears the latest
disappear_latest_trials = np.array(last_visible_ffs[last_visible_ffs['target_index']==last_visible_ffs['ff_index']]['target_index'])
not_disappear_latest_trials = np.unique(np.array(last_visible_ffs[last_visible_ffs['target_index']!=last_visible_ffs['ff_index']]['target_index']))
'''
#By points (problem: the time for annotation is too short, and can be distracting)
## When did the targets start to be visible for the last time?
# For each trial, find the rows from ff_dataframe about the target
disappear_latest_points = []
for i in disappear_latest_trials:
  rows = ff_dataframe[ff_dataframe['ff_index'] == i]
  # Find the index of the point when the target changes from invisible to visible. Keep the last point of such change.
  # If not such point exists, then just keep the first point
  turning_points = np.where(np.ediff1d(np.array(rows['visible'])) == 1)[0]
  if len(turning_points) > 0:
    starting_index = turning_points[-1]+1
    disappear_latest_points = disappear_latest_points + list(range(np.array(rows['point_index'])[starting_index], np.array(rows['point_index'])[-1]+1))
# Subtract 50 points from all points
'''

In [ ]:
(ff_dataframe['ff_distance']<250)

In [ ]:
len(disappear_latest_trials)

## Cluster around targets

The method is changed from the previous stage so that the visibility of ffs (distance & angle) is also considered

In [ ]:
max_time_apart = 1.25
# See if the target is close to any ff that has been visible in the past 5s
ffs_around_target = []
ffs_around_target_indices = []
ffs_around_target_positions = [] # Stores the positions of the ffs around the target
temp_frame = ff_dataframe[['ff_index', 'target_index', 'ff_distance', 'visible', 'time']]
for i in range(catched_ff_num):
  time = ff_catched_T_sorted[i]
  duration = [time-max_time_apart, time+max_time_apart]
  target_nums = np.arange(i-1, i+2)
  temp_frame2 = temp_frame[(temp_frame['time']>duration[0])&(temp_frame['time']<duration[1])]
  #temp_frame2 = temp_frame[temp_frame['target_index'].isin(target_nums)]
  temp_frame2 = temp_frame2[~temp_frame2['ff_index'].isin(target_nums)]
  temp_frame2 = temp_frame2[(temp_frame2['visible'] ==1)]
  temp_frame2 = temp_frame2[temp_frame2['ff_distance'] < 250]
  past_visible_ff_indices = np.unique(np.array(temp_frame2.ff_index))
  # Get positions of these ffs
  past_visible_ff_positions = ff_real_position_sorted[past_visible_ff_indices]
  # See if any one of it is within 50 cm of the target
  distance2target = LA.norm(past_visible_ff_positions - ff_real_position_sorted[i], axis=1)
  close_ff_indices = np.where(distance2target < 50)[0]
  num_ff = len(close_ff_indices)
  ffs_around_target.append(num_ff)
  if len(close_ff_indices) > 0:
    ffs_around_target_positions.append(past_visible_ff_positions[close_ff_indices])
    ffs_around_target_indices.append(past_visible_ff_indices[close_ff_indices])
  else:
    ffs_around_target_positions.append(np.array([]))
    ffs_around_target_indices.append(np.array([]))
ffs_around_target = np.array(ffs_around_target)
ffs_around_target_trials = np.where(ffs_around_target > 0)[0]

## Waste cluster around last target

The method is changed from the previous stage

In [ ]:
# And then look at "capture # ffs in a row"
# If in the previous trial, the target has at least one ff in the vicinity, and yet the current trial is not categorized as capturing more than 1 ff in a row. 
# Then we categorize such a trial as "waste cluster of last target"
# This means that while the previous target is in a cluster, the monkey didn't capture the ffs in its vicinity.

waste_cluster_last_target_trials = np.intersect1d(ffs_around_target_trials+1, np.where(n_ff_in_a_row == 1)[0] )
# Note, if one wants to see the target around which there is a cluster, use waste_cluster_last_target_trials-1

## Clusters of ffs




In [ ]:
# temp_dataframe1 = pd.DataFrame(point_vs_cluster, columns=['point_index', 'ff_index', 'cluster_label'])
# u, c = np.unique(point_vs_cluster[:,0], return_counts=True)
# temp_dataframe2 = pd.DataFrame(np.concatenate([u.reshape(-1,1), c.reshape(-1,1)], axis=1), columns = ['point_index','num_ff_at_point'])
# temp_dataframe3 = temp_dataframe1.merge(temp_dataframe2, how = "left", on = "point_index")
# corresponding_t =  monkey_information['monkey_t'][np.array(temp_dataframe3['point_index'])]
# temp_dataframe3['time'] = corresponding_t 
# temp_dataframe3['target_index'] = np.digitize(corresponding_t, ff_catched_T_sorted)
# temp_dataframe3 = temp_dataframe3[temp_dataframe3['target_index']<len(ff_catched_T_sorted)]
# cluster_dataframe_point = temp_dataframe3
# cluster_dataframe_trial = cluster_dataframe_point[['target_index', 'num_ff_at_point']].groupby('target_index', as_index=True).agg({'num_ff_at_point':['max', 'count']})
# cluster_dataframe_trial.columns = ["max_ff_in_cluster", "num_points_w_cluster"]
# cluster_exist_trials = cluster_dataframe_point.target_index.unique()

## Waste cluster

In [ ]:
# # If in the trial before the previous trial, there has been at least one cluster, but the monkey does not capture two ffs in a row
# cluster2 = cluster_exist_trials+1
# capture2 = np.where(n_ff_in_a_row == 1)[0]
# waste_cluster_trials = np.intersect1d(capture2, cluster2)

## ignore sudden flash

Not choose a closeby ff (closer than the target) that suddenly becomes visible

One change I made: Before, I selected "The indices and trials of ff_dataframe where the suddenly visible ff is the target." Now I select "The indices and trials of ff_dataframe where the suddenly visible ff is the target or next target"

In [ ]:
from math import pi
df_ffdistance = np.array(ff_dataframe['ff_distance'])
max_ff_index = max(np.array(ff_dataframe['ff_index']))



# These are the indices of points where a ff changes from being invisible to become visible
start_index1 = np.where(np.ediff1d(np.array(ff_dataframe['visible'])) == 1)[0]+1
# These are the indices of points where the ff_index has changed
start_index2 = np.where(np.ediff1d(np.array(ff_dataframe['ff_index']))!= 0)[0]+1
# Combine the two
start_index3 = np.concatenate((start_index1, start_index2))
start_index = np.unique(start_index3)

# Find the indices of ff_dataframe where the monkey encounters a closeby ff that suddenly becomes visible
suddent_flash_index = start_index[np.where(df_ffdistance[start_index] < 50)]

# The indices and trials of ff_dataframe where the suddenly visible ff is the target or next target
#sudden_flash_capture = suddent_flash_index[np.where(np.array(ff_dataframe['ff_index'])[suddent_flash_index] == np.array(ff_dataframe['target_index'])[suddent_flash_index])[0]]
condition = np.logical_or((np.array(ff_dataframe['ff_index'])[suddent_flash_index] == np.array(ff_dataframe['target_index'])[suddent_flash_index]), (np.array(ff_dataframe['ff_index'])[suddent_flash_index] == np.array(ff_dataframe['target_index']+1)[suddent_flash_index]))
sudden_flash_capture = suddent_flash_index[condition]
sudden_flash_capture_trials = np.array(ff_dataframe['target_index'])[sudden_flash_capture]
sudden_flash_capture_trials = np.unique(sudden_flash_capture_trials)

# The indices and trials of ff_dataframe where the suddenly visible ff is not the target
#sudden_flash_ignore = suddent_flash_index[np.where(np.array(ff_dataframe['ff_index'])[suddent_flash_index] != np.array(ff_dataframe['target_index'])[suddent_flash_index])[0]]
sudden_flash_ignore = suddent_flash_index[~condition]

# Find the distance to targets at these points
cum_x = np.array(ff_dataframe.monkey_x)[sudden_flash_ignore]
cum_y = np.array(ff_dataframe.monkey_y)[sudden_flash_ignore]
cum_target_indices = np.array(ff_dataframe.target_index)[sudden_flash_ignore]
cum_target_distances = LA.norm(np.stack([cum_x, cum_y], axis=1)-ff_real_position_sorted[cum_target_indices], axis=1)
cum_ff_distances = df_ffdistance[sudden_flash_ignore]
valid_indices = np.where(cum_target_distances > cum_ff_distances)
sudden_flash_ignore_trials_non_unique = cum_target_indices[valid_indices]
sudden_flash_ignore_trials = np.unique(sudden_flash_ignore_trials_non_unique)


## By points
# Find the points corresponding to such a sudden flash
sudden_flash_ignore_indices = np.array(ff_dataframe['point_index'])[sudden_flash_ignore[valid_indices]]
sudden_flash_ignore_ff = np.array(ff_dataframe['ff_index'])[sudden_flash_ignore[valid_indices]]
# Append each point into a list and the following n points so that the message can be visible for 2 seconds
sudden_flash_ignore_points = []
for i in sudden_flash_ignore_indices:
  sudden_flash_ignore_points = sudden_flash_ignore_points+list(range(i, i+121))

sudden_flash_ignore_time = monkey_information['monkey_t'][sudden_flash_ignore_indices]


In [ ]:
len(sudden_flash_ignore_trials)

## try a few times

In [ ]:
# Show the trials where the last cluster has more than 2 stops
try_a_few_times_trials = []
try_a_few_times_indices = []
try_a_few_times_info = [] # for each incident, I'll get [trial, min_stop_index_overall, max_stop_index_overall]
for i in range(catched_ff_num): 
  # Find clusters based on a distance of 50
  clusters = find_clusters(i,50)
  # if clusters is not empty:
  if len(clusters) > 0:
    num_for_last_cluster = clusters[-1]
    # If the last cluster has more than 2 stops
    if clusters.count(num_for_last_cluster) > 1:
      distinct_stops = num_of_stops(i)
      distinct_stops_indices = num_of_stops_indices(i)
      # If the last stop is close enough to the believed position of the target
      if LA.norm(distinct_stops[-1]-ff_believed_position_sorted[i]) < 50:
        try_a_few_times_trials.append(i)
# By points 
        min_index = distinct_stops_indices[-clusters.count(num_for_last_cluster)]
        max_index = distinct_stops_indices[-1]
        try_a_few_times_indices = try_a_few_times_indices + \
        list(range(min_index-20, min(max_index+20, max_point_index)))
        try_a_few_times_info.append([i, min_index, max_index])
try_a_few_times_info = np.array(try_a_few_times_info)

## give up after trying

In [ ]:
# By trials
# Show the trials where there at least one cluster has more than 2 stops, and this cluster is neither at the beginning nor at the end
give_up_after_trying_trials = []
for i in range(catched_ff_num):
  # Find clusters based on a distance of 50
  clusters = find_clusters(i,50)
  # if clusters is not empty:
  if len(clusters) > 0:
    clusters_counts = collections.Counter(clusters)
    distinct_stop_positions = num_of_stops(i)
    for k in range(1, clusters[-1]+1):  # for each cluster
      # if it has more than one element:
      if clusters_counts[k] > 1:
        # Get positions of these points
        stop_positions = [distinct_stop_positions[index] for index, value in enumerate(clusters) if value == k]
        # If the first stop is not close to beginning, and the last stop is not too close to the end:
        if LA.norm(stop_positions[0]-ff_believed_position_sorted[i-1]) > 50 and LA.norm(stop_positions[-1]-ff_believed_position_sorted[i]) > 50:
          give_up_after_trying_trials.append(i) 

In [ ]:
# By points

# Show the trials where there at least one cluster has more than 2 stops, and this cluster is neither at the beginning nor at the end

give_up_after_trying_indices = []
give_up_after_trying_info = [] # for each incident, I'll get [trial, min_stop_index_overall, max_stop_index_overall]
for i in range(catched_ff_num):
  # Find clusters based on a distance of 50
  clusters = find_clusters(i,50)
  # if clusters is not empty:
  if len(clusters) > 0:
    clusters_counts = collections.Counter(clusters)
    distinct_stop_positions = num_of_stops(i)
    distinct_stops_indices = num_of_stops_indices(i)
    for k in range(1, clusters[-1]+1):  # for each cluster
      # if it has more than one element:
      if clusters_counts[k] > 1:
        # Get positions of these points
        stop_positions = [distinct_stop_positions[index] for index, value in enumerate(clusters) if value == k]
        # If the first stop is not close to beginning, and the last stop is not too close to the end:
        if LA.norm(stop_positions[0]-ff_believed_position_sorted[i-1]) > 50 and LA.norm(stop_positions[-1]-ff_believed_position_sorted[i]) > 50:
          # Get indices of these points
          stop_positions_indices = [distinct_stops_indices[index] for index, value in enumerate(clusters) if value == k]
          give_up_after_trying_indices = give_up_after_trying_indices + list(range(min(stop_positions_indices)-20, max(stop_positions_indices)+21)) 
          give_up_after_trying_info.append([i, min(stop_positions_indices), max(stop_positions_indices)])
give_up_after_trying_info = np.array(give_up_after_trying_info)

# trials_char

Problem: for all the last_visible...I should consider the cluster that the target is in.

In [ ]:
relevant_df0 = ff_dataframe[ff_dataframe['visible'] == 1]
last_trial = catched_ff_num-200
trial_array = [i for i in range(1, last_trial+1)]
# Trial number is named after the index of the target. 
# Trial number starts at 1

t_array = ff_catched_T_sorted[1:last_trial+1] - ff_catched_T_sorted[:last_trial]
t_array = t_array.tolist()

## How long can the monkey remember a target?
## time elapses between the target last being visible and its capture
t_last_visible = []
d_last_visible = []
abs_angle_last_visible = []
for i in range(1, last_trial+1):
  relevant_df = relevant_df0[((relevant_df0['target_index']==i) & (relevant_df0['ffdistance2target']<25))| (relevant_df0['ff_index']==i)]
  if len(relevant_df) > 0:
    t_last_visible.append(ff_catched_T_sorted[i] - max(np.array(relevant_df.time)))
    d_last_visible.append(max(np.array(relevant_df.ff_distance)))
    abs_angle_last_visible.append(max(np.absolute(np.array(relevant_df.ff_angle_boundary))))
  else:
    t_last_visible.append(9999)
    d_last_visible.append(9999)
    abs_angle_last_visible.append(9999)


hit_boundary = []
for i in range(1, last_trial+1):
  duration = [ff_catched_T_sorted[i-1], ff_catched_T_sorted[i]]
  cum_indices = np.where((monkey_information['monkey_t'] >= duration[0]) & (monkey_information['monkey_t'] <= duration[1]))[0]
  if len(cum_indices) > 1:
    cum_t, cum_angle = monkey_information['monkey_t'][cum_indices],  monkey_information['monkey_angle'][cum_indices]
    cum_mx, cum_my = monkey_information['monkey_x'][cum_indices], monkey_information['monkey_y'][cum_indices] 
    if np.any(cum_mx[1:]-cum_mx[:-1] > 55) or np.any(cum_my[1:]-cum_my[:-1] > 55):
      hit_boundary.append(False)
    else:
      hit_boundary.append(False)
  else:
    hit_boundary.append(False)

# num_stops
num_stops_array = [len(num_of_stops(i)) for i in range(1, last_trial+1)]

num_stops_since_last_seen = [len(num_of_stops_since_target_last_seen(i)) for i in range(1, last_trial+1)]


num_stops_near_target = []
for i in range(1, last_trial+1):
  clusters = find_clusters(i,50)
  # for each trial, append the trial number and the number of stop in the last cluster, if the cluster is close enough to the target
  distinct_stops = num_of_stops(i)
  if len(distinct_stops) > 0:
    # If the last stop is close enough to the believed position of the target
    if LA.norm(distinct_stops[-1]-ff_believed_position_sorted[i]) < 50:
      num_stops_near_target.append(clusters.count(clusters[-1]))
    else: 
      num_stops_near_target.append(0)
  else: 
    num_stops_near_target.append(0)




# trials_char
trials_dict = {'trial':trial_array, 
               't':t_array,  
               't_last_visible':t_last_visible,
               'd_last_visible':d_last_visible,
               'abs_angle_last_visible': abs_angle_last_visible,
               'hit_boundary': hit_boundary,  
               'num_stops': num_stops_array, 
               'num_stops_near_target': num_stops_near_target                            
                      }
trials_char = pd.DataFrame(trials_dict)

trials_char['n_ff_in_a_row'] = n_ff_in_a_row[:len(trials_char)]


valid_trials = trials_char[(trials_char['t_last_visible']<50) & (trials_char['hit_boundary']==False)].reset_index()
median_values = valid_trials.median(axis=0)


len(valid_trials)

In [ ]:
data_folder_name = "LSTM_July25"

In [ ]:
# # # save
# data_folder_name = '0219'
# filepath = 'gdrive/MyDrive/fireflies_data/' + data_folder_name + '/trials_char.csv'
# os.makedirs('gdrive/MyDrive/fireflies_data/' + data_folder_name, exist_ok = True)
# trials_char.to_csv(filepath) 

In [ ]:
# # # save
# data_folder_name = 'LSTM_July_26_2'
# filepath = 'gdrive/MyDrive/fireflies_data/' + data_folder_name + '/trials_char.csv'
# os.makedirs('gdrive/MyDrive/fireflies_data/' + data_folder_name, exist_ok = True)
# trials_char.to_csv(filepath) 

In [ ]:
# retrieve
filepath = 'gdrive/MyDrive/fireflies_data/' + '0219' + '/trials_char.csv'
#filepath = 'gdrive/MyDrive/fireflies_data/' + data_folder_name + '/trials_char.csv'
trials_char_m = pd.read_csv(filepath)
valid_trials_m = trials_char_m[(trials_char_m['t_last_visible']<50) & (trials_char_m['hit_boundary']==False)].reset_index()
median_values_m = valid_trials_m.median(axis=0)

In [ ]:
filepath = 'gdrive/MyDrive/fireflies_data/' + 'LSTM_July_26_2' + '/trials_char.csv'
#filepath = 'gdrive/MyDrive/fireflies_data/' + data_folder_name + '/trials_char.csv'
trials_char_lstm = pd.read_csv(filepath)
valid_trials_lstm = trials_char_lstm[(trials_char_lstm['t_last_visible']<50) & (trials_char_lstm['hit_boundary']==False)].reset_index()
median_values_lstm = valid_trials_lstm.median(axis=0)

In [ ]:
trials_char

# Stats dict

## agent (new)

In [ ]:
max_time = 2000
catched_T_bounded = ff_catched_T_sorted[np.where(ff_catched_T_sorted < max_time)[0]]
max_trial = 650
num_stops_array = [len(num_of_stops(i)) for i in range(1, max_trial+1)]
stop_success_rate = max_trial/sum(num_stops_array)

temp_dataframe = ff_dataframe[(ff_dataframe['target_index']==ff_dataframe['ff_index']) & (ff_dataframe['visible'] == 1)]
trials_not_to_select = np.unique(np.array(temp_dataframe['target_index']))
all_trials = np.unique(np.array(ff_dataframe['target_index']))
visible_before_last_one_trials = np.setdiff1d(all_trials, trials_not_to_select)

n_trial = max_trial
n_points = np.where(monkey_information['monkey_t'] <= ff_catched_T_sorted[n_trial])[0][-1]

stats_dict = {
"Two in a row" : len(np.where(n_ff_in_a_row[:n_trial]>=2)[0])/(n_trial-2),
"Visible before last capture" : len(np.where(visible_before_last_one_trials < n_trial)[0])/(n_trial-2),
"Target disappears latest" : len(np.where(disappear_latest_trials < n_trial)[0])/(n_trial-1),
"Waste cluster around last target": len(np.where(waste_cluster_last_target_trials < n_trial)[0])/(n_trial-2),
"Ignore sudden flash": len(np.where(sudden_flash_ignore_trials < n_trial)[0])/(n_trial-1),
"Try a few times": len(np.where(np.array(try_a_few_times_trials) < n_trial)[0])/(n_trial-1),
"Give up after trying": len(np.where(np.array(give_up_after_trying_trials) < n_trial)[0])/(n_trial-1),
"ff capture rate": (len(catched_T_bounded)-1)/(catched_T_bounded[-1]-catched_T_bounded[0]),
"Stop success rate": max_trial/sum(num_stops_array),
}
stats_dict

In [ ]:
n_points = np.where(monkey_information['monkey_t'] <= ff_catched_T_sorted[n_trial])[0][-1]

stats_dict_all = {
"Two in a row" : len(np.where(n_ff_in_a_row[:n_trial]>=2)[0])/(n_trial-2),
"Visible before last capture" : len(np.where(visible_before_last_one_trials < n_trial)[0])/(n_trial-2),
"Target disappears latest" : len(np.where(disappear_latest_trials < n_trial)[0])/(n_trial-1),
"Waste cluster around last target": len(np.where(waste_cluster_last_target_trials < n_trial)[0])/(n_trial-2),
"Ignore sudden flash": len(np.where(sudden_flash_ignore_trials < n_trial)[0])/(n_trial-1),
"Try a few times": len(np.where(np.array(try_a_few_times_trials) < n_trial)[0])/(n_trial-1),
"Give up after trying": len(np.where(np.array(give_up_after_trying_trials) < n_trial)[0])/(n_trial-1),
"ff capture rate": (len(catched_T_bounded)-1)/(catched_T_bounded[-1]-catched_T_bounded[0]),
"Stop success rate": max_trial/sum(num_stops_array),
"Median time": median_values['t'],
"Median time target last seen": median_values['t_last_visible'],
#"Median distance target last seen": median_values['d_last_visible'],
"Median abs angle target last seen ": median_values['abs_angle_last_visible'],
"Median num stops": median_values['num_stops'],
"Median num stops near target": median_values['num_stops_near_target'],
}
stats_dict_all

In [ ]:
stats_dict_median = {"Median time": median_values['t'],
"Median time target last seen": median_values['t_last_visible'],
#"Median distance target last seen": median_values['d_last_visible'],
"Median abs angle target last seen ": median_values['abs_angle_last_visible'],
"Median num stops": median_values['num_stops'],
"Median num stops near target": median_values['num_stops_near_target'],
}
stats_dict_median

## agent (stored)

In [ ]:
# c10.2: dt = 0.25, noise = 1, memory = 3
stats_dict = {'Give up after trying': 0.20955315870570107,
 'Ignore sudden flash': 0.032357473035439135,
 'Stop success rate': 0.3551912568306011,
 'Target disappears latest': 0.5978428351309707,
 'Try a few times': 0.10015408320493066,
 'Two in a row': 0.17746913580246915,
 'Visible before last capture': 0.1111111111111111,
 'Waste cluster around last target': 0.08333333333333333,
 'ff capture rate': 0.3093119438526131}


stats_dict_median ={'Median abs angle target last seen ': 0.8535855351984136,
 'Median num stops': 3.0,
 'Median num stops near target': 1.0,
 'Median time': 2.25,
 'Median time target last seen': 0.25}

In [ ]:
stats_dict2 = {'Give up after trying': 0.21879815100154082,
 'Ignore sudden flash': 0.04776579352850539,
 'Stop success rate': 0.3504043126684636,
 'Target disappears latest': 0.6579352850539292,
 'Try a few times': 0.1386748844375963,
 'Two in a row': 0.1712962962962963,
 'Visible before last capture': 0.13117283950617284,
 'Waste cluster around last target': 0.09259259259259259,
 'ff capture rate': 0.34771700953336676}


stats_dict_median2 ={'Median abs angle target last seen ': 0.9426093204677228,
 'Median num stops': 3.0,
 'Median num stops near target': 1.0,
 'Median time': 2.0,
 'Median time target last seen': 0.25}

## monkey

In [ ]:
stats_dict_m = {'Give up after trying': 0.1263482280431433,
 'Ignore sudden flash': 0.1325115562403698,
 'Stop success rate': 0.33095723014256617,
 'Target disappears latest': 0.17873651771956856,
 'Try a few times': 0.3728813559322034,
 'Two in a row': 0.09876543209876543,
 'Visible before last capture': 0.13580246913580246,
 'Waste cluster around last target': 0.12037037037037036,
 'ff capture rate': 0.35397913078919646}

In [ ]:
stats_dict_all_m = {'Give up after trying': 0.1263482280431433,
 'Ignore sudden flash': 0.1325115562403698,
 'Median abs angle target last seen ': 0.6661714343096776,
 'Median num stops': 3.0,
 'Median num stops near target': 1.0,
 'Median time': 2.4152945000000727,
 'Median time target last seen': 0.7636549999999716,
 'Stop success rate': 0.2128356254092993,
 'Target disappears latest': 0.17873651771956856,
 'Try a few times': 0.3728813559322034,
 'Two in a row': 0.09876543209876543,
 'Visible before last capture': 0.13580246913580246,
 'Waste cluster around last target': 0.12037037037037036,
 'ff capture rate': 0.35397913078919646}

In [ ]:
stats_dict_median_m = {'Median abs angle target last seen ': 0.6661714343096776,
 'Median num stops': 3.0,
 'Median num stops near target': 1.0,
 'Median time': 2.4152945000000727,
 'Median time target last seen': 0.7636549999999716}

## big graphs

### compare 2

In [ ]:
#filename = f"noise = {noise}"

agent_stats_df = pd.DataFrame.from_dict(stats_dict, orient='index', columns=['Value'])
agent_stats_df['Player'] = "Agent"
monkey_stats_df = pd.DataFrame.from_dict(stats_dict_m, orient='index', columns=['Value'])
monkey_stats_df['Player'] = 'Monkey'
merged_stats_df = pd.concat([agent_stats_df, monkey_stats_df], axis=0)
merged_stats_df = merged_stats_df.reset_index()
merged_stats_df.rename(columns = {'index':'Category'}, inplace = True)


# set plot style: grey grid in the background:
#sns.set(style="darkgrid")

# load dataset
tips = sns.load_dataset("tips")

# Set the figure size
plt.figure(figsize=(8, 8))

# grouped barplot
ax = sns.barplot(x="Category", y="Value", hue="Player", data=merged_stats_df, ci=None);
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.tight_layout()

images_dir = '/content/gdrive/MyDrive/fireflies_analysis/modify_env'


# CHECK_FOLDER = os.path.isdir(images_dir)
# # If folder doesn't exist, then create it.
# if not CHECK_FOLDER:
#     os.makedirs(images_dir)
# #os.makedirs(images_dir)
# ax.set_title(title, fontsize = 22)
# #plt.savefig(f"{images_dir}/{filename}.png")

plt.show()
plt.close()

In [ ]:
#filename = f"noise = {noise}"

agent_stats_df = pd.DataFrame.from_dict(stats_dict_median, orient='index', columns=['Value'])
agent_stats_df['Player'] = "Agent"
monkey_stats_df = pd.DataFrame.from_dict(stats_dict_median_m, orient='index', columns=['Value'])
monkey_stats_df['Player'] = 'Monkey'
merged_stats_df = pd.concat([agent_stats_df, monkey_stats_df], axis=0)
merged_stats_df = merged_stats_df.reset_index()
merged_stats_df.rename(columns = {'index':'Category'}, inplace = True)


# set plot style: grey grid in the background:
sns.set(style="darkgrid")

# load dataset
tips = sns.load_dataset("tips")

# Set the figure size
plt.figure(figsize=(8, 8))

# grouped barplot
ax = sns.barplot(x="Category", y="Value", hue="Player", data=merged_stats_df, ci=None);
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.tight_layout()

images_dir = '/content/gdrive/MyDrive/fireflies_analysis/modify_env'


# CHECK_FOLDER = os.path.isdir(images_dir)
# # If folder doesn't exist, then create it.
# if not CHECK_FOLDER:
#     os.makedirs(images_dir)
# #os.makedirs(images_dir)
# ax.set_title(title, fontsize = 22)
# #plt.savefig(f"{images_dir}/{filename}.png")

plt.show()
plt.close()

### compare 3

In [ ]:
#filename = f"noise = {noise}"

agent_stats_df = pd.DataFrame.from_dict(stats_dict, orient='index', columns=['Value'])
agent_stats_df['Player'] = "Agent (noise factor = 1)"
agent_stats_df2 = pd.DataFrame.from_dict(stats_dict2, orient='index', columns=['Value'])
agent_stats_df2['Player'] = "Agent (noise factor = 2)"
monkey_stats_df = pd.DataFrame.from_dict(stats_dict_m, orient='index', columns=['Value'])
monkey_stats_df['Player'] = 'Monkey'
merged_stats_df = pd.concat([monkey_stats_df, agent_stats_df, agent_stats_df2], axis=0)
merged_stats_df = merged_stats_df.reset_index()
merged_stats_df.rename(columns = {'index':'Category'}, inplace = True)


# set plot style: grey grid in the background:
#sns.set(style="darkgrid")

# load dataset
tips = sns.load_dataset("tips")

# Set the figure size
plt.figure(figsize=(8, 8))

# grouped barplot
ax = sns.barplot(x="Category", y="Value", hue="Player", data=merged_stats_df, ci=None);
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.tight_layout()

images_dir = '/content/gdrive/MyDrive/fireflies_analysis/modify_env'


# CHECK_FOLDER = os.path.isdir(images_dir)
# # If folder doesn't exist, then create it.
# if not CHECK_FOLDER:
#     os.makedirs(images_dir)
# #os.makedirs(images_dir)
# ax.set_title(title, fontsize = 22)
# #plt.savefig(f"{images_dir}/{filename}.png")
plt.ylabel("Percentage of Trials")
plt.show()
plt.close()

In [ ]:
#filename = f"noise = {noise}"

agent_stats_df = pd.DataFrame.from_dict(stats_dict_median, orient='index', columns=['Value'])
agent_stats_df['Player'] = "Agent"
agent_stats_df2 = pd.DataFrame.from_dict(stats_dict_median2, orient='index', columns=['Value'])
agent_stats_df2['Player'] = "Agent2"
monkey_stats_df = pd.DataFrame.from_dict(stats_dict_median_m, orient='index', columns=['Value'])
monkey_stats_df['Player'] = 'Monkey'
merged_stats_df = pd.concat([agent_stats_df, agent_stats_df2, monkey_stats_df], axis=0)
merged_stats_df = merged_stats_df.reset_index()
merged_stats_df.rename(columns = {'index':'Category'}, inplace = True)


# set plot style: grey grid in the background:
sns.set(style="darkgrid")

# load dataset
tips = sns.load_dataset("tips")

# Set the figure size
plt.figure(figsize=(8, 8))

# grouped barplot
ax = sns.barplot(x="Category", y="Value", hue="Player", data=merged_stats_df, ci=None);
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.tight_layout()

images_dir = '/content/gdrive/MyDrive/fireflies_analysis/modify_env'


# CHECK_FOLDER = os.path.isdir(images_dir)
# # If folder doesn't exist, then create it.
# if not CHECK_FOLDER:
#     os.makedirs(images_dir)
# #os.makedirs(images_dir)
# ax.set_title(title, fontsize = 22)
# #plt.savefig(f"{images_dir}/{filename}.png")

plt.show()
plt.close()

## graphs by categories

### compare 2

compare current agent and monkey

In [ ]:
agent_stats_df = pd.DataFrame.from_dict(stats_dict, orient='index', columns=['Value'])
agent_stats_df['Player'] = "Agent"
monkey_stats_df = pd.DataFrame.from_dict(stats_dict_m, orient='index', columns=['Value'])
monkey_stats_df['Player'] = 'Monkey'
merged_stats_df = pd.concat([agent_stats_df, monkey_stats_df], axis=0)
merged_stats_df = merged_stats_df.reset_index()
merged_stats_df.rename(columns = {'index':'Category'}, inplace = True)
merged_stats_df['Value%'] = merged_stats_df['Value']*100


In [ ]:
merged_stats_df

In [ ]:
for cate in merged_stats_df.Category:
  print(cate)
  category_df = merged_stats_df[merged_stats_df['Category']==cate]

  # set plot style: grey grid in the background:
  sns.set(style="darkgrid")

  # load dataset
  tips = sns.load_dataset("tips")

  # Set the figure size
  plt.figure(figsize=(4, 8))

  # grouped barplot
  ##ax = sns.barplot(x="Player", y="Value", hue="Player", data=category_df, ci=None);
  ax = sns.barplot(x="Player", y="Value", data=category_df, ci=None);
  ##ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
  #ax.set_ylabel("Percentage of captured fireflies", fontsize = 22)
  ax.set_ylabel("")
  plt.xticks(fontsize= 22) 
  plt.yticks(fontsize= 15) 
  ##ax.set_xticklabels("")
  ax.set_xlabel("")
  ax.yaxis.set_major_formatter(mtick.PercentFormatter())
  plt.tight_layout()


  plt.show()
  plt.close()

### compare 3

In [ ]:
# agent from env d22
stats_dict_lstm={'Give up after trying': 0.040061633281972264,
 'Ignore sudden flash': 0.007704160246533128,
 'Stop success rate': 0.7072905331882481,
 'Target disappears latest': 0.2711864406779661,
 'Try a few times': 0.4869029275808937,
 'Two in a row': 0.2052469135802469,
 'Visible before last capture': 0.29012345679012347,
 'Waste cluster around last target': 0.013888888888888888,
 'ff capture rate': 0.48360450563204005}

stats_dict_median_lstm={'Median abs angle target last seen ': 0.3873275738322851,
 'Median num stops': 2.0,
 'Median num stops near target': 1.0,
 'Median time': 1.75,
 'Median time target last seen': 1.5}

In [ ]:
# agent from env c12
stats_dict_sb3 = {'Give up after trying': 0.07087827426810478,
 'Ignore sudden flash': 0.02157164869029276,
 'Stop success rate': 0.5191693290734825,
 'Target disappears latest': 0.7673343605546995,
 'Try a few times': 0.06933744221879815,
 'Two in a row': 0.19290123456790123,
 'Visible before last capture': 0.08024691358024691,
 'Waste cluster around last target': 0.08796296296296297,
 'ff capture rate': 0.4580152671755725}
 
stats_dict_median_sb3 = {'Median abs angle target last seen ': 0.9224812934369755,
 'Median num stops': 2.0,
 'Median num stops near target': 1.0,
 'Median time': 1.75,
 'Median time target last seen': 0.0}

In [ ]:
agent_stats_df = pd.DataFrame.from_dict(stats_dict_sb3, orient='index', columns=['Value'])
agent_stats_df['Player'] = "Agent"
monkey_stats_df = pd.DataFrame.from_dict(stats_dict_m, orient='index', columns=['Value'])
monkey_stats_df['Player'] = 'Monkey'
lstm_stats_df = pd.DataFrame.from_dict(stats_dict_lstm, orient='index', columns=['Value'])
lstm_stats_df['Player'] = 'Agent(LSTM)'
merged_stats_df = pd.concat([monkey_stats_df, agent_stats_df, lstm_stats_df], axis=0)
merged_stats_df = merged_stats_df.reset_index()
merged_stats_df.rename(columns = {'index':'Category'}, inplace = True)
merged_stats_df['Value%'] = merged_stats_df['Value']*100


In [ ]:
merged_stats_df

In [ ]:
for cate in merged_stats_df.Category:
  print(cate)
  category_df = merged_stats_df[merged_stats_df['Category']==cate]

  # set plot style: grey grid in the background:
  sns.set(style="darkgrid")

  # load dataset
  tips = sns.load_dataset("tips")

  # Set the figure size
  plt.figure(figsize=(6, 8))

  # grouped barplot
  ##ax = sns.barplot(x="Player", y="Value", hue="Player", data=category_df, ci=None);
  ax = sns.barplot(x="Player", y="Value%", data=category_df, ci=None);
  ##ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
  #ax.set_ylabel("Percentage of captured fireflies", fontsize = 22)
  ax.set_ylabel("")
  plt.xticks(fontsize= 22
             ) 
  plt.yticks(fontsize= 22) 
  ##ax.set_xticklabels("")
  ax.set_xlabel("")
  ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
  plt.tight_layout()


  plt.show()
  plt.close()

In [ ]:
merged_stats_df.Category

In [ ]:
from matplotlib.ticker import FormatStrFormatter

In [ ]:

cate = "ff capture rate"
category_df = merged_stats_df[merged_stats_df['Category']==cate]

# set plot style: grey grid in the background:
sns.set(style="darkgrid")

# load dataset
tips = sns.load_dataset("tips")

# Set the figure size
plt.figure(figsize=(6, 8))

# grouped barplot
##ax = sns.barplot(x="Player", y="Value", hue="Player", data=category_df, ci=None);
ax = sns.barplot(x="Player", y="Value", data=category_df, ci=None);
##ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
#ax.set_ylabel("Percentage of captured fireflies", fontsize = 22)
ax.set_ylabel("")
plt.xticks(fontsize= 22
            ) 
plt.yticks(fontsize= 22) 

current_values = plt.gca().get_yticks()
# using format string '{:.0f}' here but you can choose others
plt.gca().set_yticklabels(['{:.1f}/s'.format(x) for x in current_values])

##ax.set_xticklabels("")
ax.set_xlabel("")
#ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
plt.tight_layout()


plt.show()
plt.close()

# histograms

## Overall

In [ ]:
valid_trials

In [ ]:
variable_of_interest_list = ['t', 't_last_visible', 'd_last_visible',
    'abs_angle_last_visible', 'num_stops','num_stops_near_target']
subplot_list = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2)]
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for i in range(6):
  variable_of_interest = variable_of_interest_list[i]
  sns.histplot(ax=axes[subplot_list[i]], data = valid_trials[variable_of_interest], kde = True, alpha = 0.4, color = "blue")
  sns.histplot(ax=axes[subplot_list[i]], data = valid_trials_m[variable_of_interest], kde = True, alpha = 0.4, color = "orange")
  axes[subplot_list[i]].legend(labels=["Agent","Monkey"])

plt.show()

Not in subplot

In [ ]:
variable_of_interest_list = ['t', 't_last_visible', 'd_last_visible',
    'abs_angle_last_visible', 'num_stops','num_stops_near_target']

for i in range(6):
  variable_of_interest = variable_of_interest_list[i]
  fig, axes = plt.subplots(figsize=(6, 5))
  sns.histplot(data = valid_trials[variable_of_interest], kde = True, alpha = 0.4, color = "blue")
  sns.histplot(data = valid_trials_m[variable_of_interest], kde = True, alpha = 0.4, color = "orange")
  axes.legend(labels=["Agent","Monkey"])
  axes.set_ylabel("")
  axes.set_yticklabels("")
plt.show()

## By category

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
mstyle.use('seaborn-white')

In [ ]:
valid_trials = valid_trials_lstm

In [ ]:
variable_of_interest = "t_last_visible"
fig, axes = plt.subplots(figsize=(4, 5))
sns.histplot(data = valid_trials[variable_of_interest], kde = True, alpha = 0.4, color = "green", binwidth = 0.25)
sns.histplot(data = valid_trials_m[variable_of_interest], kde = True, alpha = 0.4, color = "blue")
axes.legend(labels=["Agent(LSTM)","Monkey"], fontsize = 14)
#axes.set_ylabel("")
#axes.set_yticklabels("")
plt.xticks(fontsize = 18)
plt.title("Time Since Target Last Visible", fontsize=18)
plt.xlabel("Time (s)", fontsize = 18)
plt.xlim([0, 6])

#axes.xaxis.set_major_locator(ticker.NullLocator())
axes.yaxis.set_major_locator(ticker.NullLocator())
plt.show()

In [ ]:
variable_of_interest = "num_stops_near_target"
fig, axes = plt.subplots(figsize=(4, 5))
sns.histplot(data = valid_trials[variable_of_interest], kde = False, alpha = 0.4, color = "green", binwidth=0.5, stat="probability")
sns.histplot(data = valid_trials_m[variable_of_interest], kde = False, alpha = 0.4, color = "blue", binwidth=0.5, stat="probability")
axes.legend(labels=["Agent(LSTM)","Monkey"], fontsize = 14)
sns.kdeplot(data = valid_trials, x = variable_of_interest, bw=1, color = "green")
sns.kdeplot(data = valid_trials_m, x = variable_of_interest, bw=1, color = "blue")
axes.set_ylabel("Probability",fontsize=15)
#axes.set_yticklabels("")
plt.xlim(0,5)
plt.title("Number of Stops Near Targets", fontsize=17)
plt.xlabel("Number of Stops", fontsize=15)
axes.xaxis.set_major_locator(MaxNLocator(integer=True))
# axes.xaxis.set_major_locator(ticker.NullLocator())
#axes.yaxis.set_major_locator(ticker.NullLocator())
plt.show()

In [ ]:
variable_of_interest = "n_ff_in_a_row"
fig, axes = plt.subplots(figsize=(4, 5))
sns.histplot(data = valid_trials[variable_of_interest], kde = False, alpha = 0.4, color = "green", binwidth=0.5, stat="probability")
sns.histplot(data = valid_trials_m[variable_of_interest], kde = False, alpha = 0.4, color = "blue", binwidth=0.5, stat="probability")
axes.legend(labels=["Agent(LSTM)","Monkey"], fontsize = 14)
sns.kdeplot(data = valid_trials, x = variable_of_interest, bw=1, color = "green")
sns.kdeplot(data = valid_trials_m, x = variable_of_interest, bw=1, color = "blue")
axes.set_ylabel("Probability",fontsize=15)
#axes.set_yticklabels("")
plt.xlim(1,5)
plt.title("Number of fireflies caught in a cluster", fontsize=17)
plt.xlabel("Number of Fireflies", fontsize=15)
axes.xaxis.set_major_locator(MaxNLocator(integer=True))
# axes.xaxis.set_major_locator(ticker.NullLocator())
#axes.yaxis.set_major_locator(ticker.NullLocator())
plt.show()

In [ ]:
variable_of_interest = "d_last_visible"
fig, axes = plt.subplots(figsize=(4, 5))
sns.histplot(data = valid_trials[variable_of_interest], kde = True, alpha = 0.3, binwidth=40, binrange=(0,400), color = "green", stat="probability")
sns.histplot(data = valid_trials_m[variable_of_interest], kde = True, alpha = 0.3, binwidth=40, binrange=(0,400), color = "blue", stat="probability")
axes.legend(labels=["Agent(LSTM)","Monkey"], fontsize = 14)
# sns.kdeplot(data = valid_trials, x = variable_of_interest, color = "green", bw=1)
# sns.kdeplot(data = valid_trials_m, x = variable_of_interest, color = "blue", bw=1)
axes.set_ylabel("Probability",fontsize=15)
#axes.set_yticklabels("")
plt.xlim(0, 400)
plt.title("Distance of Target Last Visible", fontsize=17)
plt.xlabel("Distance (cm)", fontsize=15)
axes.xaxis.set_major_locator(MaxNLocator(integer=True))
# axes.xaxis.set_major_locator(ticker.NullLocator())
#axes.yaxis.set_major_locator(ticker.NullLocator())


xticklabels=axes.get_xticks().tolist()
xticklabels = [str(int(label)) for label in xticklabels]
xticklabels[-1]='400+'
axes.set_xticklabels(xticklabels)
plt.show()

In [ ]:
variable_of_interest = "abs_angle_last_visible"
fig, axes = plt.subplots(figsize=(4, 5))
sns.histplot(data = valid_trials[variable_of_interest], kde = True, binwidth=0.1, binrange=(0,0.8), alpha = 0.3, color = "green", stat="probability")
sns.histplot(data = valid_trials_m[variable_of_interest], kde = True, binwidth=0.1, binrange=(0,0.8), alpha = 0.3, color = "blue", stat="probability")
axes.legend(labels=["Agent(LSTM)","Monkey"], fontsize = 14)
# sns.kdeplot(data = valid_trials, x = variable_of_interest, color = "green", bw=1)
# sns.kdeplot(data = valid_trials_m, x = variable_of_interest, color = "blue", bw=1)
axes.set_ylabel("Probability",fontsize=15)
#axes.set_yticklabels("")
plt.title("Abs Angle of Target Last Visible", fontsize=17)
plt.xlabel("Angle (rad)", fontsize=15)
axes.xaxis.set_major_locator(MaxNLocator(integer=True))
# axes.xaxis.set_major_locator(ticker.NullLocator())
#axes.yaxis.set_major_locator(ticker.NullLocator())



plt.show()

In [ ]:
variable_of_interest = "t"
fig, axes = plt.subplots(figsize=(4, 5))
sns.histplot(data = valid_trials[variable_of_interest], kde = True, binwidth=1, binrange=(0,20), alpha = 0.3, color = "green", stat="probability")
sns.histplot(data = valid_trials_m[variable_of_interest], kde = True, binwidth=1, binrange=(0,20), alpha = 0.3, color = "blue", stat="probability")
axes.legend(labels=["Agent(LSTM)","Monkey"], fontsize = 14)
# sns.kdeplot(data = valid_trials, x = variable_of_interest, color = "green", bw=1)
# sns.kdeplot(data = valid_trials_m, x = variable_of_interest, color = "blue", bw=1)
axes.set_ylabel("Probability",fontsize=15)
#axes.set_yticklabels("")
plt.title("Trial Duration", fontsize=17)
plt.xlabel("Duration (s)", fontsize=15)
axes.xaxis.set_major_locator(MaxNLocator(integer=True))
# axes.xaxis.set_major_locator(ticker.NullLocator())
#axes.yaxis.set_major_locator(ticker.NullLocator())



plt.show()

In [ ]:
valid_trials["num_stops"].loc[np.where(np.array(valid_trials["num_stops"]==0))] = 1
valid_trials_m["num_stops"].loc[np.where(np.array(valid_trials_m["num_stops"]==0))] = 1

In [ ]:
variable_of_interest = "num_stops"
fig, axes = plt.subplots(figsize=(4, 5))
sns.histplot(data = valid_trials[variable_of_interest], binwidth=1, binrange=(1,12), alpha = 0.3, color = "green", stat="probability")
sns.histplot(data = valid_trials_m[variable_of_interest], binwidth=1, alpha = 0.3, binrange=(1,12), color = "blue", stat="probability")
axes.legend(labels=["Agent(LSTM)","Monkey"], fontsize = 14)
sns.kdeplot(data = valid_trials, x = variable_of_interest,  bw=2, color = "green")
sns.kdeplot(data = valid_trials_m, x = variable_of_interest, bw=2, color = "blue")
axes.set_ylabel("Probability",fontsize=15)
plt.xlim(0,12)
#axes.set_yticklabels("")
plt.title("Number of Stops During Trials", fontsize=17)
plt.xlabel("Number of Stops", fontsize=15)
axes.xaxis.set_major_locator(MaxNLocator(integer=True))
# axes.xaxis.set_major_locator(ticker.NullLocator())
#axes.yaxis.set_major_locator(ticker.NullLocator())



plt.show()

# Monkey animation

## Animation

In [ ]:
# New version
currentTrial = 500
num_trials = 7
print(f"Trials {currentTrial}-{currentTrial+num_trials-1}")
ff_position_during_this_trial = np.array([ff_real_position_sorted[i] for i,value in 
                                enumerate(ff_life_sorted) if (value[-1]>=ff_catched_T_sorted[currentTrial-1]) and (value[0]<=ff_catched_T_sorted[currentTrial+num_trials-1])])
duration = [ff_catched_T_sorted[currentTrial-1], ff_catched_T_sorted[currentTrial+num_trials-1]]
index_temp = np.where((monkey_information['monkey_t'] > ff_catched_T_sorted[currentTrial-1]) & 
                        (monkey_information['monkey_t'] <= ff_catched_T_sorted[currentTrial+num_trials-1]))
if len(index_temp) > 0:
  flash_index = flashing_ff(ff_flash_sorted, duration)
  flash_fireflies = ff_real_position_sorted[flash_index]


  cum_t = monkey_information['monkey_t'][index_temp]
  cum_mx = monkey_information['monkey_x'][index_temp]
  cum_my = monkey_information['monkey_y'][index_temp]
  cum_believedffp = ff_believed_position_sorted[currentTrial: currentTrial+num_trials]
  cum_angle = monkey_information['monkey_angle'][index_temp]

# figure
global fig; fig = plt.figure(dpi=100)
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.size'] = 15
fig, ax = plt.subplots()

# Animation
filename = f"Trial {currentTrial}"
k = 5
anim_index = index_temp[0][0:-1:k].copy()
anim_t = cum_t[0:-1:k].copy()
anim_mx = cum_mx[0:-1:k].copy()
anim_my = cum_my[0:-1:k].copy()
anim_angle = cum_angle[0:-1:k].copy()
flash_on_ff_dict = flash_on_ff(anim_t, currentTrial, num_trials, ff_flash_sorted)
believed_ff_dict = believed_ff(anim_t, currentTrial, num_trials, ff_believed_position_sorted, ff_catched_T_sorted)
circle_theta = np.arange(0, 2*pi, 0.01)
circle_x = np.cos(circle_theta)*1000
circle_y = np.sin(circle_theta)*1000
ff_position_during_this_trial = np.array([ff_real_position_sorted[i] for i,value in 
                            enumerate(ff_life_sorted) if (value[-1]>=ff_catched_T_sorted[currentTrial-1]) and (value[0]<ff_catched_T_sorted[currentTrial+num_trials-1])])

fig, ax = plt.subplots()
ax.scatter(ff_position_during_this_trial[:,0] , ff_position_during_this_trial[:,1] , alpha=0.9, c="gray", s=20)

# boundary
margin = 400
xmin, xmax = np.min(anim_mx), np.max(anim_mx)
ymin, ymax = np.min(cum_my), np.max(cum_my)
ax.set_xlim((xmin-margin, xmax+margin))
ax.set_ylim((ymin-margin, ymax+margin))
ax.set_aspect('equal')



def animate(i):
    ax.cla()
    ax.axis('off')
    ax.set_xlim((xmin-margin, xmax+margin))
    ax.set_ylim((ymin-margin, ymax+margin))
    ax.set_aspect('equal')
    index = anim_index[i]
    time = anim_t[i]
    trial_num = np.where(ff_catched_T_sorted > time)[0][0]
    flashing_on_ff = ff_real_position_sorted[flash_on_ff_dict[anim_t[i]]]
    relevant_ff = ff_dataframe[['point_index', 'visible', 'ff_index', 'ff_x', 'ff_y']]
    relevant_ff = relevant_ff[relevant_ff['point_index']==index]
    in_memory_ffs = relevant_ff[relevant_ff['visible']==0]
    visible_ffs = relevant_ff[relevant_ff['visible']==1]
    ax.plot(circle_x, circle_y)

    ax.scatter(ff_position_during_this_trial[:,0] , ff_position_during_this_trial[:,1] , alpha=0.7, c="gray", s=20)
    #ax.scatter(ff_real_position_sorted[trial_num][0], ff_real_position_sorted[trial_num][1], marker='*', c='blue', s = 130, alpha = 0.5)
    ax.scatter(flashing_on_ff[:,0],flashing_on_ff[:,1] , alpha=1, c="red", s=30)
    #ax.scatter(in_memory_ffs.ff_x , in_memory_ffs.ff_y , alpha=1, c="green", s=30)
    ax.scatter(anim_mx[:i+1],anim_my[:i+1], s=15, c='royalblue')
    

    # for j in range(len(in_memory_ffs)):
    #   circle = plt.Circle((in_memory_ffs.ff_x.iloc[j], in_memory_ffs.ff_y.iloc[j]), 25, facecolor='grey', edgecolor='orange', alpha=0.3, zorder=1)
    #   ax.add_patch(circle)

    for k in range(len(visible_ffs)):
      circle = plt.Circle((visible_ffs.ff_x.iloc[k], visible_ffs.ff_y.iloc[k]), 25, facecolor='yellow', edgecolor='gray', alpha=0.5, zorder=1)
      ax.add_patch(circle)
      
    if len(believed_ff_dict[i]) > 0:
      for z in believed_ff_dict[i]:
        ax.scatter(z[0],z[1], color="purple", s=30)

    ax.plot(np.array([anim_mx[i], anim_mx[i] + 30*np.cos(anim_angle[i]+2*pi/9)]), np.array([anim_my[i] , anim_my[i] + 30*np.sin(anim_angle[i]+2*pi/9)]), linewidth = 2)
    ax.plot(np.array([anim_mx[i], anim_mx[i] + 30*np.cos(anim_angle[i]-2*pi/9)]), np.array([anim_my[i] , anim_my[i] + 30*np.sin(anim_angle[i]-2*pi/9)]), linewidth = 2)
  


num_frame = anim_t.size
anim = animation.FuncAnimation(fig, animate, 
                frames=num_frame, interval=100, repeat=True) 
filename = "0219_" + str(currentTrial) + "to" + str(currentTrial+num_trials) 
gif_dir = '/content/gdrive/MyDrive/fireflies_anim/monkey'
os.makedirs(gif_dir, exist_ok=True)
writervideo = animation.FFMpegWriter(fps=4*3) # use fps = 208.3 if u want the time to be realistic
anim.save(f"{gif_dir}/{filename}.mp4", writer=writervideo)
#anim.save(f"{gif_dir}/{filename}.gif", writer='pillow', fps=60)
anim
HTML(anim.to_html5_video())

## Animation with anotation

### Organize data

In [ ]:
len(give_up_after_trying_dummy)

In [ ]:
give_up_after_trying_indices[-1]

In [ ]:
max(sudden_flash_ignore_points)

In [ ]:
len(sudden_flash_ignore_dummy)

In [ ]:
sudden_flash_ignore_points = np.array(sudden_flash_ignore_points)[np.where(sudden_flash_ignore_points <= max_point_index)]


give_up_after_trying_dummy = np.zeros(max_point_index+1, dtype=int)
give_up_after_trying_dummy[give_up_after_trying_indices] = 1
try_a_few_times_dummy = np.zeros(max_point_index+1, dtype=int)
try_a_few_times_dummy[try_a_few_times_indices] = 1
sudden_flash_ignore_dummy = np.zeros(max_point_index+1, dtype=int)
sudden_flash_ignore_dummy[sudden_flash_ignore_points] = 1



zero_array = np.zeros(catched_ff_num, dtype=int)

multiple_in_a_row = np.where(n_ff_in_a_row>=2)[0]
multiple_in_a_row_all = np.union1d(multiple_in_a_row, multiple_in_a_row-1)
multiple_in_a_row2 = zero_array.copy()
multiple_in_a_row_all2 = zero_array.copy()
multiple_in_a_row2[multiple_in_a_row] = 1
multiple_in_a_row_all2[multiple_in_a_row_all] = 1

two_in_a_row2 = zero_array.copy()
two_in_a_row2[two_in_a_row] = 1 

three_in_a_row2 = zero_array.copy()
three_in_a_row= np.where(n_ff_in_a_row==3)[0]
three_in_a_row2[three_in_a_row] = 1

four_in_a_row2 = zero_array.copy()
four_in_a_row= np.where(n_ff_in_a_row==4)[0]
four_in_a_row2[four_in_a_row] = 1

none_in_a_row = np.where(n_ff_in_a_row<2)[0]
none_in_a_row2 = zero_array.copy()
none_in_a_row2[none_in_a_row] = 1

visible_before_last_one2 = zero_array.copy()
visible_before_last_one2[visible_before_last_one_trials] = 1

disappear_latest2 = zero_array.copy()
disappear_latest2[disappear_latest_trials] = 1 

sudden_flash_ignore2 = zero_array.copy()
sudden_flash_ignore2[sudden_flash_ignore_trials] = 1

try_a_few_times2 = zero_array.copy()
try_a_few_times2[try_a_few_times_trials] = 1

give_up_after_trying2 = zero_array.copy()
give_up_after_trying2[give_up_after_trying_trials] = 1

# cluster_exists2 = zero_array.copy()
# cluster_exists2[cluster_exist_trials] = 1 
  

# cluster_around_target2 = zero_array.copy()
# cluster_around_target2[ffs_around_target_trials] = 1 

# waste_cluster2 = zero_array.copy()
# waste_cluster2[waste_cluster_trials] = 1

waste_cluster_last_target2 = zero_array.copy()
waste_cluster_last_target2[waste_cluster_last_target_trials] = 1

# cluster_max_ff2 = zero_array.copy()
# cluster_max_ff2[np.array(cluster_dataframe_trial.index)] = np.array(cluster_dataframe_trial.max_ff_in_cluster)

# time_interval = monkey_information['monkey_t'][100] - monkey_information['monkey_t'][99]
# cluster_exists_duration2 = np.zeros(catched_ff_num)
# cluster_exists_duration2[np.array(cluster_dataframe_trial.index)] = np.array(cluster_dataframe_trial.num_points_w_cluster)*time_interval

ffs_around_target2 = ffs_around_target


anno_trial_dict = {
# bool
'two_in_a_row': two_in_a_row2,
'three_in_a_row': three_in_a_row2,
'four_in_a_row': four_in_a_row2,
'none_in_a_row': none_in_a_row2,
'multiple_in_a_row': multiple_in_a_row2,
'multiple_in_a_row_all': multiple_in_a_row_all2,
'visible_before_last_one': visible_before_last_one2,
'disappear_latest': disappear_latest2,
'sudden_flash_ignore': sudden_flash_ignore2,
'try_a_few_times': try_a_few_times2,
'give_up_after_trying': give_up_after_trying2,
# 'cluster_exists': cluster_exists2,
# 'cluster_around_target': cluster_around_target2,
# 'waste_cluster': waste_cluster2,
'waste_cluster_last_target': waste_cluster_last_target2,
## num
# 'cluster_max_ff' : cluster_max_ff2,  # The maximum number of ffs in a cluster during the trial
# 'cluster_exists_duration': cluster_exists_duration2,
# 'ffs_around_target': ffs_around_target2
                                  }
anno_trial = pd.DataFrame(anno_trial_dict) 

anno_trial[:3]


### Animation

In [ ]:
# New version
currentTrial = 360
num_trials = 10
print(f"Trials {currentTrial}-{currentTrial+num_trials-1}")
ff_position_during_this_trial = np.array([ff_real_position_sorted[i] for i,value in 
                                enumerate(ff_life_sorted) if (value[-1]>=ff_catched_T_sorted[currentTrial-1]) and (value[0]<=ff_catched_T_sorted[currentTrial+num_trials-1])])
duration = [ff_catched_T_sorted[currentTrial-1], ff_catched_T_sorted[currentTrial+num_trials-1]]
index_temp = np.where((monkey_information['monkey_t'] > ff_catched_T_sorted[currentTrial-1]) & 
                        (monkey_information['monkey_t'] <= ff_catched_T_sorted[currentTrial+num_trials-1]))
if len(index_temp) > 0:
  flash_index = flashing_ff(ff_flash_sorted, duration)
  flash_fireflies = ff_real_position_sorted[flash_index]


  cum_t = monkey_information['monkey_t'][index_temp]
  cum_mx = monkey_information['monkey_x'][index_temp]
  cum_my = monkey_information['monkey_y'][index_temp]
  cum_believedffp = ff_believed_position_sorted[currentTrial: currentTrial+num_trials]
  cum_angle = monkey_information['monkey_angle'][index_temp]
# figure
global fig; fig = plt.figure(dpi=100)
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.size'] = 15
fig, ax = plt.subplots()

# Animation
filename = f"Trial {currentTrial}"
k = 15
anim_index = index_temp[0][0:-1:k].copy()
anim_t = cum_t[0:-1:k].copy()
anim_mx = cum_mx[0:-1:k].copy()
anim_my = cum_my[0:-1:k].copy()
anim_angle = cum_angle[0:-1:k].copy()
flash_on_ff_dict = flash_on_ff(anim_t, currentTrial, num_trials, ff_flash_sorted)
believed_ff_dict = believed_ff(anim_t, currentTrial, num_trials, ff_believed_position_sorted, ff_catched_T_sorted)
circle_theta = np.arange(0, 2*pi, 0.01)
circle_x = np.cos(circle_theta)*1000
circle_y = np.sin(circle_theta)*1000
ff_position_during_this_trial = np.array([ff_real_position_sorted[i] for i,value in 
                            enumerate(ff_life_sorted) if (value[-1]>=ff_catched_T_sorted[currentTrial-1]) and (value[0]<ff_catched_T_sorted[currentTrial+num_trials-1])])

fig, ax = plt.subplots()
ax.scatter(ff_position_during_this_trial[:,0] , ff_position_during_this_trial[:,1] , alpha=0.9, c="gray", s=20)

# boundary
margin = 400
xmin, xmax = np.min(anim_mx), np.max(anim_mx)
ymin, ymax = np.min(cum_my), np.max(cum_my)
ax.set_xlim((xmin-margin, xmax+margin))
ax.set_ylim((ymin-margin, ymax+margin))
ax.set_aspect('equal')



def animate(i):
    ax.cla()
    ax.axis('off')
    ax.set_xlim((xmin-margin, xmax+margin))
    ax.set_ylim((ymin-margin, ymax+margin))
    ax.set_aspect('equal')
    index = anim_index[i]
    time = anim_t[i]
    trial_num = np.where(ff_catched_T_sorted > time)[0][0]
    flashing_on_ff = ff_real_position_sorted[flash_on_ff_dict[anim_t[i]]]
    relevant_ff = ff_dataframe[['point_index', 'visible', 'ff_index', 'ff_x', 'ff_y']]
    relevant_ff = relevant_ff[relevant_ff['point_index']==index]
    in_memory_ffs = relevant_ff[relevant_ff['visible']==0]
    visible_ffs = relevant_ff[relevant_ff['visible']==1]
    ax.plot(circle_x, circle_y)

    ax.scatter(ff_position_during_this_trial[:,0] , ff_position_during_this_trial[:,1] , alpha=0.7, c="gray", s=20)
    #ax.scatter(ff_real_position_sorted[trial_num][0], ff_real_position_sorted[trial_num][1], marker='*', c='purple', s = 130, alpha = 0.5)
    ax.scatter(flashing_on_ff[:,0],flashing_on_ff[:,1] , alpha=1, c="red", s=30)
    #ax.scatter(in_memory_ffs.ff_x , in_memory_ffs.ff_y , alpha=1, c="green", s=30)
    ax.scatter(anim_mx[:i+1],anim_my[:i+1], s=15, c='royalblue')
    

    # for j in range(len(in_memory_ffs)):
    #   circle = plt.Circle((in_memory_ffs.ff_x.iloc[j], in_memory_ffs.ff_y.iloc[j]), 25, facecolor='grey', edgecolor='orange', alpha=0.3, zorder=1)
    #   ax.add_patch(circle)

    for k in range(len(visible_ffs)):
      circle = plt.Circle((visible_ffs.ff_x.iloc[k], visible_ffs.ff_y.iloc[k]), 25, facecolor='yellow', edgecolor='gray', alpha=0.5, zorder=1)
      ax.add_patch(circle)
      
    if len(believed_ff_dict[i]) > 0:
      for z in believed_ff_dict[i]:
        ax.scatter(z[0],z[1], color="purple", s=30)
    # Annotation
    annotation = ""
    # If the monkey has captured more than one 1 ff in a cluster
    if n_ff_in_a_row[trial_num] > 1:
      annotation = annotation + f"Captured {n_ff_in_a_row[trial_num]} ffs in a cluster\n"
    # If the target stops being on before the monkey captures the previous firefly
    if visible_before_last_one2[trial_num] == 1:
      annotation = annotation + "Target visible before last captre\n"
    # If the target disappears the latest among visible ffs
    if disappear_latest2[trial_num] == 1:   
      annotation = annotation + "Target disappears latest\n"
    # If the monkey ignored a closeby ff that suddenly became visible
    if sudden_flash_ignore_dummy[index] > 0:
      annotation = annotation + "Ignored sudden flash\n"
    # If the monkey uses a few tries to capture a firefly
    if try_a_few_times_dummy[index] > 0:
      annotation = annotation + "Try a few times to catch ff\n"
    # If during the trial, the monkey fails to capture a firefly with a few tries and moves on to capture another one 
    if give_up_after_trying_dummy[index] > 0:
      annotation = annotation + "Give up after trying\n"
    # If there is at least one cluster of visible ffs
    # if all_point_vs_num_ff[index] > 0: 
    #   annotation = annotation + "Clusters: max" + str(all_point_vs_num_ff[index]) + " ffs" 
      #annotation = annotation + f"Clusters: {point_vs_cluster[index]}\n"
    # # Check whether the target is the closest ff among all that are visible or in memory (i.e. visible within 1.6s)
    # if target_closest[index] == 2:
    #   annotation = annotation +  "Target is the closest\n"
    # elif target_closest[index] ==1:
    #   annotation = annotation +  "Target is not the closest\n"
    # # Check whether the target has the smallest angle among all ffs that are visible or in memory (i.e. visible within 1.6s)
    # if target_angle_smallest[index] == 2:
    #   annotation = annotation + "Target has the smallest angle"
    # elif target_angle_smallest[index] ==1:
    #   annotation = annotation +  "Target does not have the smallest angle"
    # else:
    #   annotation = annotation +  "Target not visible in 1.6s"
    ax.text(0.5, 1.04, annotation, horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=12, color="black", bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    ax.plot(np.array([anim_mx[i], anim_mx[i] + 30*np.cos(anim_angle[i]+2*pi/9)]), np.array([anim_my[i] , anim_my[i] + 30*np.sin(anim_angle[i]+2*pi/9)]), linewidth = 2)
    ax.plot(np.array([anim_mx[i], anim_mx[i] + 30*np.cos(anim_angle[i]-2*pi/9)]), np.array([anim_my[i] , anim_my[i] + 30*np.sin(anim_angle[i]-2*pi/9)]), linewidth = 2)
  

num_frame = anim_t.size



anim = animation.FuncAnimation(fig, animate, 
                frames=num_frame, interval=100, repeat=True) 
filename = "0219_" + str(currentTrial) + "to" + str(currentTrial+num_trials) 
gif_dir = '/content/gdrive/MyDrive/fireflies_anim/monkey_annotated'
os.makedirs(gif_dir, exist_ok=True)
writervideo = animation.FFMpegWriter(fps=4*3) # use fps = 208.3 if u want the time to be realistic
anim.save(f"{gif_dir}/{filename}.mp4", writer=writervideo)
#anim.save(f"{gif_dir}/{filename}.gif", writer='pillow', fps=60)
HTML(anim.to_html5_video())

# Other graphs

In [ ]:
import matplotlib.style as mstyle

In [ ]:
mstyle.available

In [ ]:
category = two_in_a_row_non_simul
images_dir = '/content/gdrive/MyDrive/fireflies_analysis/algo_selected_summer_2022_updated/two_in_a_row_non_simul'
num_trials = 2
Show_Colorbar = True
trail_color = "orange"
trial_to_show_cluster = None # None, 0, or -1

if len(category) > 0:
  with initiate_plot(10,10,100):
    #for currentTrial in category[:trial_total_num]:
    for currentTrial in [50]:

        duration = [ff_catched_T_sorted[currentTrial-num_trials], ff_catched_T_sorted[currentTrial]]
        ff_position_during_this_trial = np.array([ff_real_position_sorted[i] for i,value in 
                                          enumerate(ff_life_sorted) if (value[-1]>=duration[0]) and (value[0]<duration[1])])

        cum_indices = np.where((monkey_information['monkey_t'] > duration[0]) & (monkey_information['monkey_t'] <= duration[1]))[0]
        cum_t, cum_angle = monkey_information['monkey_t'][cum_indices], monkey_information['monkey_angle'][cum_indices]
        cum_mx, cum_my = monkey_information['monkey_x'][cum_indices], monkey_information['monkey_y'][cum_indices]
        cum_speed, cum_speeddummy = monkey_information['monkey_speed'][cum_indices], monkey_information['monkey_speeddummy'][cum_indices]
        cum_speed, cum_speeddummy = monkey_information['monkey_speed'][cum_indices], monkey_information['monkey_speeddummy'][cum_indices]

        # Eliminate trials where the monkey has gone across the edge or if the trial is too short
        cum_r = LA.norm(np.stack((cum_mx, cum_my)), axis = 0)
        if (np.any(cum_r > 949)) or (len(cum_t) < 5):
          continue

        # Find the angle from the origin to the target
        theta = pi/2-np.arctan2(cum_my[-1]-cum_my[0], cum_mx[-1]-cum_mx[0])     
        c, s = np.cos(theta), np.sin(theta)
        # Rotation matrix
        R = np.array(((c, -s), (s, c)))

        fig = plt.figure()
        axes = fig.add_subplot(111)
        axes.set_aspect(1)
        PlotTrials(currentTrial = currentTrial,
                      num_trials = num_trials, 
                      trail_color = trail_color, # "orange" or "viridis" or None
                      show_reward_boundary = True,
                      show_stops = True,
                      show_colorbar = False, 
                      show_believed_target_positions = True,
                      #show_connect_path_target = np.arange(currentTrial-num_trials+1, currentTrial+1),
                      #show_connect_path_pre_target = np.arange(currentTrial-num_trials+1, currentTrial+1),
                      #show_connect_path_ff = np.arange(currentTrial-num_trials+1, currentTrial+1),
                      #trial_to_show_cluster = -1, 
                      show_scale_bar = True,
                      )
        

        if images_dir != None:
          filename = "data" + str(data_num) + "_trial_" + str(currentTrial)
          CHECK_FOLDER = os.path.isdir(images_dir)
          if not CHECK_FOLDER:
              os.makedirs(images_dir)
          #axes.set_title(f"Trial {currentTrial}", fontsize = 22)
          #plt.savefig(f"{images_dir}/{filename}.png")
        plt.show()  
        plt.close()